# **ОПЦИОНЫ**

---



# Установка дополнительных библиотек, начальная настройка











In [1]:
# Установка пакетов
!pip install tinkoff-investments
!pip install loguru

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.2/227.2 kB 971.9 kB/s eta 0:00:00m eta 0:00:010:01:01
  Using cached cachetools-5.5.0-py3-none-any.whl (9.5 kB)
  Using cached deprecation-2.1.0-py2.py3-none-any.whl (11 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 2.1 MB/s eta 0:00:00m eta 0:00:010:01:01
  Using cached protobuf-4.25.5-py3-none-any.whl (156 kB)
  Using cached tinkoff-0.1.1-py3-none-any.whl (1.0 kB)
  Using cached loguru-0.7.2-py3-none-any.whl (62 kB)


In [2]:
!pip install python-dotenv

  Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)


In [3]:
!pip install bokeh

  Using cached bokeh-3.6.0-py3-none-any.whl (6.9 MB)
  Using cached contourpy-1.3.0-cp311-cp311-linux_i686.whl
  Using cached xyzservices-2024.9.0-py3-none-any.whl (85 kB)


In [4]:
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 812.2 kB/s eta 0:00:000:00:01m eta 0:00:01


In [1]:
# Активация Bokeh
from bokeh.io import output_notebook
# библиотеки
# ...
# код для графика
# ...
# активация
output_notebook()
# показ - show

Loading BokehJS ...

# ВЫГРУЗКА ОБЩИХ ДАННЫХ

---



# Выгрузка данных по оционам и их БА в таблицы

Выгружаются последние актуальные данные БА, пока только акций, и их опционов

In [5]:
import os

import pandas as pd

import datetime
import time

from math import floor, log10

from tinkoff.invest import AsyncClient
from tinkoff.invest.constants import INVEST_GRPC_API_SANDBOX
from tinkoff.invest.schemas import (
    Deviation,
    GetTechAnalysisRequest,
    IndicatorInterval,
    IndicatorType,
    Smoothing,
    TypeOfPrice,
)

from loguru import logger

from dotenv import load_dotenv

# Словарь данных по всем опционам
dict_all_options = {
    'Название': [],
    'Цена страйка': [],
    'UID': [],
    'Основной актив': [],
    'Дата начала обращения контракта': [],
    'Дата истечения срока': [],
    'Дата исполнения': [],
}

# Словарь данных по БА(только акции)
dict_BA_data = {
    'Название': [],
    'UID': [],
    'Лотность': [],
    'FIGI': [],
    'Тикер': [],
    'Цена последней сделки': [],
    'Время получения последней цены': [],
    'Тип последней цены': [],
}

# Словарь полного фрейма по БА(только акции)
dict_full_BA_share = {}

# Токен санбокс т-инвестиции
load_dotenv()
TOKEN_TIN_SanBox = os.getenv("TOKEN_TIN_SanBox")

# Пеобразование тин-чисел в флоат
def format_T_num_in_float(units, nano):
    if nano < 0:
        nano = abs(nano)
    if nano == 0:
        return float(units)
    return units + nano * 10**-(floor(log10(nano))+1)

# Получение всех опционов
async def show_options():
    try:
        async with AsyncClient(TOKEN_TIN_SanBox,
                               target=INVEST_GRPC_API_SANDBOX) as client:
            all_options = await client.instruments.options()
            return all_options
    except Exception as e:
        logger.debug(
            f"Тип исключения: {type(e).__name__}, сообщение: {str(e)}"
        )
        return None
# Получение данных опциона по UID
async def get_data_sing_option(uid='uid'):
    try:
        async with AsyncClient(TOKEN_TIN_SanBox,
                               target=INVEST_GRPC_API_SANDBOX) as client:
            data_option = await client.instruments.option_by(
                id_type=3,
                id=uid
            )
            return data_option.instrument
    except Exception as e:
        logger.debug(
            f"Тип исключения: {type(e).__name__}, сообщение: {str(e)}"
        )
        return None
# Получение данных БА по его тикеру для перебора класс кода(толко по акциям)
async def get_share_data(tiker_='tiker', class_code_='class_code'):
    try:
        async with AsyncClient(TOKEN_TIN_SanBox,
                               target=INVEST_GRPC_API_SANDBOX) as client:
            data_share = await client.instruments.share_by(
                id_type=2,
                class_code=class_code_,
                id=tiker_
            )
            return data_share
    except Exception as e:
        # logger.debug('Не угадали с "Класс-код (секция торгов)"')
        return None
# Перебор класс кода и выдача окончательного результата(толко по акциям)
async def get_share_data_cd(tiker_='tiker'):
    class_codes = ['MTQR', 'SPBRU', 'TQBR', 'TQPI']
    res = None
    try:
        for class_code in class_codes:
            share_data = await get_share_data(tiker_=tiker_, class_code_=class_code)
            if share_data != None:
                res = share_data
        return res.instrument
    except Exception as e:
        logger.debug(
            f"Тип исключения: {type(e).__name__}, сообщение: {str(e)}"
        )
        return None
# Получение последней цены БА(толко по акциям)
async def get_share_price(figi='FIGI', instrument_id="UID"):
    try:
        async with AsyncClient(TOKEN_TIN_SanBox,
                               target=INVEST_GRPC_API_SANDBOX) as client:
            last_prices_array = await client.market_data.get_last_prices(
                figi={figi}, instrument_id={instrument_id}
            )
            return last_prices_array.last_prices[0]
    except Exception as e:
        logger.debug(
            f"Тип исключения: {type(e).__name__}, сообщение: {str(e)}"
        )
        return None

# Получение данных по всем опционам
data_all_options = await show_options()
for option in data_all_options.instruments:
    # Выборка только PUT опционов
    if option.name:
        dict_all_options['Название'].append(option.name)
        dict_all_options['UID'].append(option.uid)
        # Преобразование числа во флоат
        strike_price_float = format_T_num_in_float(option.strike_price.units, option.strike_price.nano)
        # И его запись
        dict_all_options['Цена страйка'].append(strike_price_float)
        dict_all_options['Основной актив'].append(option.basic_asset)
        # Переводим время в локальное без привязки к UTC
        local_dt_1 = option.first_trade_date - datetime.timedelta(seconds=time.timezone)
        local_dt_2 = option.expiration_date - datetime.timedelta(seconds=time.timezone)
        local_dt_3 = option.last_trade_date - datetime.timedelta(seconds=time.timezone)
        # Убираем метки часового пояса и записываем в словарь
        dict_all_options['Дата начала обращения контракта'].append(local_dt_1.replace(tzinfo=None))
        dict_all_options['Дата истечения срока'].append(local_dt_2.replace(tzinfo=None))
        dict_all_options['Дата исполнения'].append(local_dt_3.replace(tzinfo=None))
# Создание pd фрейма с данными опционов
df_all_options = pd.DataFrame(dict_all_options)
# Выгрузка pd фрейма с данными опционов в таблицу
df_all_options.to_excel(r'all_options.xlsx')
# Список тикеров БА
lst_BA_tiker = dict_all_options['Основной актив']
# Множество тикеров БА
set_BA_tiker = set(lst_BA_tiker)

# Вывод данных опциона
logger.debug(f"\n{df_all_options['Основной актив']}")

# Получение данных одного опциона
# data_one_option = await get_data_sing_option(uid='a868c217-7232-46c2-ab55-237950300525')
# dict_one_option['Название'].append(data_one_option.name)
# dict_one_option['UID'].append(data_one_option.uid)
# dict_one_option['Цена страйка'].append(data_one_option.strike_price)
# dict_one_option['Основной актив'].append(data_one_option.basic_asset)
# dict_one_option['Дата начала обращения контракта'].append(data_one_option.first_trade_date)
# dict_one_option['Дата истечения срока'].append(data_one_option.expiration_date)
# dict_one_option['Дата исполнения'].append(data_one_option.last_trade_date)
# df_one_option = pd.DataFrame(dict_one_option)
# logger.debug(f'\n{df_one_option}')

# Получение данных БА(только по акциям)
# data_share = await get_share_data_cd(tiker_='SBER')
# last_prices_share = await get_share_price(figi=data_share.figi, instrument_id=data_share.uid)
# dict_BA_data['Название'].append(data_share.name)
# dict_BA_data['UID'].append(data_share.uid)
# dict_BA_data['FIGI'].append(data_share.figi)
# dict_BA_data['Лотность'].append(data_share.lot)
# dict_BA_data['Цена последней сделки'].append(last_prices_share.price)
# dict_BA_data['Время получения последней цены'].append(last_prices_share.time)
# dict_BA_data['Тип последней цены'].append(last_prices_share.last_price_type)
# df_share_data = pd.DataFrame(dict_BA_data)
# logger.debug(f'\n{df_share_data}')

# По уникальным тикерам получаем данные БА(акции)
for ba in set_BA_tiker:
    data_share = await get_share_data_cd(tiker_=ba)
    if data_share != None:
        last_prices_share = await get_share_price(figi=data_share.figi, instrument_id=data_share.uid)
        dict_BA_data['Название'].append(data_share.name)
        dict_BA_data['UID'].append(data_share.uid)
        dict_BA_data['FIGI'].append(data_share.figi)
        dict_BA_data['Тикер'].append(data_share.ticker)
        dict_BA_data['Лотность'].append(data_share.lot)
        # Преобразование числа во флоат
        last_price_float = format_T_num_in_float(last_prices_share.price.units, last_prices_share.price.nano)
        # И его запись
        dict_BA_data['Цена последней сделки'].append(last_price_float)
        # Переводим время в локальное без привязки к UTC
        local_dt_price = last_prices_share.time - datetime.timedelta(seconds=time.timezone)
        # И его запись без метки часового пояса
        dict_BA_data['Время получения последней цены'].append(local_dt_price.replace(tzinfo=None))
        dict_BA_data['Тип последней цены'].append(last_prices_share.last_price_type)
    else:
        logger.debug(f"\n{ba} - Это не акция")
df_share_data = pd.DataFrame(dict_BA_data)
# Выгрузка pd фрейма с данными акций в таблицу
df_share_data.to_excel(r'all_shares.xlsx')

# logger.debug(f'\n{df_share_data}')
# logger.debug(f'\n{df_all_options}')

# Сборка окончательного фрейма
# i = 0
# for share in dict_BA_data['Название']:
#     dict_full_BA_share[share] = {share: {}}
#     dict_full_BA_share[share][share]['Название'] = dict_BA_data['Название'][i]
#     dict_full_BA_share[share][share]['UID'] = dict_BA_data['UID'][i]
#     dict_full_BA_share[share][share]['FIGI'] = dict_BA_data['FIGI'][i]
#     dict_full_BA_share[share][share]['Тикер'] = dict_BA_data['Тикер'][i]
#     dict_full_BA_share[share][share]['Лотность'] = dict_BA_data['Лотность'][i]
#     dict_full_BA_share[share][share]['Цена последней сделки'] = dict_BA_data['Цена последней сделки'][i]
#     dict_full_BA_share[share][share]['Время получения последней цены'] = dict_BA_data['Время получения последней цены'][i]
#     dict_full_BA_share[share][share]['Тип последней цены'] = dict_BA_data['Тип последней цены'][i]
#     i += 1

# for show, vol in dict_full_BA_share.items():
#     logger.debug(f'\n{show}:')
#     for vol1, show1 in vol.items():
#         logger.debug(f'\n{vol1}:')
#         for show2, vol2 in show1.items():
#             logger.debug(f'\n{show2} -  {vol2}')


/tmp/ipykernel_5913/427619739.py:68: DeprecatedWarning: options is deprecated. Use `Client.instruments.options_by(...)` method instead
  all_options = await client.instruments.options()
2024-11-16 13:47:36.629 | DEBUG    | __main__:<module>:164 - 
0         VKCO
1         ROSN
2         ROSN
3         ABIO
4         GAZP
         ...  
11967     ROSN
11968     SBER
11969     ASTR
11970     MOEX
11971    TATNP
Name: Основной актив, Length: 11972, dtype: object
3d8baa13f7e3fd1ae575dfb005d286c6 ShareBy NOT_FOUND 50002
084bc0e58fe802dff20563c8ba52c443 ShareBy NOT_FOUND 50002
78430f9bfd4d75d1a032dcaf466ee296 ShareBy NOT_FOUND 50002
302fb6c5feda2612fbe6b382b48ab3d2 ShareBy NOT_FOUND 50002
531331113532f86a3d0f16d9b1aa83f1 ShareBy NOT_FOUND 50002
c1263a118da674ad4273f9cf8df47a11 ShareBy NOT_FOUND 50002
34b665bacef452d63aac7acc7dc26bc3 ShareBy NOT_FOUND 50002
0c0785fea18a4a462ef737f668e7babf ShareBy NOT_FOUND 50002
95c6280a3ec4c3c9340f9830d4c631f8 ShareBy NOT_FOUND 50002
f95c7571db829d1b2cb8e4a

# Консолидация данных БА с их опционами

Предворительный просмотр возможных вариантов, в строке 4 задается дата по которую выводить опционы, в строке 11 фильтруем цену лота БА, в строке 19 фильтруем вид опциона

In [3]:
# Просмотр содержимого таблиц
import pandas as pd
# Временная глубина выгрузки опционов
temp_opt_unload = pd.Timestamp(2024, 11, 15, 23, 59)
# Выгрузка данных
df_opt = pd.read_excel(io='all_options.xlsx', engine='openpyxl')
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
# Вывод названия БА
for i in df_share['Unnamed: 0']:
    # Фильтр на цену лота
    if (df_share['Цена последней сделки'][i] * df_share['Лотность'][i]) <= 1500:
        print(f"{df_share['Название'][i]:^100}\n\n{df_share['Тикер'][i]:<20}\tЦена последней сделки\tакции: {df_share['Цена последней сделки'][i]} рублей\tлота: {df_share['Цена последней сделки'][i] * df_share['Лотность'][i]} рублей\n\n{'------------ОПЦИОНЫ------------':^100}\n")
        # Вывод опционнов по БА
        for i_opt in df_opt['Unnamed: 0']:
            if df_opt['Основной актив'][i_opt] == df_share['Тикер'][i]:
                # Фильтр по дате
                if df_opt['Дата исполнения'][i_opt] <= temp_opt_unload:
                    # Фильтыр на PUT
                    if 'PUT' in df_opt['Название'][i_opt]:
                        print(f"{df_opt['Основной актив'][i_opt]:<20}\t{df_opt['Дата начала обращения контракта'][i_opt]}\t{df_opt['Дата исполнения'][i_opt]}\t{df_opt['Название'][i_opt]}")
        print(f"\n{'*************************************************':^100}\n")

                                                ПИК                                                 

PIKK                	Цена последней сделки	акции: 497.8 рублей	лота: 497.8 рублей

                                  ------------ОПЦИОНЫ------------                                   

PIKK                	2024-10-08 11:00:00	2024-10-30 20:05:00	ПИК PUT 500₽ 30.10
PIKK                	2024-10-29 11:00:00	2024-11-13 20:05:00	ПИК PUT 500₽ 13.11
PIKK                	2024-10-22 11:00:00	2024-11-06 20:05:00	ПИК PUT 550₽ 06.11
PIKK                	2024-11-01 11:00:00	2024-11-13 20:05:00	ПИК PUT 400₽ 13.11
PIKK                	2024-10-29 11:00:00	2024-11-13 20:05:00	ПИК PUT 475₽ 13.11
PIKK                	2024-10-22 11:00:00	2024-11-06 20:05:00	ПИК PUT 675₽ 06.11
PIKK                	2024-10-22 11:00:00	2024-11-06 20:05:00	ПИК PUT 500₽ 06.11
PIKK                	2024-10-22 11:00:00	2024-11-06 20:05:00	ПИК PUT 600₽ 06.11
PIKK                	2024-10-08 11:00:00	2024-10-30 20:05:00	ПИК PUT 700₽

# ПОДБОР ОПЦИОНОВ

---



# Выгрузка графика по одному БА и возможных опционов

В строке 89 вводим тикер БА, в строках 114 и 118 фильтруем опционы

In [5]:
# Получение исторических свеч за год
import os

import pandas as pd

from datetime import timedelta, datetime
import time

from math import floor, log10

from tinkoff.invest import AsyncClient
from tinkoff.invest.constants import INVEST_GRPC_API_SANDBOX
from tinkoff.invest.schemas import (
    Deviation,
    GetTechAnalysisRequest,
    IndicatorInterval,
    CandleInterval,
    CandleSource,
    IndicatorType,
    Smoothing,
    TypeOfPrice,
)
from tinkoff.invest.utils import decimal_to_quotation, now

from dotenv import load_dotenv

from bokeh.models import HoverTool
from bokeh.plotting import figure, show
# Активация Bokeh
from bokeh.io import output_notebook, curdoc

# Токен санбокс т-инвестиции
load_dotenv()
TOKEN_TIN_SanBox = os.getenv("TOKEN_TIN_SanBox")

# Пеобразование тин-чисел в флоат
def format_T_num_in_float(units, nano):
    if nano < 0:
        nano = abs(nano)
    if nano == 0:
        return float(units)
    return units + nano * 10**-(floor(log10(nano))+1)

# интервал - CandleInterval.CANDLE_INTERVAL_DAY в данном случае 1 день
# период в днях 1 год = 365 дней

# Запрос свечей по одному БА
async def get_history_candles(
        period, interval, figi='figi', uid='uid'
):
    try:
        async with AsyncClient(TOKEN_TIN_SanBox,
                               target=INVEST_GRPC_API_SANDBOX) as client:
            historical_candles = await client.market_data.get_candles(
                figi=figi,
                from_=now() - timedelta(days=period),
                to=now(),
                interval=interval,
                instrument_id=uid,
                candle_source_type=CandleSource.CANDLE_SOURCE_EXCHANGE
            )
            historical_candles_array = {
                'Время свечи': [x.time for x in historical_candles.candles],
                'Цена открытия': [x.open for x in historical_candles.candles],
                'Макс цена': [x.high for x in historical_candles.candles],
                'Мин цена': [x.low for x in historical_candles.candles],
                'Цена закрытия': [format_T_num_in_float(units=x.close.units, nano=x.close.nano) for x in historical_candles.candles],
                'Сформированость': [x.is_complete for x in historical_candles.candles],
                'Объём торгов': [x.volume for x in historical_candles.candles],
                # 'Тип источника': [x.candle_source.name for x in historical_candles.candles],
            }
            df = pd.DataFrame.from_dict(historical_candles_array)
            return df
    except Exception as e:
        print(
            f"Тип исключения: {type(e).__name__}, сообщение: {str(e)}"
        )
        return None
# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
# Получить опционы из таблицы
df_opt = pd.read_excel(io='all_options.xlsx', engine='openpyxl')
# Получить индекс БА по тикеру инструмента
def index_by_ticker(tiker='tiker'):
    for tik, ind in zip(df_share['Тикер'], df_share['Unnamed: 0']):
        if tik == tiker:
            return [ind, tik]
# Получить массив исторических свеч по тикеру инструмента tiker='ввести тикер'
ind_tik_BA = (index_by_ticker(tiker='ROSN'))
df_candles_BA = await get_history_candles(365, CandleInterval.CANDLE_INTERVAL_DAY, figi=df_share['FIGI'][ind_tik_BA[0]], uid=df_share['UID'][ind_tik_BA[0]])

# Тема графика
curdoc().theme = "dark_minimal"
# Панель инструментов
TOOLS = "pan,wheel_zoom,box_zoom,reset, crosshair"
# Данные для изображения в hover
data_in_hover = HoverTool(
    tooltips=[
        ( 'Дата',   '@x{%F}'),
        ( 'Цена закрытия',  '@y{%0.2f} руб' ),
    ],
    formatters={
        '@x': 'datetime',
        '@y': 'printf',
    },
)
# Создание поля графика
p = figure(title=df_share['Название'][ind_tik_BA[0]], x_axis_type="datetime", sizing_mode="stretch_width", tools=[data_in_hover, TOOLS])
# Автоматически скрывать панель инструментов на неактивном графике
p.toolbar.autohide = True
# Создание графика исторических свечей
p.line(df_candles_BA['Время свечи'], df_candles_BA['Цена закрытия'], legend_label=str(f"{(df_share['Лотность'][ind_tik_BA[0]] * df_share['Цена последней сделки'][ind_tik_BA[0]]):.2f} рублей за лот"), color="blue", line_width=2)
# Линии опционов
temp_opt_unload = pd.Timestamp(2024, 11, 15, 23, 59)
for i in df_opt['Unnamed: 0']:
    if df_opt['Основной актив'][i] == ind_tik_BA[1]:
        # Фильтр по PUT
        if 'PUT' in df_opt['Название'][i]:
            # Фильтр по дате
            if df_opt['Дата исполнения'][i] <= temp_opt_unload:
                # Линия Страйка
                x = [df_candles_BA['Время свечи'][0], df_opt['Дата исполнения'][i]]
                y = [df_opt['Цена страйка'][i], df_opt['Цена страйка'][i]]
                p.line(x, y, color='red', line_width=0.5)
                # Линия окончания срока
                x_t = [df_opt['Дата исполнения'][i], df_opt['Дата исполнения'][i]]
                y_t = [df_opt['Цена страйка'][i], df_candles_BA['Цена закрытия'].max()]
                p.line(x_t, y_t, color='red', line_width=0.5)

# Активация Bokeh и вывод графика
output_notebook()
show(p)

Loading BokehJS ...

# Выгрузка графика БА с выбранным опционом и определение зоны убыточности



1.   Вводим тикер БА, дата исполнения, текущущую дату(дата покупки), страйки(опциона и нижестоящего), премию(оциона и нижестоящего), комиссию при покупке опциона, цену БА на дату покупки;
2.   Расчитываем дельту;

> В настоящий момент расчёт такой: D = (Цена премии нижестоящего опциона - Цена премии опциона) \ (Цена стайка опциона - Цена страйка нижестоящего опциона)

3.   Расчитаваем цену неубыточности при провале опциона

> В настоящий момент расчет такой: Цена неубыточности = Цена БА на дату покупки + Сумма цена опционов на одну акцию БА + (Цена коммисии опциона * Количество опционов на одну акцию БА)


4.   Получаем месячную историю дневных свечей;
5.   Из истории свечей выделяем последнюю цену закрытия, которую будем принимать за номинальную цену БА;
6.   Расчитываем хедж;

> В данный момент расчет такой: Хедж = Цена номинала БА / Модуль(Дельта), полученное число округляем до целого числа - оно и будет количеством опционов, которое надо приобрести на одну акцию хеджирования

7.   Сторим график: слева хвост в виде месячного графика цен закрытия БА, справа прямоугольник со сторонами(левая - дата покупки, правая - дата исполнения, верхняя - цена неубыточности при провале опциона, нижняя - цена неубыточности опциона).





In [2]:
import pandas as pd

# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
for indx in df_share['Unnamed: 0']:
    if (df_share['Лотность'][indx] * df_share['Цена последней сделки'][indx]) < 600:
        print(f"{df_share['Тикер'][indx]}\t{(df_share['Цена последней сделки'][indx] * df_share['Лотность'][indx]):.2f}\t{df_share['Название'][indx]}")

RUAL	352.25	РУСАЛ
PIKK	490.00	ПИК
TATN	542.60	Татнефть
IRAO	380.80	Интер РАО ЕЭС
MAGN	354.50	Магнитогорский металлургический комбинат
MTLR	98.65	Мечел
ALRS	482.20	АЛРОСА
SNGSP	556.50	Сургутнефтегаз - привилегированные акции
VKCO	287.40	ВК
ROSN	446.85	Роснефть
ASTR	498.00	Группа Астра
TATNP	541.00	Татнефть - привилегированные акции
AFLT	542.80	Аэрофлот
VTBR	75.80	Банк ВТБ


In [3]:
import os

import pandas as pd

from datetime import timedelta, datetime
import time

from math import floor, log10

from tinkoff.invest import AsyncClient
from tinkoff.invest.constants import INVEST_GRPC_API_SANDBOX
from tinkoff.invest.schemas import (
    Deviation,
    GetTechAnalysisRequest,
    IndicatorInterval,
    CandleInterval,
    CandleSource,
    IndicatorType,
    Smoothing,
    TypeOfPrice,
)
from tinkoff.invest.utils import decimal_to_quotation, now

from dotenv import load_dotenv

from bokeh.plotting import figure, show
# Активация Bokeh
from bokeh.io import output_notebook, curdoc

# Токен санбокс т-инвестиции
load_dotenv()
TOKEN_TIN_SanBox = os.getenv("TOKEN_TIN_SanBox")

# Пеобразование тин-чисел в флоат
def format_T_num_in_float(units, nano):
    if nano < 0:
        nano = abs(nano)
    if nano == 0:
        return float(units)
    return units + nano * 10**-(floor(log10(nano))+1)

# Введите данные по условию
# Тикер БА
ticker_BA = 'RTKM'
# Дата исполнения
date_execution = pd.Timestamp(2024, 11, 20)
# Дата покупки(начала пролонгирования)
date_purchase = pd.Timestamp(2024, 10, 30)
# Страйк
option_strike = 69
# Цена нижестоящего страйка
option_low_strike = 66
# Цена премии опциона
premium_price = 7.908
# Цена премии нижестоящего опциона
premium_low_price = 4.5
# Комиссия опциона
commiss_premium = 0.0
# Стоимость БА на момент покупки(пролонгирования)
price_BA_date_purchase = 62.1
# Цена безубыточности опциона
break_even_price_opt = 60.86

# Расчёт дельты
delta_calculation = (premium_low_price - premium_price) / (option_strike - option_low_strike)

# Расчитываем хедж или другими словами количество опционов на одну акцию
hedge_calculating = round(price_BA_date_purchase / (abs(delta_calculation) * 100))

# Запрос месячных свечей
async def get_history_candles(
        period, interval, figi='figi', uid='uid'
):
    try:
        async with AsyncClient(TOKEN_TIN_SanBox,
                               target=INVEST_GRPC_API_SANDBOX) as client:
            historical_candles = await client.market_data.get_candles(
                figi=figi,
                from_=now() - timedelta(days=period),
                to=now(),
                interval=interval,
                instrument_id=uid,
                candle_source_type=CandleSource.CANDLE_SOURCE_EXCHANGE
            )
            historical_candles_array = {
                'Время свечи': [x.time for x in historical_candles.candles],
                'Цена открытия': [x.open for x in historical_candles.candles],
                'Макс цена': [x.high for x in historical_candles.candles],
                'Мин цена': [x.low for x in historical_candles.candles],
                'Цена закрытия': [format_T_num_in_float(units=x.close.units, nano=x.close.nano) for x in historical_candles.candles],
                'Сформированость': [x.is_complete for x in historical_candles.candles],
                'Объём торгов': [x.volume for x in historical_candles.candles],
                'Тип источника': [x.candle_source.name for x in historical_candles.candles],
            }
            df = pd.DataFrame.from_dict(historical_candles_array)
            return df
    except Exception as e:
        print(
            f"Тип исключения: {type(e).__name__}, сообщение: {str(e)}"
        )
        return None
# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
# Получить опционы из таблицы
df_opt = pd.read_excel(io='all_options.xlsx', engine='openpyxl')
# Получить индекс БА по тикеру инструмента
def index_by_ticker(tiker='tiker'):
    for tik, ind in zip(df_share['Тикер'], df_share['Unnamed: 0']):
        if tik == tiker:
            return [ind, tik]
# Получить массив исторических свеч по тикеру инструмента
ind_tik_BA = (index_by_ticker(tiker=ticker_BA))
df_candles_BA = await get_history_candles(365, CandleInterval.CANDLE_INTERVAL_DAY, figi=df_share['FIGI'][ind_tik_BA[0]], uid=df_share['UID'][ind_tik_BA[0]])

# Цена БА последней сделки
price_BA_last_transaction = df_candles_BA['Цена закрытия'][(len(df_candles_BA['Цена закрытия']) - 1)]

# Цена неубыточности БА
break_even_price_BA = price_BA_date_purchase + (premium_price * hedge_calculating) + (commiss_premium * hedge_calculating)

# Тема графика
curdoc().theme = "dark_minimal"

# Создание поля графика
p = figure(title=f"{df_share['Название'][ind_tik_BA[0]]} PUT {option_strike} {date_execution:%d.%m.%Y}", x_axis_type="datetime", sizing_mode="stretch_width")
# деактивация панели инструментов
p.toolbar_location = None
# Создание графика исторических свечей
p.line(df_candles_BA['Время свечи'], df_candles_BA['Цена закрытия'], color="blue", line_width=4, legend_label=f"БА: {df_share['Название'][ind_tik_BA[0]]}")

# Создание графика-box убыточности
# Справа
p.segment(x0=date_purchase, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_BA, legend_label="Дата покупки", line_color="gold", line_dash="8 8", line_width=2)
# Сверху
p.segment(x0=date_purchase, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_BA, legend_label="Цена неубыточности БА", line_color="mediumseagreen", line_dash="8 8", line_width=2)
# Слева
p.segment(x0=date_execution, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_opt, legend_label="Дата исполнения", line_color="navajowhite", line_dash="8 8", line_width=2)
# Снизу
p.segment(x0=date_execution, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_opt, legend_label="Цена неубыточности опциона", line_color="tomato", line_dash="8 8", line_width=2)

# Создание легенды
p.legend.title = f"Данные по опциону:\nЛотность БА: {df_share['Лотность'][ind_tik_BA[0]]}\nЦена БА на на время покупки: {price_BA_date_purchase}\nЦена последней сделки БА: {price_BA_last_transaction}\nДинамика: {((price_BA_last_transaction - price_BA_date_purchase) / price_BA_date_purchase * 100):.2f} %\nХедж: {hedge_calculating} (Количество опционов на один лот)\nЦена одного опциона: {premium_price}\nЦена суммы опционов за хедж: {(hedge_calculating * premium_price):.2f}\nДата покупки опциона: {date_purchase:%d.%m.%Y}\nДата исполнения: {date_execution:%d.%m.%Y}"
p.legend.title_text_font_size = "16px"
p.legend.location = "top_left"
p.legend.title_text_color = 'orange'

# Активация Bokeh и вывод графика
output_notebook()
show(p)

Тип исключения: AttributeError, сообщение: 'HistoricCandle' object has no attribute 'candle_source'


TypeError: 'NoneType' object is not subscriptable

# РАБОТА С ОПЦИОНАМИ

---



# IRAO	6.11.24	Интер РАО ЕЭС

Куплен 1 пут без хеджирования

In [ ]:
import pandas as pd

# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
for indx in df_share['Unnamed: 0']:
    if (df_share['Лотность'][indx] * df_share['Цена последней сделки'][indx]) < 600:
        print(f"{df_share['Тикер'][indx]}\t{(df_share['Цена последней сделки'][indx] * df_share['Лотность'][indx]):.2f}\t{df_share['Название'][indx]}")

IRAO	375.70	Интер РАО ЕЭС
SNGSP	547.70	Сургутнефтегаз - привилегированные акции
ASTR	504.55	Группа Астра
ROSN	437.85	Роснефть
MTLR	96.63	Мечел
TATN	548.00	Татнефть
TATNP	546.10	Татнефть - привилегированные акции
VKCO	297.40	ВК
MAGN	365.45	Магнитогорский металлургический комбинат
VTBR	76.21	Банк ВТБ
RUAL	337.45	РУСАЛ
ALRS	475.20	АЛРОСА
PIKK	569.10	ПИК
AFLT	544.00	Аэрофлот


In [2]:
import os

import pandas as pd

from datetime import timedelta, datetime
import time

from math import floor, log10

from tinkoff.invest import AsyncClient
from tinkoff.invest.constants import INVEST_GRPC_API_SANDBOX
from tinkoff.invest.schemas import (
    Deviation,
    GetTechAnalysisRequest,
    IndicatorInterval,
    CandleInterval,
    CandleSource,
    IndicatorType,
    Smoothing,
    TypeOfPrice,
)
from tinkoff.invest.utils import decimal_to_quotation, now

from dotenv import load_dotenv

from bokeh.plotting import figure, show
# Активация Bokeh
from bokeh.io import output_notebook, curdoc

# Токен санбокс т-инвестиции
load_dotenv()
TOKEN_TIN_SanBox = os.getenv("TOKEN_TIN_SanBox")

# Пеобразование тин-чисел в флоат
def format_T_num_in_float(units, nano):
    if nano < 0:
        nano = abs(nano)
    if nano == 0:
        return float(units)
    return units + nano * 10**-(floor(log10(nano))+1)

# Введите данные по условию
# Тикер БА
ticker_BA = 'IRAO'
# Дата исполнения
date_execution = pd.Timestamp(2024, 11, 6)
# Дата покупки(начала пролонгирования)
date_purchase = pd.Timestamp(2024, 10, 31)
# Страйк
option_strike = 4.2
# Цена нижестоящего страйка
option_low_strike = 4
# Цена премии опциона
premium_price = 0.402
# Цена премии нижестоящего опциона
premium_low_price = 0.212
# Комиссия опциона
commiss_premium = 0.0
# Стоимость БА на момент покупки(пролонгирования)
price_BA_date_purchase = 3.797
# Цена безубыточности опциона
break_even_price_opt = 3.79

# Расчёт дельты
delta_calculation = (premium_low_price - premium_price) / (option_strike - option_low_strike)

# Расчитываем хедж или другими словами количество опционов на одну акцию
hedge_calculating = round(price_BA_date_purchase / (abs(delta_calculation) * 100))

# Запрос месячных свечей
async def get_history_candles(
        period, interval, figi='figi', uid='uid'
):
    try:
        async with AsyncClient(TOKEN_TIN_SanBox,
                               target=INVEST_GRPC_API_SANDBOX) as client:
            historical_candles = await client.market_data.get_candles(
                figi=figi,
                from_=now() - timedelta(days=period),
                to=now(),
                interval=interval,
                instrument_id=uid,
                candle_source_type=CandleSource.CANDLE_SOURCE_EXCHANGE
            )
            historical_candles_array = {
                'Время свечи': [x.time for x in historical_candles.candles],
                'Цена открытия': [x.open for x in historical_candles.candles],
                'Макс цена': [x.high for x in historical_candles.candles],
                'Мин цена': [x.low for x in historical_candles.candles],
                'Цена закрытия': [format_T_num_in_float(units=x.close.units, nano=x.close.nano) for x in historical_candles.candles],
                'Сформированость': [x.is_complete for x in historical_candles.candles],
                'Объём торгов': [x.volume for x in historical_candles.candles],
                # 'Тип источника': [x.candle_source.name for x in historical_candles.candles],
            }
            df = pd.DataFrame.from_dict(historical_candles_array)
            return df
    except Exception as e:
        print(
            f"Тип исключения: {type(e).__name__}, сообщение: {str(e)}"
        )
        return None
# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
# Получить опционы из таблицы
df_opt = pd.read_excel(io='all_options.xlsx', engine='openpyxl')
# Получить индекс БА по тикеру инструмента
def index_by_ticker(tiker='tiker'):
    for tik, ind in zip(df_share['Тикер'], df_share['Unnamed: 0']):
        if tik == tiker:
            return [ind, tik]
# Получить массив исторических свеч по тикеру инструмента
ind_tik_BA = (index_by_ticker(tiker=ticker_BA))
df_candles_BA = await get_history_candles(365, CandleInterval.CANDLE_INTERVAL_DAY, figi=df_share['FIGI'][ind_tik_BA[0]], uid=df_share['UID'][ind_tik_BA[0]])

# Цена БА последней сделки
price_BA_last_transaction = df_candles_BA['Цена закрытия'][(len(df_candles_BA['Цена закрытия']) - 1)]

# Цена неубыточности БА
break_even_price_BA = price_BA_date_purchase + (premium_price * hedge_calculating) + (commiss_premium * hedge_calculating)

# Тема графика
curdoc().theme = "dark_minimal"

# Создание поля графика
p = figure(title=f"{df_share['Название'][ind_tik_BA[0]]} PUT {option_strike} {date_execution:%d.%m.%Y}", x_axis_type="datetime", sizing_mode="stretch_width")
# деактивация панели инструментов
# p.toolbar_location = None
# Создание графика исторических свечей
p.line(df_candles_BA['Время свечи'], df_candles_BA['Цена закрытия'], color="blue", line_width=4, legend_label=f"БА: {df_share['Название'][ind_tik_BA[0]]}")

# Создание графика-box убыточности
# Справа
p.segment(x0=date_purchase, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_BA, legend_label="Дата покупки", line_color="gold", line_dash="8 8", line_width=2)
# Сверху
p.segment(x0=date_purchase, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_BA, legend_label="Цена неубыточности БА", line_color="mediumseagreen", line_dash="8 8", line_width=2)
# Слева
p.segment(x0=date_execution, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_opt, legend_label="Дата исполнения", line_color="navajowhite", line_dash="8 8", line_width=2)
# Снизу
p.segment(x0=date_execution, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_opt, legend_label="Цена неубыточности опциона", line_color="tomato", line_dash="8 8", line_width=2)

# Создание легенды
p.legend.title = f"Данные по опциону:\nЛотность БА: {df_share['Лотность'][ind_tik_BA[0]]}\nЦена БА на на время покупки: {price_BA_date_purchase}\nЦена последней сделки БА: {price_BA_last_transaction}\nДинамика: {((price_BA_last_transaction - price_BA_date_purchase) / price_BA_date_purchase * 100):.2f} %\nХедж: {hedge_calculating} (Количество опционов на один лот)\nЦена одного опциона: {premium_price}\nЦена суммы опционов за хедж: {(hedge_calculating * premium_price):.2f}\nДата покупки опциона: {date_purchase:%d.%m.%Y}\nДата исполнения: {date_execution:%d.%m.%Y}"
p.legend.title_text_font_size = "16px"
p.legend.location = "top_left"
p.legend.title_text_color = 'orange'
print(break_even_price_opt)
print(break_even_price_BA)
# Активация Bokeh и вывод графика
output_notebook()
show(p)

3.79
3.797


Loading BokehJS ...

/home/sergo/TinvestOptions/venv/lib/python3.11/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


# Сургутнефтегаз - привилегированные акции 6.11.24

Цена колеблеца в обе сстороны: куплен 1 пут и захеджирован одним лотом

In [ ]:
import pandas as pd

# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
for indx in df_share['Unnamed: 0']:
    if (df_share['Лотность'][indx] * df_share['Цена последней сделки'][indx]) < 600:
        print(f"{df_share['Тикер'][indx]}\t{(df_share['Цена последней сделки'][indx] * df_share['Лотность'][indx]):.2f}\t{df_share['Название'][indx]}")

IRAO	375.70	Интер РАО ЕЭС
SNGSP	547.70	Сургутнефтегаз - привилегированные акции
ASTR	504.55	Группа Астра
ROSN	437.85	Роснефть
MTLR	96.63	Мечел
TATN	548.00	Татнефть
TATNP	546.10	Татнефть - привилегированные акции
VKCO	297.40	ВК
MAGN	365.45	Магнитогорский металлургический комбинат
VTBR	76.21	Банк ВТБ
RUAL	337.45	РУСАЛ
ALRS	475.20	АЛРОСА
PIKK	569.10	ПИК
AFLT	544.00	Аэрофлот


In [4]:
import os

import pandas as pd

from datetime import timedelta, datetime
import time

from math import floor, log10

from tinkoff.invest import AsyncClient
from tinkoff.invest.constants import INVEST_GRPC_API_SANDBOX
from tinkoff.invest.schemas import (
    Deviation,
    GetTechAnalysisRequest,
    IndicatorInterval,
    CandleInterval,
    CandleSource,
    IndicatorType,
    Smoothing,
    TypeOfPrice,
)
from tinkoff.invest.utils import decimal_to_quotation, now

from dotenv import load_dotenv

from bokeh.plotting import figure, show
# Активация Bokeh
from bokeh.io import output_notebook, curdoc

# Токен санбокс т-инвестиции
load_dotenv()
TOKEN_TIN_SanBox = os.getenv("TOKEN_TIN_SanBox")

# Пеобразование тин-чисел в флоат
def format_T_num_in_float(units, nano):
    if nano < 0:
        nano = abs(nano)
    if nano == 0:
        return float(units)
    return units + nano * 10**-(floor(log10(nano))+1)

# Введите данные по условию
# Тикер БА
ticker_BA = 'SNGSP'
# Дата исполнения
date_execution = pd.Timestamp(2024, 11, 6)
# Дата покупки(начала пролонгирования)
date_purchase = pd.Timestamp(2024, 10, 31)
# Страйк
option_strike = 56
# Цена нижестоящего страйка
option_low_strike = 54
# Цена премии опциона
premium_price = 1.92
# Цена премии нижестоящего опциона
premium_low_price = 0.76
# Комиссия опциона
commiss_premium = 0.0
# Стоимость БА на момент покупки(пролонгирования)
price_BA_date_purchase = 54.67
# Цена безубыточности опциона
break_even_price_opt = 54.02

# Расчёт дельты
delta_calculation = (premium_low_price - premium_price) / (option_strike - option_low_strike)

# Расчитываем хедж или другими словами количество опционов на одну акцию
hedge_calculating = round(price_BA_date_purchase / (abs(delta_calculation) * 100))

# Запрос месячных свечей
async def get_history_candles(
        period, interval, figi='figi', uid='uid'
):
    try:
        async with AsyncClient(TOKEN_TIN_SanBox,
                               target=INVEST_GRPC_API_SANDBOX) as client:
            historical_candles = await client.market_data.get_candles(
                figi=figi,
                from_=now() - timedelta(days=period),
                to=now(),
                interval=interval,
                instrument_id=uid,
                candle_source_type=CandleSource.CANDLE_SOURCE_EXCHANGE
            )
            historical_candles_array = {
                'Время свечи': [x.time for x in historical_candles.candles],
                'Цена открытия': [x.open for x in historical_candles.candles],
                'Макс цена': [x.high for x in historical_candles.candles],
                'Мин цена': [x.low for x in historical_candles.candles],
                'Цена закрытия': [format_T_num_in_float(units=x.close.units, nano=x.close.nano) for x in historical_candles.candles],
                'Сформированость': [x.is_complete for x in historical_candles.candles],
                'Объём торгов': [x.volume for x in historical_candles.candles],
                # 'Тип источника': [x.candle_source.name for x in historical_candles.candles],
            }
            df = pd.DataFrame.from_dict(historical_candles_array)
            return df
    except Exception as e:
        print(
            f"Тип исключения: {type(e).__name__}, сообщение: {str(e)}"
        )
        return None
# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
# Получить опционы из таблицы
df_opt = pd.read_excel(io='all_options.xlsx', engine='openpyxl')
# Получить индекс БА по тикеру инструмента
def index_by_ticker(tiker='tiker'):
    for tik, ind in zip(df_share['Тикер'], df_share['Unnamed: 0']):
        if tik == tiker:
            return [ind, tik]
# Получить массив исторических свеч по тикеру инструмента
ind_tik_BA = (index_by_ticker(tiker=ticker_BA))
df_candles_BA = await get_history_candles(365, CandleInterval.CANDLE_INTERVAL_DAY, figi=df_share['FIGI'][ind_tik_BA[0]], uid=df_share['UID'][ind_tik_BA[0]])

# Цена БА последней сделки
price_BA_last_transaction = df_candles_BA['Цена закрытия'][(len(df_candles_BA['Цена закрытия']) - 1)]

# Цена неубыточности БА
break_even_price_BA = price_BA_date_purchase + (premium_price * hedge_calculating) + (commiss_premium * hedge_calculating)

# Тема графика
curdoc().theme = "dark_minimal"

# Создание поля графика
p = figure(title=f"{df_share['Название'][ind_tik_BA[0]]} PUT {option_strike} {date_execution:%d.%m.%Y}", x_axis_type="datetime", sizing_mode="stretch_width")
# деактивация панели инструментов
# p.toolbar_location = None
# Создание графика исторических свечей
p.line(df_candles_BA['Время свечи'], df_candles_BA['Цена закрытия'], color="blue", line_width=4, legend_label=f"БА: {df_share['Название'][ind_tik_BA[0]]}")

# Создание графика-box убыточности
# Справа
p.segment(x0=date_purchase, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_BA, legend_label="Дата покупки", line_color="gold", line_dash="8 8", line_width=2)
# Сверху
p.segment(x0=date_purchase, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_BA, legend_label="Цена неубыточности БА", line_color="mediumseagreen", line_dash="8 8", line_width=2)
# Слева
p.segment(x0=date_execution, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_opt, legend_label="Дата исполнения", line_color="navajowhite", line_dash="8 8", line_width=2)
# Снизу
p.segment(x0=date_execution, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_opt, legend_label="Цена неубыточности опциона", line_color="tomato", line_dash="8 8", line_width=2)

# Создание легенды
p.legend.title = f"Данные по опциону:\nЛотность БА: {df_share['Лотность'][ind_tik_BA[0]]}\nЦена БА на на время покупки: {price_BA_date_purchase}\nЦена последней сделки БА: {price_BA_last_transaction}\nДинамика: {((price_BA_last_transaction - price_BA_date_purchase) / price_BA_date_purchase * 100):.2f} %\nХедж: {hedge_calculating} (Количество опционов на один лот)\nЦена одного опциона: {premium_price}\nЦена суммы опционов за хедж: {(hedge_calculating * premium_price * df_share['Лотность'][ind_tik_BA[0]]):.2f}\nДата покупки опциона: {date_purchase:%d.%m.%Y}\nДата исполнения: {date_execution:%d.%m.%Y}"
p.legend.title_text_font_size = "16px"
p.legend.location = "top_left"
p.legend.title_text_color = 'orange'

print(break_even_price_opt)
print(break_even_price_BA)

# Активация Bokeh и вывод графика
output_notebook()
show(p)

54.02
56.59


Loading BokehJS ...

/home/sergo/TinvestOptions/venv/lib/python3.11/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


# Мечел 20.11.24

пролонгирование до 20 ноября

In [ ]:
import pandas as pd

# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
for indx in df_share['Unnamed: 0']:
    if (df_share['Лотность'][indx] * df_share['Цена последней сделки'][indx]) < 600:
        print(f"{df_share['Тикер'][indx]}\t{(df_share['Цена последней сделки'][indx] * df_share['Лотность'][indx]):.2f}\t{df_share['Название'][indx]}")

IRAO	375.70	Интер РАО ЕЭС
SNGSP	547.70	Сургутнефтегаз - привилегированные акции
ASTR	504.55	Группа Астра
ROSN	437.85	Роснефть
MTLR	96.63	Мечел
TATN	548.00	Татнефть
TATNP	546.10	Татнефть - привилегированные акции
VKCO	297.40	ВК
MAGN	365.45	Магнитогорский металлургический комбинат
VTBR	76.21	Банк ВТБ
RUAL	337.45	РУСАЛ
ALRS	475.20	АЛРОСА
PIKK	569.10	ПИК
AFLT	544.00	Аэрофлот


In [5]:
import os

import pandas as pd

from datetime import timedelta, datetime
import time

from math import floor, log10

from tinkoff.invest import AsyncClient
from tinkoff.invest.constants import INVEST_GRPC_API_SANDBOX
from tinkoff.invest.schemas import (
    Deviation,
    GetTechAnalysisRequest,
    IndicatorInterval,
    CandleInterval,
    CandleSource,
    IndicatorType,
    Smoothing,
    TypeOfPrice,
)
from tinkoff.invest.utils import decimal_to_quotation, now

from dotenv import load_dotenv

from bokeh.plotting import figure, show
# Активация Bokeh
from bokeh.io import output_notebook, curdoc

# Токен санбокс т-инвестиции
load_dotenv()
TOKEN_TIN_SanBox = os.getenv("TOKEN_TIN_SanBox")

# Пеобразование тин-чисел в флоат
def format_T_num_in_float(units, nano):
    if nano < 0:
        nano = abs(nano)
    if nano == 0:
        return float(units)
    return units + nano * 10**-(floor(log10(nano))+1)

# Введите данные по условию
# Тикер БА
ticker_BA = 'MTLR'
# Дата исполнения
date_execution = pd.Timestamp(2024, 11, 20)
# Дата покупки(начала пролонгирования)
date_purchase = pd.Timestamp(2024, 10, 31)
# Страйк
option_strike = 120
# Цена нижестоящего страйка
option_low_strike = 110
# Цена премии опциона
premium_price = 21.57
# Цена премии нижестоящего опциона
premium_low_price = 14.05
# Комиссия опциона
commiss_premium = 0.0
# Стоимость БА на момент покупки(пролонгирования)
price_BA_date_purchase = 97.68
# Цена безубыточности опциона
break_even_price_opt = 97.78

# Расчёт дельты
delta_calculation = (premium_low_price - premium_price) / (option_strike - option_low_strike)

# Расчитываем хедж или другими словами количество опционов на одну акцию
hedge_calculating = round(price_BA_date_purchase / (abs(delta_calculation) * 100))

# Запрос месячных свечей
async def get_history_candles(
        period, interval, figi='figi', uid='uid'
):
    try:
        async with AsyncClient(TOKEN_TIN_SanBox,
                               target=INVEST_GRPC_API_SANDBOX) as client:
            historical_candles = await client.market_data.get_candles(
                figi=figi,
                from_=now() - timedelta(days=period),
                to=now(),
                interval=interval,
                instrument_id=uid,
                candle_source_type=CandleSource.CANDLE_SOURCE_EXCHANGE
            )
            historical_candles_array = {
                'Время свечи': [x.time for x in historical_candles.candles],
                'Цена открытия': [x.open for x in historical_candles.candles],
                'Макс цена': [x.high for x in historical_candles.candles],
                'Мин цена': [x.low for x in historical_candles.candles],
                'Цена закрытия': [format_T_num_in_float(units=x.close.units, nano=x.close.nano) for x in historical_candles.candles],
                'Сформированость': [x.is_complete for x in historical_candles.candles],
                'Объём торгов': [x.volume for x in historical_candles.candles],
                # 'Тип источника': [x.candle_source.name for x in historical_candles.candles],
            }
            df = pd.DataFrame.from_dict(historical_candles_array)
            return df
    except Exception as e:
        print(
            f"Тип исключения: {type(e).__name__}, сообщение: {str(e)}"
        )
        return None
# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
# Получить опционы из таблицы
df_opt = pd.read_excel(io='all_options.xlsx', engine='openpyxl')
# Получить индекс БА по тикеру инструмента
def index_by_ticker(tiker='tiker'):
    for tik, ind in zip(df_share['Тикер'], df_share['Unnamed: 0']):
        if tik == tiker:
            return [ind, tik]
# Получить массив исторических свеч по тикеру инструмента
ind_tik_BA = (index_by_ticker(tiker=ticker_BA))
df_candles_BA = await get_history_candles(365, CandleInterval.CANDLE_INTERVAL_DAY, figi=df_share['FIGI'][ind_tik_BA[0]], uid=df_share['UID'][ind_tik_BA[0]])

# Цена БА последней сделки
price_BA_last_transaction = df_candles_BA['Цена закрытия'][(len(df_candles_BA['Цена закрытия']) - 1)]

# Цена неубыточности БА
break_even_price_BA = price_BA_date_purchase + (premium_price * hedge_calculating) + (commiss_premium * hedge_calculating)

# Тема графика
curdoc().theme = "dark_minimal"

# Создание поля графика
p = figure(title=f"{df_share['Название'][ind_tik_BA[0]]} PUT {option_strike} {date_execution:%d.%m.%Y}", x_axis_type="datetime", sizing_mode="stretch_width")
# деактивация панели инструментов
# p.toolbar_location = None
# Создание графика исторических свечей
p.line(df_candles_BA['Время свечи'], df_candles_BA['Цена закрытия'], color="blue", line_width=4, legend_label=f"БА: {df_share['Название'][ind_tik_BA[0]]}")

# Создание графика-box убыточности
# Справа
p.segment(x0=date_purchase, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_BA, legend_label="Дата покупки", line_color="gold", line_dash="8 8", line_width=2)
# Сверху
p.segment(x0=date_purchase, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_BA, legend_label="Цена неубыточности БА", line_color="mediumseagreen", line_dash="8 8", line_width=2)
# Слева
p.segment(x0=date_execution, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_opt, legend_label="Дата исполнения", line_color="navajowhite", line_dash="8 8", line_width=2)
# Снизу
p.segment(x0=date_execution, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_opt, legend_label="Цена неубыточности опциона", line_color="tomato", line_dash="8 8", line_width=2)

# Создание легенды
p.legend.title = f"Данные по опциону:\nЛотность БА: {df_share['Лотность'][ind_tik_BA[0]]}\nЦена БА на на время покупки: {price_BA_date_purchase}\nЦена последней сделки БА: {price_BA_last_transaction}\nДинамика: {((price_BA_last_transaction - price_BA_date_purchase) / price_BA_date_purchase * 100):.2f} %\nХедж: {hedge_calculating} (Количество опционов на один лот)\nЦена одного опциона: {premium_price}\nЦена суммы опционов за хедж: {(hedge_calculating * premium_price * df_share['Лотность'][ind_tik_BA[0]]):.2f}\nДата покупки опциона: {date_purchase:%d.%m.%Y}\nДата исполнения: {date_execution:%d.%m.%Y}"
p.legend.title_text_font_size = "16px"
p.legend.location = "top_left"
p.legend.title_text_color = 'orange'
print(break_even_price_opt)
print(break_even_price_BA)
# Активация Bokeh и вывод графика
output_notebook()
show(p)

97.78
119.25


Loading BokehJS ...

/home/sergo/TinvestOptions/venv/lib/python3.11/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


# РУСАЛ 6.11.24

ожидаю дальнейшего падения взято 3 пут без хеджа

In [ ]:
import pandas as pd

# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
for indx in df_share['Unnamed: 0']:
    if (df_share['Лотность'][indx] * df_share['Цена последней сделки'][indx]) < 600:
        print(f"{df_share['Тикер'][indx]}\t{(df_share['Цена последней сделки'][indx] * df_share['Лотность'][indx]):.2f}\t{df_share['Название'][indx]}")

IRAO	375.70	Интер РАО ЕЭС
SNGSP	547.70	Сургутнефтегаз - привилегированные акции
ASTR	504.55	Группа Астра
ROSN	437.85	Роснефть
MTLR	96.63	Мечел
TATN	548.00	Татнефть
TATNP	546.10	Татнефть - привилегированные акции
VKCO	297.40	ВК
MAGN	365.45	Магнитогорский металлургический комбинат
VTBR	76.21	Банк ВТБ
RUAL	337.45	РУСАЛ
ALRS	475.20	АЛРОСА
PIKK	569.10	ПИК
AFLT	544.00	Аэрофлот


In [6]:
import os

import pandas as pd

from datetime import timedelta, datetime
import time

from math import floor, log10

from tinkoff.invest import AsyncClient
from tinkoff.invest.constants import INVEST_GRPC_API_SANDBOX
from tinkoff.invest.schemas import (
    Deviation,
    GetTechAnalysisRequest,
    IndicatorInterval,
    CandleInterval,
    CandleSource,
    IndicatorType,
    Smoothing,
    TypeOfPrice,
)
from tinkoff.invest.utils import decimal_to_quotation, now

from dotenv import load_dotenv

from bokeh.plotting import figure, show
# Активация Bokeh
from bokeh.io import output_notebook, curdoc

# Токен санбокс т-инвестиции
load_dotenv()
TOKEN_TIN_SanBox = os.getenv("TOKEN_TIN_SanBox")

# Пеобразование тин-чисел в флоат
def format_T_num_in_float(units, nano):
    if nano < 0:
        nano = abs(nano)
    if nano == 0:
        return float(units)
    return units + nano * 10**-(floor(log10(nano))+1)

# Введите данные по условию
# Тикер БА
ticker_BA = 'RUAL'
# Дата исполнения
date_execution = pd.Timestamp(2024, 11, 6)
# Дата покупки(начала пролонгирования)
date_purchase = pd.Timestamp(2024, 10, 31)
# Страйк
option_strike = 37.5
# Цена нижестоящего страйка
option_low_strike = 36
# Цена премии опциона
premium_price = 2.72
# Цена премии нижестоящего опциона
premium_low_price = 1.43
# Комиссия опциона
commiss_premium = 0.0
# Стоимость БА на момент покупки(пролонгирования)
price_BA_date_purchase = 34.79
# Цена безубыточности опциона
break_even_price_opt = 34.7

# Расчёт дельты
delta_calculation = (premium_low_price - premium_price) / (option_strike - option_low_strike)

# Расчитываем хедж или другими словами количество опционов на одну акцию
hedge_calculating = round(price_BA_date_purchase / (abs(delta_calculation) * 100))

# Запрос месячных свечей
async def get_history_candles(
        period, interval, figi='figi', uid='uid'
):
    try:
        async with AsyncClient(TOKEN_TIN_SanBox,
                               target=INVEST_GRPC_API_SANDBOX) as client:
            historical_candles = await client.market_data.get_candles(
                figi=figi,
                from_=now() - timedelta(days=period),
                to=now(),
                interval=interval,
                instrument_id=uid,
                candle_source_type=CandleSource.CANDLE_SOURCE_EXCHANGE
            )
            historical_candles_array = {
                'Время свечи': [x.time for x in historical_candles.candles],
                'Цена открытия': [x.open for x in historical_candles.candles],
                'Макс цена': [x.high for x in historical_candles.candles],
                'Мин цена': [x.low for x in historical_candles.candles],
                'Цена закрытия': [format_T_num_in_float(units=x.close.units, nano=x.close.nano) for x in historical_candles.candles],
                'Сформированость': [x.is_complete for x in historical_candles.candles],
                'Объём торгов': [x.volume for x in historical_candles.candles],
                # 'Тип источника': [x.candle_source.name for x in historical_candles.candles],
            }
            df = pd.DataFrame.from_dict(historical_candles_array)
            return df
    except Exception as e:
        print(
            f"Тип исключения: {type(e).__name__}, сообщение: {str(e)}"
        )
        return None
# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
# Получить опционы из таблицы
df_opt = pd.read_excel(io='all_options.xlsx', engine='openpyxl')
# Получить индекс БА по тикеру инструмента
def index_by_ticker(tiker='tiker'):
    for tik, ind in zip(df_share['Тикер'], df_share['Unnamed: 0']):
        if tik == tiker:
            return [ind, tik]
# Получить массив исторических свеч по тикеру инструмента
ind_tik_BA = (index_by_ticker(tiker=ticker_BA))
df_candles_BA = await get_history_candles(365, CandleInterval.CANDLE_INTERVAL_DAY, figi=df_share['FIGI'][ind_tik_BA[0]], uid=df_share['UID'][ind_tik_BA[0]])

# Цена БА последней сделки
price_BA_last_transaction = df_candles_BA['Цена закрытия'][(len(df_candles_BA['Цена закрытия']) - 1)]

# Цена неубыточности БА
break_even_price_BA = price_BA_date_purchase + (premium_price * hedge_calculating) + (commiss_premium * hedge_calculating)

# Тема графика
curdoc().theme = "dark_minimal"

# Создание поля графика
p = figure(title=f"{df_share['Название'][ind_tik_BA[0]]} PUT {option_strike} {date_execution:%d.%m.%Y}", x_axis_type="datetime", sizing_mode="stretch_width")
# деактивация панели инструментов
# p.toolbar_location = None
# Создание графика исторических свечей
p.line(df_candles_BA['Время свечи'], df_candles_BA['Цена закрытия'], color="blue", line_width=4, legend_label=f"БА: {df_share['Название'][ind_tik_BA[0]]}")

# Создание графика-box убыточности
# Справа
p.segment(x0=date_purchase, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_BA, legend_label="Дата покупки", line_color="gold", line_dash="8 8", line_width=2)
# Сверху
p.segment(x0=date_purchase, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_BA, legend_label="Цена неубыточности БА", line_color="mediumseagreen", line_dash="8 8", line_width=2)
# Слева
p.segment(x0=date_execution, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_opt, legend_label="Дата исполнения", line_color="navajowhite", line_dash="8 8", line_width=2)
# Снизу
p.segment(x0=date_execution, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_opt, legend_label="Цена неубыточности опциона", line_color="tomato", line_dash="8 8", line_width=2)

# Создание легенды
p.legend.title = f"Данные по опциону:\nЛотность БА: {df_share['Лотность'][ind_tik_BA[0]]}\nЦена БА на на время покупки: {price_BA_date_purchase}\nЦена последней сделки БА: {price_BA_last_transaction}\nДинамика: {((price_BA_last_transaction - price_BA_date_purchase) / price_BA_date_purchase * 100):.2f} %\nХедж: {hedge_calculating} (Количество опционов на один лот)\nЦена одного опциона: {premium_price}\nЦена суммы опционов за хедж: {(hedge_calculating * premium_price * df_share['Лотность'][ind_tik_BA[0]]):.2f}\nДата покупки опциона: {date_purchase:%d.%m.%Y}\nДата исполнения: {date_execution:%d.%m.%Y}"
p.legend.title_text_font_size = "16px"
p.legend.location = "top_left"
p.legend.title_text_color = 'orange'
print(break_even_price_opt)
print(break_even_price_BA)
# Активация Bokeh и вывод графика
output_notebook()
show(p)

34.7
34.79


Loading BokehJS ...

/home/sergo/TinvestOptions/venv/lib/python3.11/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


# Аэрофлот 6.11.24

Куплен 1 опцион и 11 акция для хеджа

In [ ]:
import pandas as pd

# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
for indx in df_share['Unnamed: 0']:
    if (df_share['Лотность'][indx] * df_share['Цена последней сделки'][indx]) < 600:
        print(f"{df_share['Тикер'][indx]}\t{(df_share['Цена последней сделки'][indx] * df_share['Лотность'][indx]):.2f}\t{df_share['Название'][indx]}")

IRAO	375.70	Интер РАО ЕЭС
SNGSP	547.70	Сургутнефтегаз - привилегированные акции
ASTR	504.55	Группа Астра
ROSN	437.85	Роснефть
MTLR	96.63	Мечел
TATN	548.00	Татнефть
TATNP	546.10	Татнефть - привилегированные акции
VKCO	297.40	ВК
MAGN	365.45	Магнитогорский металлургический комбинат
VTBR	76.21	Банк ВТБ
RUAL	337.45	РУСАЛ
ALRS	475.20	АЛРОСА
PIKK	569.10	ПИК
AFLT	544.00	Аэрофлот


In [7]:
import os

import pandas as pd

from datetime import timedelta, datetime
import time

from math import floor, log10

from tinkoff.invest import AsyncClient
from tinkoff.invest.constants import INVEST_GRPC_API_SANDBOX
from tinkoff.invest.schemas import (
    Deviation,
    GetTechAnalysisRequest,
    IndicatorInterval,
    CandleInterval,
    CandleSource,
    IndicatorType,
    Smoothing,
    TypeOfPrice,
)
from tinkoff.invest.utils import decimal_to_quotation, now

from dotenv import load_dotenv

from bokeh.plotting import figure, show
# Активация Bokeh
from bokeh.io import output_notebook, curdoc

# Токен санбокс т-инвестиции
load_dotenv()
TOKEN_TIN_SanBox = os.getenv("TOKEN_TIN_SanBox")

# Пеобразование тин-чисел в флоат
def format_T_num_in_float(units, nano):
    if nano < 0:
        nano = abs(nano)
    if nano == 0:
        return float(units)
    return units + nano * 10**-(floor(log10(nano))+1)

# Введите данные по условию
# Тикер БА
ticker_BA = 'AFLT'
# Дата исполнения
date_execution = pd.Timestamp(2024, 11, 6)
# Дата покупки(начала пролонгирования)
date_purchase = pd.Timestamp(2024, 10, 31)
# Страйк
option_strike = 58.5
# Цена нижестоящего страйка
option_low_strike = 57
# Цена премии опциона
premium_price = 4.63
# Цена премии нижестоящего опциона
premium_low_price = 3.24
# Комиссия опциона
commiss_premium = 0.0
# Стоимость БА на момент покупки(пролонгирования)
price_BA_date_purchase = 53.99
# Цена безубыточности опциона
break_even_price_opt = 53.73

# Расчёт дельты
delta_calculation = (premium_low_price - premium_price) / (option_strike - option_low_strike)

# Расчитываем хедж или другими словами количество опционов на одну акцию
hedge_calculating = round(price_BA_date_purchase / (abs(delta_calculation) * 100))

# Запрос месячных свечей
async def get_history_candles(
        period, interval, figi='figi', uid='uid'
):
    try:
        async with AsyncClient(TOKEN_TIN_SanBox,
                               target=INVEST_GRPC_API_SANDBOX) as client:
            historical_candles = await client.market_data.get_candles(
                figi=figi,
                from_=now() - timedelta(days=period),
                to=now(),
                interval=interval,
                instrument_id=uid,
                candle_source_type=CandleSource.CANDLE_SOURCE_EXCHANGE
            )
            historical_candles_array = {
                'Время свечи': [x.time for x in historical_candles.candles],
                'Цена открытия': [x.open for x in historical_candles.candles],
                'Макс цена': [x.high for x in historical_candles.candles],
                'Мин цена': [x.low for x in historical_candles.candles],
                'Цена закрытия': [format_T_num_in_float(units=x.close.units, nano=x.close.nano) for x in historical_candles.candles],
                'Сформированость': [x.is_complete for x in historical_candles.candles],
                'Объём торгов': [x.volume for x in historical_candles.candles],
                # 'Тип источника': [x.candle_source.name for x in historical_candles.candles],
            }
            df = pd.DataFrame.from_dict(historical_candles_array)
            return df
    except Exception as e:
        print(
            f"Тип исключения: {type(e).__name__}, сообщение: {str(e)}"
        )
        return None
# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
# Получить опционы из таблицы
df_opt = pd.read_excel(io='all_options.xlsx', engine='openpyxl')
# Получить индекс БА по тикеру инструмента
def index_by_ticker(tiker='tiker'):
    for tik, ind in zip(df_share['Тикер'], df_share['Unnamed: 0']):
        if tik == tiker:
            return [ind, tik]
# Получить массив исторических свеч по тикеру инструмента
ind_tik_BA = (index_by_ticker(tiker=ticker_BA))
df_candles_BA = await get_history_candles(365, CandleInterval.CANDLE_INTERVAL_DAY, figi=df_share['FIGI'][ind_tik_BA[0]], uid=df_share['UID'][ind_tik_BA[0]])

# Цена БА последней сделки
price_BA_last_transaction = df_candles_BA['Цена закрытия'][(len(df_candles_BA['Цена закрытия']) - 1)]

# Цена неубыточности БА
break_even_price_BA = price_BA_date_purchase + (premium_price * hedge_calculating) + (commiss_premium * hedge_calculating)

# Тема графика
curdoc().theme = "dark_minimal"

# Создание поля графика
p = figure(title=f"{df_share['Название'][ind_tik_BA[0]]} PUT {option_strike} {date_execution:%d.%m.%Y}", x_axis_type="datetime", sizing_mode="stretch_width")
# деактивация панели инструментов
# p.toolbar_location = None
# Создание графика исторических свечей
p.line(df_candles_BA['Время свечи'], df_candles_BA['Цена закрытия'], color="blue", line_width=4, legend_label=f"БА: {df_share['Название'][ind_tik_BA[0]]}")

# Создание графика-box убыточности
# Справа
p.segment(x0=date_purchase, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_BA, legend_label="Дата покупки", line_color="gold", line_dash="8 8", line_width=2)
# Сверху
p.segment(x0=date_purchase, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_BA, legend_label="Цена неубыточности БА", line_color="mediumseagreen", line_dash="8 8", line_width=2)
# Слева
p.segment(x0=date_execution, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_opt, legend_label="Дата исполнения", line_color="navajowhite", line_dash="8 8", line_width=2)
# Снизу
p.segment(x0=date_execution, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_opt, legend_label="Цена неубыточности опциона", line_color="tomato", line_dash="8 8", line_width=2)

# Создание легенды
p.legend.title = f"Данные по опциону:\nЛотность БА: {df_share['Лотность'][ind_tik_BA[0]]}\nЦена БА на на время покупки: {price_BA_date_purchase}\nЦена последней сделки БА: {price_BA_last_transaction}\nДинамика: {((price_BA_last_transaction - price_BA_date_purchase) / price_BA_date_purchase * 100):.2f} %\nХедж: {hedge_calculating} (Количество опционов на один лот)\nЦена одного опциона: {premium_price}\nЦена суммы опционов за хедж: {(hedge_calculating * premium_price * df_share['Лотность'][ind_tik_BA[0]]):.2f}\nДата покупки опциона: {date_purchase:%d.%m.%Y}\nДата исполнения: {date_execution:%d.%m.%Y}"
p.legend.title_text_font_size = "16px"
p.legend.location = "top_left"
p.legend.title_text_color = 'orange'
print(break_even_price_opt)
print(break_even_price_BA)
# Активация Bokeh и вывод графика
output_notebook()
show(p)

53.73
58.620000000000005


Loading BokehJS ...

/home/sergo/TinvestOptions/venv/lib/python3.11/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


# Аэрофлот PUT 20.11

> Захеджирована акция 5 опционами

In [1]:
import pandas as pd

# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
for indx in df_share['Unnamed: 0']:
    if (df_share['Лотность'][indx] * df_share['Цена последней сделки'][indx]) < 600:
        print(f"{df_share['Тикер'][indx]}\t{(df_share['Цена последней сделки'][indx] * df_share['Лотность'][indx]):.2f}\t{df_share['Название'][indx]}")

IRAO	375.70	Интер РАО ЕЭС
SNGSP	547.70	Сургутнефтегаз - привилегированные акции
ASTR	504.55	Группа Астра
ROSN	437.85	Роснефть
MTLR	96.63	Мечел
TATN	548.00	Татнефть
TATNP	546.10	Татнефть - привилегированные акции
VKCO	297.40	ВК
MAGN	365.45	Магнитогорский металлургический комбинат
VTBR	76.21	Банк ВТБ
RUAL	337.45	РУСАЛ
ALRS	475.20	АЛРОСА
PIKK	569.10	ПИК
AFLT	544.00	Аэрофлот


In [8]:
import os

import pandas as pd

from datetime import timedelta, datetime
import time

from math import floor, log10

from tinkoff.invest import AsyncClient
from tinkoff.invest.constants import INVEST_GRPC_API_SANDBOX
from tinkoff.invest.schemas import (
    Deviation,
    GetTechAnalysisRequest,
    IndicatorInterval,
    CandleInterval,
    CandleSource,
    IndicatorType,
    Smoothing,
    TypeOfPrice,
)
from tinkoff.invest.utils import decimal_to_quotation, now

from dotenv import load_dotenv

from bokeh.models import HoverTool
from bokeh.plotting import figure, show
# Активация Bokeh
from bokeh.io import output_notebook, curdoc

# Токен санбокс т-инвестиции
load_dotenv()
TOKEN_TIN_SanBox = os.getenv("TOKEN_TIN_SanBox")

# Пеобразование тин-чисел в флоат
def format_T_num_in_float(units, nano):
    if nano < 0:
        nano = abs(nano)
    if nano == 0:
        return float(units)
    return units + nano * 10**-(floor(log10(nano))+1)

# Введите данные по условию
# Тикер БА
ticker_BA = 'AFLT'
# Дата исполнения
date_execution = pd.Timestamp(2024, 11, 20)
# Дата покупки(начала пролонгирования)
date_purchase = pd.Timestamp(2024, 11, 7)
# Страйк
option_strike = 54
# Цена нижестоящего страйка
option_low_strike = 52.5
# Цена премии опциона
premium_price = 0.59
# Цена премии нижестоящего опциона
premium_low_price = 0.4
# Комиссия опциона
commiss_premium = 0.0
# Стоимость БА на момент покупки(пролонгирования)
price_BA_date_purchase = 58.35
# Цена безубыточности опциона
break_even_price_opt = 53.36

# Расчёт дельты
delta_calculation = (premium_low_price - premium_price) / (option_strike - option_low_strike)

# Расчитываем хедж или другими словами количество опционов на одну акцию
hedge_calculating = round(price_BA_date_purchase / (abs(delta_calculation) * 100))

# Запрос месячных свечей
async def get_history_candles(
        period, interval, figi='figi', uid='uid'
):
    try:
        async with AsyncClient(TOKEN_TIN_SanBox,
                               target=INVEST_GRPC_API_SANDBOX) as client:
            historical_candles = await client.market_data.get_candles(
                figi=figi,
                from_=now() - timedelta(days=period),
                to=now(),
                interval=interval,
                instrument_id=uid,
                candle_source_type=CandleSource.CANDLE_SOURCE_EXCHANGE
            )
            historical_candles_array = {
                'Время свечи': [x.time for x in historical_candles.candles],
                'Цена открытия': [x.open for x in historical_candles.candles],
                'Макс цена': [x.high for x in historical_candles.candles],
                'Мин цена': [x.low for x in historical_candles.candles],
                'Цена закрытия': [format_T_num_in_float(units=x.close.units, nano=x.close.nano) for x in historical_candles.candles],
                'Сформированость': [x.is_complete for x in historical_candles.candles],
                'Объём торгов': [x.volume for x in historical_candles.candles],
                # 'Тип источника': [x.candle_source.name for x in historical_candles.candles],
            }
            df = pd.DataFrame.from_dict(historical_candles_array)
            return df
    except Exception as e:
        print(
            f"Тип исключения: {type(e).__name__}, сообщение: {str(e)}"
        )
        return None
# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
# Получить опционы из таблицы
df_opt = pd.read_excel(io='all_options.xlsx', engine='openpyxl')
# Получить индекс БА по тикеру инструмента
def index_by_ticker(tiker='tiker'):
    for tik, ind in zip(df_share['Тикер'], df_share['Unnamed: 0']):
        if tik == tiker:
            return [ind, tik]
# Получить массив исторических свеч по тикеру инструмента
ind_tik_BA = (index_by_ticker(tiker=ticker_BA))
df_candles_BA = await get_history_candles(365, CandleInterval.CANDLE_INTERVAL_DAY, figi=df_share['FIGI'][ind_tik_BA[0]], uid=df_share['UID'][ind_tik_BA[0]])

# Цена БА последней сделки
price_BA_last_transaction = df_candles_BA['Цена закрытия'][(len(df_candles_BA['Цена закрытия']) - 1)]

# Цена неубыточности БА
break_even_price_BA = price_BA_date_purchase + (premium_price * hedge_calculating) + (commiss_premium * hedge_calculating)

# Тема графика
curdoc().theme = "dark_minimal"
# Панель инструментов
TOOLS = "pan,wheel_zoom,box_zoom,reset, crosshair"
# Данные для изображения в hover
data_in_hover = HoverTool(
    tooltips=[
        ( 'Дата',   '@x{%F}'),
        ( 'Цена закрытия',  '@y{%0.2f} руб' ),
    ],
    formatters={
        '@x': 'datetime',
        '@y': 'printf',
    },
)

# Создание поля графика
p = figure(title=f"{df_share['Название'][ind_tik_BA[0]]} PUT {option_strike} {date_execution:%d.%m.%Y}", x_axis_type="datetime", sizing_mode="stretch_width", tools=[data_in_hover, TOOLS])
# деактивация панели инструментов
# p.toolbar_location = None
# Создание графика исторических свечей
p.line(df_candles_BA['Время свечи'], df_candles_BA['Цена закрытия'], color="blue", line_width=4, legend_label=f"БА: {df_share['Название'][ind_tik_BA[0]]}")

# Создание графика-box убыточности
# Справа
p.segment(x0=date_purchase, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_BA, legend_label="Дата покупки", line_color="gold", line_dash="8 8", line_width=2)
# Сверху
p.segment(x0=date_purchase, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_BA, legend_label="Цена неубыточности БА", line_color="mediumseagreen", line_dash="8 8", line_width=2)
# Слева
p.segment(x0=date_execution, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_opt, legend_label="Дата исполнения", line_color="navajowhite", line_dash="8 8", line_width=2)
# Снизу
p.segment(x0=date_execution, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_opt, legend_label="Цена неубыточности опциона", line_color="tomato", line_dash="8 8", line_width=2)

# Создание легенды
p.legend.title = f"Данные по опциону:\nЛотность БА: {df_share['Лотность'][ind_tik_BA[0]]}\nЦена БА на на время покупки: {price_BA_date_purchase}\nЦена последней сделки БА: {price_BA_last_transaction}\nДинамика: {((price_BA_last_transaction - price_BA_date_purchase) / price_BA_date_purchase * 100):.2f} %\nХедж: {hedge_calculating} (Количество опционов на один лот)\nЦена одного опциона: {premium_price}\nЦена суммы опционов за хедж: {(hedge_calculating * premium_price * df_share['Лотность'][ind_tik_BA[0]]):.2f}\nДата покупки опциона: {date_purchase:%d.%m.%Y}\nДата исполнения: {date_execution:%d.%m.%Y}"
p.legend.title_text_font_size = "16px"
p.legend.location = "top_left"
p.legend.title_text_color = 'orange'
print(break_even_price_opt)
print(break_even_price_BA)
# Активация Bokeh и вывод графика
output_notebook()
show(p)

53.36
61.300000000000004


Loading BokehJS ...

/home/sergo/TinvestOptions/venv/lib/python3.11/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


# Сургутнефтегаз - привилегированные акции PUT 20.11

> Выставлена заявка на один пут для хеджа одной акции

In [19]:
import pandas as pd

# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
for indx in df_share['Unnamed: 0']:
    if (df_share['Лотность'][indx] * df_share['Цена последней сделки'][indx]) < 600:
        print(f"{df_share['Тикер'][indx]}\t{(df_share['Цена последней сделки'][indx] * df_share['Лотность'][indx]):.2f}\t{df_share['Название'][indx]}")

IRAO	375.70	Интер РАО ЕЭС
SNGSP	547.70	Сургутнефтегаз - привилегированные акции
ASTR	504.55	Группа Астра
ROSN	437.85	Роснефть
MTLR	96.63	Мечел
TATN	548.00	Татнефть
TATNP	546.10	Татнефть - привилегированные акции
VKCO	297.40	ВК
MAGN	365.45	Магнитогорский металлургический комбинат
VTBR	76.21	Банк ВТБ
RUAL	337.45	РУСАЛ
ALRS	475.20	АЛРОСА
PIKK	569.10	ПИК
AFLT	544.00	Аэрофлот


In [9]:
import os

import pandas as pd

from datetime import timedelta, datetime
import time

from math import floor, log10

from tinkoff.invest import AsyncClient
from tinkoff.invest.constants import INVEST_GRPC_API_SANDBOX
from tinkoff.invest.schemas import (
    Deviation,
    GetTechAnalysisRequest,
    IndicatorInterval,
    CandleInterval,
    CandleSource,
    IndicatorType,
    Smoothing,
    TypeOfPrice,
)
from tinkoff.invest.utils import decimal_to_quotation, now

from dotenv import load_dotenv

from bokeh.models import HoverTool
from bokeh.plotting import figure, show
# Активация Bokeh
from bokeh.io import output_notebook, curdoc

# Токен санбокс т-инвестиции
load_dotenv()
TOKEN_TIN_SanBox = os.getenv("TOKEN_TIN_SanBox")

# Пеобразование тин-чисел в флоат
def format_T_num_in_float(units, nano):
    if nano < 0:
        nano = abs(nano)
    if nano == 0:
        return float(units)
    return units + nano * 10**-(floor(log10(nano))+1)

# Введите данные по условию
# Тикер БА
ticker_BA = 'SNGSP'
# Дата исполнения
date_execution = pd.Timestamp(2024, 11, 20)
# Дата покупки(начала пролонгирования)
date_purchase = pd.Timestamp(2024, 11, 7)
# Страйк
option_strike = 56
# Цена нижестоящего страйка
option_low_strike = 54
# Цена премии опциона
premium_price = 1.46
# Цена премии нижестоящего опциона
premium_low_price = 0.57
# Комиссия опциона
commiss_premium = 0.0
# Стоимость БА на момент покупки(пролонгирования)
price_BA_date_purchase = 56.06
# Цена безубыточности опциона
break_even_price_opt = 54.49

# Расчёт дельты
delta_calculation = (premium_low_price - premium_price) / (option_strike - option_low_strike)

# Расчитываем хедж или другими словами количество опционов на одну акцию
hedge_calculating = round(price_BA_date_purchase / (abs(delta_calculation) * 100))

# Запрос месячных свечей
async def get_history_candles(
        period, interval, figi='figi', uid='uid'
):
    try:
        async with AsyncClient(TOKEN_TIN_SanBox,
                               target=INVEST_GRPC_API_SANDBOX) as client:
            historical_candles = await client.market_data.get_candles(
                figi=figi,
                from_=now() - timedelta(days=period),
                to=now(),
                interval=interval,
                instrument_id=uid,
                candle_source_type=CandleSource.CANDLE_SOURCE_EXCHANGE
            )
            historical_candles_array = {
                'Время свечи': [x.time for x in historical_candles.candles],
                'Цена открытия': [x.open for x in historical_candles.candles],
                'Макс цена': [x.high for x in historical_candles.candles],
                'Мин цена': [x.low for x in historical_candles.candles],
                'Цена закрытия': [format_T_num_in_float(units=x.close.units, nano=x.close.nano) for x in historical_candles.candles],
                'Сформированость': [x.is_complete for x in historical_candles.candles],
                'Объём торгов': [x.volume for x in historical_candles.candles],
                # 'Тип источника': [x.candle_source.name for x in historical_candles.candles],
            }
            df = pd.DataFrame.from_dict(historical_candles_array)
            return df
    except Exception as e:
        print(
            f"Тип исключения: {type(e).__name__}, сообщение: {str(e)}"
        )
        return None
# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
# Получить опционы из таблицы
df_opt = pd.read_excel(io='all_options.xlsx', engine='openpyxl')
# Получить индекс БА по тикеру инструмента
def index_by_ticker(tiker='tiker'):
    for tik, ind in zip(df_share['Тикер'], df_share['Unnamed: 0']):
        if tik == tiker:
            return [ind, tik]
# Получить массив исторических свеч по тикеру инструмента
ind_tik_BA = (index_by_ticker(tiker=ticker_BA))
df_candles_BA = await get_history_candles(365, CandleInterval.CANDLE_INTERVAL_DAY, figi=df_share['FIGI'][ind_tik_BA[0]], uid=df_share['UID'][ind_tik_BA[0]])

# Цена БА последней сделки
price_BA_last_transaction = df_candles_BA['Цена закрытия'][(len(df_candles_BA['Цена закрытия']) - 1)]

# Цена неубыточности БА
break_even_price_BA = price_BA_date_purchase + (premium_price * hedge_calculating) + (commiss_premium * hedge_calculating)

# Тема графика
curdoc().theme = "dark_minimal"
# Панель инструментов
TOOLS = "pan,wheel_zoom,box_zoom,reset, crosshair"
# Данные для изображения в hover
data_in_hover = HoverTool(
    tooltips=[
        ( 'Дата',   '@x{%F}'),
        ( 'Цена закрытия',  '@y{%0.2f} руб' ),
    ],
    formatters={
        '@x': 'datetime',
        '@y': 'printf',
    },
)

# Создание поля графика
p = figure(title=f"{df_share['Название'][ind_tik_BA[0]]} PUT {option_strike} {date_execution:%d.%m.%Y}", x_axis_type="datetime", sizing_mode="stretch_width", tools=[data_in_hover, TOOLS])
# деактивация панели инструментов
# p.toolbar_location = None
# Создание графика исторических свечей
p.line(df_candles_BA['Время свечи'], df_candles_BA['Цена закрытия'], color="blue", line_width=4, legend_label=f"БА: {df_share['Название'][ind_tik_BA[0]]}")

# Создание графика-box убыточности
# Справа
p.segment(x0=date_purchase, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_BA, legend_label="Дата покупки", line_color="gold", line_dash="8 8", line_width=2)
# Сверху
p.segment(x0=date_purchase, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_BA, legend_label="Цена неубыточности БА", line_color="mediumseagreen", line_dash="8 8", line_width=2)
# Слева
p.segment(x0=date_execution, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_opt, legend_label="Дата исполнения", line_color="navajowhite", line_dash="8 8", line_width=2)
# Снизу
p.segment(x0=date_execution, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_opt, legend_label="Цена неубыточности опциона", line_color="tomato", line_dash="8 8", line_width=2)

# Создание легенды
p.legend.title = f"Данные по опциону:\nЛотность БА: {df_share['Лотность'][ind_tik_BA[0]]}\nЦена БА на на время покупки: {price_BA_date_purchase}\nЦена последней сделки БА: {price_BA_last_transaction}\nДинамика: {((price_BA_last_transaction - price_BA_date_purchase) / price_BA_date_purchase * 100):.2f} %\nХедж: {hedge_calculating} (Количество опционов на один лот)\nЦена одного опциона: {premium_price}\nЦена суммы опционов за хедж: {(hedge_calculating * premium_price * df_share['Лотность'][ind_tik_BA[0]]):.2f}\nДата покупки опциона: {date_purchase:%d.%m.%Y}\nДата исполнения: {date_execution:%d.%m.%Y}"
p.legend.title_text_font_size = "16px"
p.legend.location = "top_left"
p.legend.title_text_color = 'orange'
print(break_even_price_opt)
print(break_even_price_BA)
# Активация Bokeh и вывод графика
output_notebook()
show(p)

54.49
57.52


Loading BokehJS ...

/home/sergo/TinvestOptions/venv/lib/python3.11/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


# Магнитогорский металлургический комбинат PUT 20.11

> Куплено 3 пута на удачу и все разервировано одной акцией

In [21]:
import pandas as pd

# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
for indx in df_share['Unnamed: 0']:
    if (df_share['Лотность'][indx] * df_share['Цена последней сделки'][indx]) < 600:
        print(f"{df_share['Тикер'][indx]}\t{(df_share['Цена последней сделки'][indx] * df_share['Лотность'][indx]):.2f}\t{df_share['Название'][indx]}")

IRAO	375.70	Интер РАО ЕЭС
SNGSP	547.70	Сургутнефтегаз - привилегированные акции
ASTR	504.55	Группа Астра
ROSN	437.85	Роснефть
MTLR	96.63	Мечел
TATN	548.00	Татнефть
TATNP	546.10	Татнефть - привилегированные акции
VKCO	297.40	ВК
MAGN	365.45	Магнитогорский металлургический комбинат
VTBR	76.21	Банк ВТБ
RUAL	337.45	РУСАЛ
ALRS	475.20	АЛРОСА
PIKK	569.10	ПИК
AFLT	544.00	Аэрофлот


In [10]:
import os

import pandas as pd

from datetime import timedelta, datetime
import time

from math import floor, log10

from tinkoff.invest import AsyncClient
from tinkoff.invest.constants import INVEST_GRPC_API_SANDBOX
from tinkoff.invest.schemas import (
    Deviation,
    GetTechAnalysisRequest,
    IndicatorInterval,
    CandleInterval,
    CandleSource,
    IndicatorType,
    Smoothing,
    TypeOfPrice,
)
from tinkoff.invest.utils import decimal_to_quotation, now

from dotenv import load_dotenv

from bokeh.models import HoverTool
from bokeh.plotting import figure, show
# Активация Bokeh
from bokeh.io import output_notebook, curdoc

# Токен санбокс т-инвестиции
load_dotenv()
TOKEN_TIN_SanBox = os.getenv("TOKEN_TIN_SanBox")

# Пеобразование тин-чисел в флоат
def format_T_num_in_float(units, nano):
    if nano < 0:
        nano = abs(nano)
    if nano == 0:
        return float(units)
    return units + nano * 10**-(floor(log10(nano))+1)

# Введите данные по условию
# Тикер БА
ticker_BA = 'MAGN'
# Дата исполнения
date_execution = pd.Timestamp(2024, 11, 20)
# Дата покупки(начала пролонгирования)
date_purchase = pd.Timestamp(2024, 11, 7)
# Страйк
option_strike = 44
# Цена нижестоящего страйка
option_low_strike = 42
# Цена премии опциона
premium_price = 6.26
# Цена премии нижестоящего опциона
premium_low_price = 8.33
# Комиссия опциона
commiss_premium = 0.0
# Стоимость БА на момент покупки(пролонгирования)
price_BA_date_purchase = 37.435
# Цена безубыточности опциона
break_even_price_opt = 37.24

# Расчёт дельты
delta_calculation = (premium_low_price - premium_price) / (option_strike - option_low_strike)

# Расчитываем хедж или другими словами количество опционов на одну акцию
hedge_calculating = round(price_BA_date_purchase / (abs(delta_calculation) * 100))

# Запрос месячных свечей
async def get_history_candles(
        period, interval, figi='figi', uid='uid'
):
    try:
        async with AsyncClient(TOKEN_TIN_SanBox,
                               target=INVEST_GRPC_API_SANDBOX) as client:
            historical_candles = await client.market_data.get_candles(
                figi=figi,
                from_=now() - timedelta(days=period),
                to=now(),
                interval=interval,
                instrument_id=uid,
                candle_source_type=CandleSource.CANDLE_SOURCE_EXCHANGE
            )
            historical_candles_array = {
                'Время свечи': [x.time for x in historical_candles.candles],
                'Цена открытия': [x.open for x in historical_candles.candles],
                'Макс цена': [x.high for x in historical_candles.candles],
                'Мин цена': [x.low for x in historical_candles.candles],
                'Цена закрытия': [format_T_num_in_float(units=x.close.units, nano=x.close.nano) for x in historical_candles.candles],
                'Сформированость': [x.is_complete for x in historical_candles.candles],
                'Объём торгов': [x.volume for x in historical_candles.candles],
                # 'Тип источника': [x.candle_source.name for x in historical_candles.candles],
            }
            df = pd.DataFrame.from_dict(historical_candles_array)
            return df
    except Exception as e:
        print(
            f"Тип исключения: {type(e).__name__}, сообщение: {str(e)}"
        )
        return None
# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
# Получить опционы из таблицы
df_opt = pd.read_excel(io='all_options.xlsx', engine='openpyxl')
# Получить индекс БА по тикеру инструмента
def index_by_ticker(tiker='tiker'):
    for tik, ind in zip(df_share['Тикер'], df_share['Unnamed: 0']):
        if tik == tiker:
            return [ind, tik]
# Получить массив исторических свеч по тикеру инструмента
ind_tik_BA = (index_by_ticker(tiker=ticker_BA))
df_candles_BA = await get_history_candles(365, CandleInterval.CANDLE_INTERVAL_DAY, figi=df_share['FIGI'][ind_tik_BA[0]], uid=df_share['UID'][ind_tik_BA[0]])

# Цена БА последней сделки
price_BA_last_transaction = df_candles_BA['Цена закрытия'][(len(df_candles_BA['Цена закрытия']) - 1)]

# Цена неубыточности БА
break_even_price_BA = price_BA_date_purchase + (premium_price * hedge_calculating) + (commiss_premium * hedge_calculating)

# Тема графика
curdoc().theme = "dark_minimal"
# Панель инструментов
TOOLS = "pan,wheel_zoom,box_zoom,reset, crosshair"
# Данные для изображения в hover
data_in_hover = HoverTool(
    tooltips=[
        ( 'Дата',   '@x{%F}'),
        ( 'Цена закрытия',  '@y{%0.2f} руб' ),
    ],
    formatters={
        '@x': 'datetime',
        '@y': 'printf',
    },
)

# Создание поля графика
p = figure(title=f"{df_share['Название'][ind_tik_BA[0]]} PUT {option_strike} {date_execution:%d.%m.%Y}", x_axis_type="datetime", sizing_mode="stretch_width", tools=[data_in_hover, TOOLS])
# деактивация панели инструментов
# p.toolbar_location = None
# Создание графика исторических свечей
p.line(df_candles_BA['Время свечи'], df_candles_BA['Цена закрытия'], color="blue", line_width=4, legend_label=f"БА: {df_share['Название'][ind_tik_BA[0]]}")

# Создание графика-box убыточности
# Справа
p.segment(x0=date_purchase, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_BA, legend_label="Дата покупки", line_color="gold", line_dash="8 8", line_width=2)
# Сверху
p.segment(x0=date_purchase, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_BA, legend_label="Цена неубыточности БА", line_color="mediumseagreen", line_dash="8 8", line_width=2)
# Слева
p.segment(x0=date_execution, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_opt, legend_label="Дата исполнения", line_color="navajowhite", line_dash="8 8", line_width=2)
# Снизу
p.segment(x0=date_execution, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_opt, legend_label="Цена неубыточности опциона", line_color="tomato", line_dash="8 8", line_width=2)

# Создание легенды
p.legend.title = f"Данные по опциону:\nЛотность БА: {df_share['Лотность'][ind_tik_BA[0]]}\nЦена БА на на время покупки: {price_BA_date_purchase}\nЦена последней сделки БА: {price_BA_last_transaction}\nДинамика: {((price_BA_last_transaction - price_BA_date_purchase) / price_BA_date_purchase * 100):.2f} %\nХедж: {hedge_calculating} (Количество опционов на один лот)\nЦена одного опциона: {premium_price}\nЦена суммы опционов за хедж: {(hedge_calculating * premium_price * df_share['Лотность'][ind_tik_BA[0]]):.2f}\nДата покупки опциона: {date_purchase:%d.%m.%Y}\nДата исполнения: {date_execution:%d.%m.%Y}"
p.legend.title_text_font_size = "16px"
p.legend.location = "top_left"
p.legend.title_text_color = 'orange'
print(break_even_price_opt)
print(break_even_price_BA)
# Активация Bokeh и вывод графика
output_notebook()
show(p)

37.24
37.435


Loading BokehJS ...

/home/sergo/TinvestOptions/venv/lib/python3.11/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


# ВТБ PUT 20.11

> Куплен один пут для пролонгирования

In [5]:
import pandas as pd

# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
for indx in df_share['Unnamed: 0']:
    if (df_share['Лотность'][indx] * df_share['Цена последней сделки'][indx]) < 600:
        print(f"{df_share['Тикер'][indx]}\t{(df_share['Цена последней сделки'][indx] * df_share['Лотность'][indx]):.2f}\t{df_share['Название'][indx]}")

IRAO	375.70	Интер РАО ЕЭС
SNGSP	547.70	Сургутнефтегаз - привилегированные акции
ASTR	504.55	Группа Астра
ROSN	437.85	Роснефть
MTLR	96.63	Мечел
TATN	548.00	Татнефть
TATNP	546.10	Татнефть - привилегированные акции
VKCO	297.40	ВК
MAGN	365.45	Магнитогорский металлургический комбинат
VTBR	76.21	Банк ВТБ
RUAL	337.45	РУСАЛ
ALRS	475.20	АЛРОСА
PIKK	569.10	ПИК
AFLT	544.00	Аэрофлот


In [11]:
import os

import pandas as pd

from datetime import timedelta, datetime
import time

from math import floor, log10

from tinkoff.invest import AsyncClient
from tinkoff.invest.constants import INVEST_GRPC_API_SANDBOX
from tinkoff.invest.schemas import (
    Deviation,
    GetTechAnalysisRequest,
    IndicatorInterval,
    CandleInterval,
    CandleSource,
    IndicatorType,
    Smoothing,
    TypeOfPrice,
)
from tinkoff.invest.utils import decimal_to_quotation, now

from dotenv import load_dotenv

from bokeh.models import HoverTool
from bokeh.plotting import figure, show
# Активация Bokeh
from bokeh.io import output_notebook, curdoc

# Токен санбокс т-инвестиции
load_dotenv()
TOKEN_TIN_SanBox = os.getenv("TOKEN_TIN_SanBox")

# Пеобразование тин-чисел в флоат
def format_T_num_in_float(units, nano):
    if nano < 0:
        nano = abs(nano)
    if nano == 0:
        return float(units)
    return units + nano * 10**-(floor(log10(nano))+1)

# Введите данные по условию
# Тикер БА
ticker_BA = 'VTBR'
# Дата исполнения
date_execution = pd.Timestamp(2024, 11, 20)
# Дата покупки(начала пролонгирования)
date_purchase = pd.Timestamp(2024, 11, 11)
# Страйк
option_strike = 86
# Цена нижестоящего страйка
option_low_strike = 84
# Цена премии опциона
premium_price = 4.68
# Цена премии нижестоящего опциона
premium_low_price = 3.06
# Комиссия опциона
commiss_premium = 0.0
# Стоимость БА на момент покупки(пролонгирования)
price_BA_date_purchase = 81.62
# Цена безубыточности опциона
break_even_price_opt = 81.18

# Расчёт дельты
delta_calculation = (premium_low_price - premium_price) / (option_strike - option_low_strike)

# Расчитываем хедж или другими словами количество опционов на одну акцию
hedge_calculating = round(price_BA_date_purchase / (abs(delta_calculation) * 100))

# Запрос месячных свечей
async def get_history_candles(
        period, interval, figi='figi', uid='uid'
):
    try:
        async with AsyncClient(TOKEN_TIN_SanBox,
                               target=INVEST_GRPC_API_SANDBOX) as client:
            historical_candles = await client.market_data.get_candles(
                figi=figi,
                from_=now() - timedelta(days=period),
                to=now(),
                interval=interval,
                instrument_id=uid,
                candle_source_type=CandleSource.CANDLE_SOURCE_EXCHANGE
            )
            historical_candles_array = {
                'Время свечи': [x.time for x in historical_candles.candles],
                'Цена открытия': [x.open for x in historical_candles.candles],
                'Макс цена': [x.high for x in historical_candles.candles],
                'Мин цена': [x.low for x in historical_candles.candles],
                'Цена закрытия': [format_T_num_in_float(units=x.close.units, nano=x.close.nano) for x in historical_candles.candles],
                'Сформированость': [x.is_complete for x in historical_candles.candles],
                'Объём торгов': [x.volume for x in historical_candles.candles],
                # 'Тип источника': [x.candle_source.name for x in historical_candles.candles],
            }
            df = pd.DataFrame.from_dict(historical_candles_array)
            return df
    except Exception as e:
        print(
            f"Тип исключения: {type(e).__name__}, сообщение: {str(e)}"
        )
        return None
# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
# Получить опционы из таблицы
df_opt = pd.read_excel(io='all_options.xlsx', engine='openpyxl')
# Получить индекс БА по тикеру инструмента
def index_by_ticker(tiker='tiker'):
    for tik, ind in zip(df_share['Тикер'], df_share['Unnamed: 0']):
        if tik == tiker:
            return [ind, tik]
# Получить массив исторических свеч по тикеру инструмента
ind_tik_BA = (index_by_ticker(tiker=ticker_BA))
df_candles_BA = await get_history_candles(365, CandleInterval.CANDLE_INTERVAL_DAY, figi=df_share['FIGI'][ind_tik_BA[0]], uid=df_share['UID'][ind_tik_BA[0]])

# Цена БА последней сделки
price_BA_last_transaction = df_candles_BA['Цена закрытия'][(len(df_candles_BA['Цена закрытия']) - 1)]

# Цена неубыточности БА
break_even_price_BA = price_BA_date_purchase + (premium_price * hedge_calculating) + (commiss_premium * hedge_calculating)

# Тема графика
curdoc().theme = "dark_minimal"
# Панель инструментов
TOOLS = "pan,wheel_zoom,box_zoom,reset, crosshair"
# Данные для изображения в hover
data_in_hover = HoverTool(
    tooltips=[
        ( 'Дата',   '@x{%F}'),
        ( 'Цена закрытия',  '@y{%0.2f} руб' ),
    ],
    formatters={
        '@x': 'datetime',
        '@y': 'printf',
    },
)

# Создание поля графика
p = figure(title=f"{df_share['Название'][ind_tik_BA[0]]} PUT {option_strike} {date_execution:%d.%m.%Y}", x_axis_type="datetime", sizing_mode="stretch_width", tools=[data_in_hover, TOOLS])
# деактивация панели инструментов
# p.toolbar_location = None
# Создание графика исторических свечей
p.line(df_candles_BA['Время свечи'], df_candles_BA['Цена закрытия'], color="blue", line_width=4, legend_label=f"БА: {df_share['Название'][ind_tik_BA[0]]}")

# Создание графика-box убыточности
# Справа
p.segment(x0=date_purchase, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_BA, legend_label="Дата покупки", line_color="gold", line_dash="8 8", line_width=2)
# Сверху
p.segment(x0=date_purchase, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_BA, legend_label="Цена неубыточности БА", line_color="mediumseagreen", line_dash="8 8", line_width=2)
# Слева
p.segment(x0=date_execution, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_opt, legend_label="Дата исполнения", line_color="navajowhite", line_dash="8 8", line_width=2)
# Снизу
p.segment(x0=date_execution, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_opt, legend_label="Цена неубыточности опциона", line_color="tomato", line_dash="8 8", line_width=2)

# Создание легенды
p.legend.title = f"Данные по опциону:\nЛотность БА: {df_share['Лотность'][ind_tik_BA[0]]}\nЦена БА на на время покупки: {price_BA_date_purchase}\nЦена последней сделки БА: {price_BA_last_transaction}\nДинамика: {((price_BA_last_transaction - price_BA_date_purchase) / price_BA_date_purchase * 100):.2f} %\nХедж: {hedge_calculating} (Количество опционов на один лот)\nЦена одного опциона: {premium_price}\nЦена суммы опционов за хедж: {(hedge_calculating * premium_price * df_share['Лотность'][ind_tik_BA[0]]):.2f}\nДата покупки опциона: {date_purchase:%d.%m.%Y}\nДата исполнения: {date_execution:%d.%m.%Y}"
p.legend.title_text_font_size = "16px"
p.legend.location = "top_left"
p.legend.title_text_color = 'orange'
print(break_even_price_opt)
print(break_even_price_BA)
# Активация Bokeh и вывод графика
output_notebook()
show(p)

81.18
86.30000000000001


Loading BokehJS ...

/home/sergo/TinvestOptions/venv/lib/python3.11/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


# РУСАЛ PUT 39 20.11

> Куплен один пут и одна акция - классика

In [14]:
import pandas as pd

# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
for indx in df_share['Unnamed: 0']:
    if (df_share['Лотность'][indx] * df_share['Цена последней сделки'][indx]) < 600:
        print(f"{df_share['Тикер'][indx]}\t{(df_share['Цена последней сделки'][indx] * df_share['Лотность'][indx]):.2f}\t{df_share['Название'][indx]}")

IRAO	375.70	Интер РАО ЕЭС
SNGSP	547.70	Сургутнефтегаз - привилегированные акции
ASTR	504.55	Группа Астра
ROSN	437.85	Роснефть
MTLR	96.63	Мечел
TATN	548.00	Татнефть
TATNP	546.10	Татнефть - привилегированные акции
VKCO	297.40	ВК
MAGN	365.45	Магнитогорский металлургический комбинат
VTBR	76.21	Банк ВТБ
RUAL	337.45	РУСАЛ
ALRS	475.20	АЛРОСА
PIKK	569.10	ПИК
AFLT	544.00	Аэрофлот


In [12]:
import os

import pandas as pd

from datetime import timedelta, datetime
import time

from math import floor, log10

from tinkoff.invest import AsyncClient
from tinkoff.invest.constants import INVEST_GRPC_API_SANDBOX
from tinkoff.invest.schemas import (
    Deviation,
    GetTechAnalysisRequest,
    IndicatorInterval,
    CandleInterval,
    CandleSource,
    IndicatorType,
    Smoothing,
    TypeOfPrice,
)
from tinkoff.invest.utils import decimal_to_quotation, now

from dotenv import load_dotenv

from bokeh.models import HoverTool
from bokeh.plotting import figure, show
# Активация Bokeh
from bokeh.io import output_notebook, curdoc

# Токен санбокс т-инвестиции
load_dotenv()
TOKEN_TIN_SanBox = os.getenv("TOKEN_TIN_SanBox")

# Пеобразование тин-чисел в флоат
def format_T_num_in_float(units, nano):
    if nano < 0:
        nano = abs(nano)
    if nano == 0:
        return float(units)
    return units + nano * 10**-(floor(log10(nano))+1)

# Введите данные по условию
# Тикер БА
ticker_BA = 'RUAL'
# Дата исполнения
date_execution = pd.Timestamp(2024, 11, 20)
# Дата покупки(начала пролонгирования)
date_purchase = pd.Timestamp(2024, 11, 11)
# Страйк
option_strike = 39
# Цена нижестоящего страйка
option_low_strike = 37.5
# Цена премии опциона
premium_price = 1.58
# Цена премии нижестоящего опциона
premium_low_price = 0.76
# Комиссия опциона
commiss_premium = 0.0
# Стоимость БА на момент покупки(пролонгирования)
price_BA_date_purchase = 37.885
# Цена безубыточности опциона
break_even_price_opt = 37.35

# Расчёт дельты
delta_calculation = (premium_low_price - premium_price) / (option_strike - option_low_strike)

# Расчитываем хедж или другими словами количество опционов на одну акцию
hedge_calculating = round(price_BA_date_purchase / (abs(delta_calculation) * 100))

# Запрос месячных свечей
async def get_history_candles(
        period, interval, figi='figi', uid='uid'
):
    try:
        async with AsyncClient(TOKEN_TIN_SanBox,
                               target=INVEST_GRPC_API_SANDBOX) as client:
            historical_candles = await client.market_data.get_candles(
                figi=figi,
                from_=now() - timedelta(days=period),
                to=now(),
                interval=interval,
                instrument_id=uid,
                candle_source_type=CandleSource.CANDLE_SOURCE_EXCHANGE
            )
            historical_candles_array = {
                'Время свечи': [x.time for x in historical_candles.candles],
                'Цена открытия': [x.open for x in historical_candles.candles],
                'Макс цена': [x.high for x in historical_candles.candles],
                'Мин цена': [x.low for x in historical_candles.candles],
                'Цена закрытия': [format_T_num_in_float(units=x.close.units, nano=x.close.nano) for x in historical_candles.candles],
                'Сформированость': [x.is_complete for x in historical_candles.candles],
                'Объём торгов': [x.volume for x in historical_candles.candles],
                # 'Тип источника': [x.candle_source.name for x in historical_candles.candles],
            }
            df = pd.DataFrame.from_dict(historical_candles_array)
            return df
    except Exception as e:
        print(
            f"Тип исключения: {type(e).__name__}, сообщение: {str(e)}"
        )
        return None
# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
# Получить опционы из таблицы
df_opt = pd.read_excel(io='all_options.xlsx', engine='openpyxl')
# Получить индекс БА по тикеру инструмента
def index_by_ticker(tiker='tiker'):
    for tik, ind in zip(df_share['Тикер'], df_share['Unnamed: 0']):
        if tik == tiker:
            return [ind, tik]
# Получить массив исторических свеч по тикеру инструмента
ind_tik_BA = (index_by_ticker(tiker=ticker_BA))
df_candles_BA = await get_history_candles(365, CandleInterval.CANDLE_INTERVAL_DAY, figi=df_share['FIGI'][ind_tik_BA[0]], uid=df_share['UID'][ind_tik_BA[0]])

# Цена БА последней сделки
price_BA_last_transaction = df_candles_BA['Цена закрытия'][(len(df_candles_BA['Цена закрытия']) - 1)]

# Цена неубыточности БА
break_even_price_BA = price_BA_date_purchase + (premium_price * hedge_calculating) + (commiss_premium * hedge_calculating)

# Тема графика
curdoc().theme = "dark_minimal"
# Панель инструментов
TOOLS = "pan,wheel_zoom,box_zoom,reset, crosshair"
# Данные для изображения в hover
data_in_hover = HoverTool(
    tooltips=[
        ( 'Дата',   '@x{%F}'),
        ( 'Цена закрытия',  '@y{%0.2f} руб' ),
    ],
    formatters={
        '@x': 'datetime',
        '@y': 'printf',
    },
)

# Создание поля графика
p = figure(title=f"{df_share['Название'][ind_tik_BA[0]]} PUT {option_strike} {date_execution:%d.%m.%Y}", x_axis_type="datetime", sizing_mode="stretch_width", tools=[data_in_hover, TOOLS])
# деактивация панели инструментов
# p.toolbar_location = None
# Создание графика исторических свечей
p.line(df_candles_BA['Время свечи'], df_candles_BA['Цена закрытия'], color="blue", line_width=4, legend_label=f"БА: {df_share['Название'][ind_tik_BA[0]]}")

# Создание графика-box убыточности
# Справа
p.segment(x0=date_purchase, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_BA, legend_label="Дата покупки", line_color="gold", line_dash="8 8", line_width=2)
# Сверху
p.segment(x0=date_purchase, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_BA, legend_label="Цена неубыточности БА", line_color="mediumseagreen", line_dash="8 8", line_width=2)
# Слева
p.segment(x0=date_execution, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_opt, legend_label="Дата исполнения", line_color="navajowhite", line_dash="8 8", line_width=2)
# Снизу
p.segment(x0=date_execution, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_opt, legend_label="Цена неубыточности опциона", line_color="tomato", line_dash="8 8", line_width=2)

# Создание легенды
p.legend.title = f"Данные по опциону:\nЛотность БА: {df_share['Лотность'][ind_tik_BA[0]]}\nЦена БА на на время покупки: {price_BA_date_purchase}\nЦена последней сделки БА: {price_BA_last_transaction}\nДинамика: {((price_BA_last_transaction - price_BA_date_purchase) / price_BA_date_purchase * 100):.2f} %\nХедж: {hedge_calculating} (Количество опционов на один лот)\nЦена одного опциона: {premium_price}\nЦена суммы опционов за хедж: {(hedge_calculating * premium_price * df_share['Лотность'][ind_tik_BA[0]]):.2f}\nДата покупки опциона: {date_purchase:%d.%m.%Y}\nДата исполнения: {date_execution:%d.%m.%Y}"
p.legend.title_text_font_size = "16px"
p.legend.location = "top_left"
p.legend.title_text_color = 'orange'
print(break_even_price_opt)
print(break_even_price_BA)
# Активация Bokeh и вывод графика
output_notebook()
show(p)

37.35
39.464999999999996


Loading BokehJS ...

/home/sergo/TinvestOptions/venv/lib/python3.11/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


# Новая стратегия
----

# Описание 

1. Для получения прибыли рассматривается только падение цены БА на ближайшей переспективе;
2. Для расчета персперктивы и построения графика берем:
    - Песледнюю цену БА;
    - Цену БА прибыльности опциона;
    - Цена премии опциона; 
    - Дата покупки опциона;
    - Дату исполнения опциона;
3. Для построения кооробки убыточности используем ребра:
    - пол: цена БА прибыльности опциона;
    - левая стенока: дата покупки;
    - правая стенка: дата исполнения;
    - потолок: премия опциона + последняя цена БА(тоесть по сути цена покупки БА) - это расчет один к одному, тоесть это цена БА которая при развороте в минус должна окупить покупку одного опциона на одну акцию;
4. При выборе точки входа методом перебора выбираем такую премию по опциону при которой пол и потолок "коробочки" находятся как можно ближе друг к другу, во время перебора учитывается как 1 к 1(БА и опцион),  так и 1 к n - поскольку на деле могут быть наиболее выгодны покупки более одного опциона на акцию;
5.  В описании должно присутствовать инфориация:
    - Цена последней сделки БА(по сути цена покупки БА);
    - Цена премии за один опцион;
    - Лотность БА;
    - Лотность опциона;
6. После исполнения опциона:
    - если цена БА ниже пола(опцион сработал), можно продолжить на этом БА, тоесть ёще раз закупиться под него опционов, можно продать БА видя его неперпекстивнось к волатильности, а можно оставить если есть предосылки к росту данного БА;
    - если цена БА выше потолка(опцион не сработал) - дожидаемся наилучшей ценыы БА и продаем его.

# ТАТНЕФТЬ PUT 575 27.11

In [31]:
import pandas as pd

# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
for indx in df_share['Unnamed: 0']:
    if (df_share['Лотность'][indx] * df_share['Цена последней сделки'][indx]) < 600:
        print(f"{df_share['Тикер'][indx]}\t{(df_share['Цена последней сделки'][indx] * df_share['Лотность'][indx]):.2f}\t{df_share['Название'][indx]}")

IRAO	375.70	Интер РАО ЕЭС
SNGSP	547.70	Сургутнефтегаз - привилегированные акции
ASTR	504.55	Группа Астра
ROSN	437.85	Роснефть
MTLR	96.63	Мечел
TATN	548.00	Татнефть
TATNP	546.10	Татнефть - привилегированные акции
VKCO	297.40	ВК
MAGN	365.45	Магнитогорский металлургический комбинат
VTBR	76.21	Банк ВТБ
RUAL	337.45	РУСАЛ
ALRS	475.20	АЛРОСА
PIKK	569.10	ПИК
AFLT	544.00	Аэрофлот


In [13]:
import os

import pandas as pd

from datetime import timedelta, datetime
import time

from math import floor, log10

from tinkoff.invest import AsyncClient
from tinkoff.invest.constants import INVEST_GRPC_API_SANDBOX
from tinkoff.invest.schemas import (
    Deviation,
    GetTechAnalysisRequest,
    IndicatorInterval,
    CandleInterval,
    CandleSource,
    IndicatorType,
    Smoothing,
    TypeOfPrice,
)
from tinkoff.invest.utils import decimal_to_quotation, now

from dotenv import load_dotenv

from bokeh.models import HoverTool
from bokeh.plotting import figure, show
# Активация Bokeh
from bokeh.io import output_notebook, curdoc

# Токен санбокс т-инвестиции
load_dotenv()
TOKEN_TIN_SanBox = os.getenv("TOKEN_TIN_SanBox")

# Пеобразование тин-чисел в флоат
def format_T_num_in_float(units, nano):
    if nano < 0:
        nano = abs(nano)
    if nano == 0:
        return float(units)
    return units + nano * 10**-(floor(log10(nano))+1)

# Введите данные по условию
# Тикер БА
ticker_BA = 'TATN'
# Дата исполнения
date_execution = pd.Timestamp(2024, 11, 27)
# Дата покупки(начала пролонгирования)
date_purchase = pd.Timestamp(2024, 11, 14)
# Цена премии опциона
premium_price = 14.44
# Лотность опциона
lots_opt = 1
# Количество покупаемых опционов на одну акцию
number_opt = 1
# Стоимость БА на момент покупки(пролонгирования)
price_BA_date_purchase = 571.5
# Цена безубыточности опциона
break_even_price_opt = 560.06

# Запрос дневных свечей за год
async def get_history_candles(
        period, interval, figi='figi', uid='uid'
):
    try:
        async with AsyncClient(TOKEN_TIN_SanBox,
                               target=INVEST_GRPC_API_SANDBOX) as client:
            historical_candles = await client.market_data.get_candles(
                figi=figi,
                from_=now() - timedelta(days=period),
                to=now(),
                interval=interval,
                instrument_id=uid,
                candle_source_type=CandleSource.CANDLE_SOURCE_EXCHANGE
            )
            historical_candles_array = {
                'Время свечи': [x.time for x in historical_candles.candles],
                'Цена открытия': [x.open for x in historical_candles.candles],
                'Макс цена': [x.high for x in historical_candles.candles],
                'Мин цена': [x.low for x in historical_candles.candles],
                'Цена закрытия': [format_T_num_in_float(units=x.close.units, nano=x.close.nano) for x in historical_candles.candles],
                'Сформированость': [x.is_complete for x in historical_candles.candles],
                'Объём торгов': [x.volume for x in historical_candles.candles],
                # 'Тип источника': [x.candle_source.name for x in historical_candles.candles],
            }
            df = pd.DataFrame.from_dict(historical_candles_array)
            return df
    except Exception as e:
        print(
            f"Тип исключения: {type(e).__name__}, сообщение: {str(e)}"
        )
        return None
# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
# Получить опционы из таблицы
df_opt = pd.read_excel(io='all_options.xlsx', engine='openpyxl')
# Получить индекс БА по тикеру инструмента
def index_by_ticker(tiker='tiker'):
    for tik, ind in zip(df_share['Тикер'], df_share['Unnamed: 0']):
        if tik == tiker:
            return [ind, tik]
# Получить массив исторических свеч по тикеру инструмента
ind_tik_BA = (index_by_ticker(tiker=ticker_BA))
df_candles_BA = await get_history_candles(365, CandleInterval.CANDLE_INTERVAL_DAY, figi=df_share['FIGI'][ind_tik_BA[0]], uid=df_share['UID'][ind_tik_BA[0]])

# Цена БА последней сделки
price_BA_last_transaction = df_candles_BA['Цена закрытия'][(len(df_candles_BA['Цена закрытия']) - 1)]

# Цена неубыточности БА
break_even_price_BA = price_BA_date_purchase + (premium_price * number_opt) 

# Тема графика
curdoc().theme = "dark_minimal"
# Панель инструментов
TOOLS = "pan,wheel_zoom,box_zoom,reset, crosshair"
# Данные для изображения в hover
data_in_hover = HoverTool(
    tooltips=[
        ( 'Дата',   '@x{%F}'),
        ( 'Цена закрытия',  '@y{%0.2f} руб' ),
    ],
    formatters={
        '@x': 'datetime',
        '@y': 'printf',
    },
)

# Создание поля графика
p = figure(title=f"{df_share['Название'][ind_tik_BA[0]]} PUT {575} {date_execution:%d.%m.%Y}", x_axis_type="datetime", sizing_mode="stretch_width", tools=[data_in_hover, TOOLS])
# Создание графика исторических свечей
p.line(df_candles_BA['Время свечи'], df_candles_BA['Цена закрытия'], color="blue", line_width=4, legend_label=f"БА: {df_share['Название'][ind_tik_BA[0]]}")

# Создание графика-box убыточности
# Справа
p.segment(x0=date_purchase, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_BA, legend_label="Дата покупки", line_color="gold", line_dash="8 8", line_width=2)
# Сверху
p.segment(x0=date_purchase, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_BA, legend_label="Цена неубыточности БА", line_color="mediumseagreen", line_dash="8 8", line_width=2)
# Слева
p.segment(x0=date_execution, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_opt, legend_label="Дата исполнения", line_color="navajowhite", line_dash="8 8", line_width=2)
# Снизу
p.segment(x0=date_execution, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_opt, legend_label="Цена неубыточности опциона", line_color="tomato", line_dash="8 8", line_width=2)

# Создание легенды
p.legend.title = f"Данные по опциону:\n\
Цена БА на на время покупки: {price_BA_date_purchase}\n\
Цена последней сделки БА: {price_BA_last_transaction}\n\
Динамика: \
{((price_BA_last_transaction - price_BA_date_purchase) / price_BA_date_purchase * 100):.2f}\
%\n\
Цена одного опциона: {premium_price}\n\
Лотность БА: {df_share['Лотность'][ind_tik_BA[0]]}\n\
Лотность опциона: {lots_opt}\n\
Соотношение БА к опциону: 1 к {number_opt}\n\
Цена БА неубыточности: {break_even_price_BA:.2f}"
p.legend.title_text_font_size = "16px"
p.legend.location = "top_left"
p.legend.title_text_color = 'orange'
print(break_even_price_opt)
print(break_even_price_BA)
# Активация Bokeh и вывод графика
output_notebook()
show(p)

560.06
585.94


Loading BokehJS ...

/home/sergo/TinvestOptions/venv/lib/python3.11/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


# ВК PUT 320 27.11

In [14]:
import os

import pandas as pd

from datetime import timedelta, datetime
import time

from math import floor, log10

from tinkoff.invest import AsyncClient
from tinkoff.invest.constants import INVEST_GRPC_API_SANDBOX
from tinkoff.invest.schemas import (
    Deviation,
    GetTechAnalysisRequest,
    IndicatorInterval,
    CandleInterval,
    CandleSource,
    IndicatorType,
    Smoothing,
    TypeOfPrice,
)
from tinkoff.invest.utils import decimal_to_quotation, now

from dotenv import load_dotenv

from bokeh.models import HoverTool
from bokeh.plotting import figure, show
# Активация Bokeh
from bokeh.io import output_notebook, curdoc

# Токен санбокс т-инвестиции
load_dotenv()
TOKEN_TIN_SanBox = os.getenv("TOKEN_TIN_SanBox")

# Пеобразование тин-чисел в флоат
def format_T_num_in_float(units, nano):
    if nano < 0:
        nano = abs(nano)
    if nano == 0:
        return float(units)
    return units + nano * 10**-(floor(log10(nano))+1)

# Введите данные по условию
# Тикер БА
ticker_BA = 'VKCO'
# Дата исполнения
date_execution = pd.Timestamp(2024, 11, 27)
# Дата покупки(начала пролонгирования)
date_purchase = pd.Timestamp(2024, 11, 14)
# Страйк
option_strike = 320
# Цена премии опциона
premium_price = 14.86
# Лотность опциона
lots_opt = 1
# Количество покупаемых опционов на одну акцию
number_opt = 1
# Стоимость БА на момент покупки(пролонгирования)
price_BA_date_purchase = 311.6
# Цена безубыточности опциона
break_even_price_opt = 304.64

# Запрос дневных свечей за год
async def get_history_candles(
        period, interval, figi='figi', uid='uid'
):
    try:
        async with AsyncClient(TOKEN_TIN_SanBox,
                               target=INVEST_GRPC_API_SANDBOX) as client:
            historical_candles = await client.market_data.get_candles(
                figi=figi,
                from_=now() - timedelta(days=period),
                to=now(),
                interval=interval,
                instrument_id=uid,
                candle_source_type=CandleSource.CANDLE_SOURCE_EXCHANGE
            )
            historical_candles_array = {
                'Время свечи': [x.time for x in historical_candles.candles],
                'Цена открытия': [x.open for x in historical_candles.candles],
                'Макс цена': [x.high for x in historical_candles.candles],
                'Мин цена': [x.low for x in historical_candles.candles],
                'Цена закрытия': [format_T_num_in_float(units=x.close.units, nano=x.close.nano) for x in historical_candles.candles],
                'Сформированость': [x.is_complete for x in historical_candles.candles],
                'Объём торгов': [x.volume for x in historical_candles.candles],
                # 'Тип источника': [x.candle_source.name for x in historical_candles.candles],
            }
            df = pd.DataFrame.from_dict(historical_candles_array)
            return df
    except Exception as e:
        print(
            f"Тип исключения: {type(e).__name__}, сообщение: {str(e)}"
        )
        return None
# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
# Получить опционы из таблицы
df_opt = pd.read_excel(io='all_options.xlsx', engine='openpyxl')
# Получить индекс БА по тикеру инструмента
def index_by_ticker(tiker='tiker'):
    for tik, ind in zip(df_share['Тикер'], df_share['Unnamed: 0']):
        if tik == tiker:
            return [ind, tik]
# Получить массив исторических свеч по тикеру инструмента
ind_tik_BA = (index_by_ticker(tiker=ticker_BA))
df_candles_BA = await get_history_candles(365, CandleInterval.CANDLE_INTERVAL_DAY, figi=df_share['FIGI'][ind_tik_BA[0]], uid=df_share['UID'][ind_tik_BA[0]])

# Цена БА последней сделки
price_BA_last_transaction = df_candles_BA['Цена закрытия'][(len(df_candles_BA['Цена закрытия']) - 1)]

# Цена неубыточности БА
break_even_price_BA = price_BA_date_purchase + (premium_price * number_opt) 

# Тема графика
curdoc().theme = "dark_minimal"
# Панель инструментов
TOOLS = "pan,wheel_zoom,box_zoom,reset, crosshair"
# Данные для изображения в hover
data_in_hover = HoverTool(
    tooltips=[
        ( 'Дата',   '@x{%F}'),
        ( 'Цена закрытия',  '@y{%0.2f} руб' ),
    ],
    formatters={
        '@x': 'datetime',
        '@y': 'printf',
    },
)

# Создание поля графика
p = figure(title=f"{df_share['Название'][ind_tik_BA[0]]} PUT {option_strike} {date_execution:%d.%m.%Y}", x_axis_type="datetime", sizing_mode="stretch_width", tools=[data_in_hover, TOOLS])
# Создание графика исторических свечей
p.line(df_candles_BA['Время свечи'], df_candles_BA['Цена закрытия'], color="blue", line_width=4, legend_label=f"БА: {df_share['Название'][ind_tik_BA[0]]}")

# Создание графика-box убыточности
# Справа
p.segment(x0=date_purchase, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_BA, legend_label="Дата покупки", line_color="gold", line_dash="8 8", line_width=2)
# Сверху
p.segment(x0=date_purchase, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_BA, legend_label="Цена неубыточности БА", line_color="mediumseagreen", line_dash="8 8", line_width=2)
# Слева
p.segment(x0=date_execution, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_opt, legend_label="Дата исполнения", line_color="navajowhite", line_dash="8 8", line_width=2)
# Снизу
p.segment(x0=date_execution, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_opt, legend_label="Цена неубыточности опциона", line_color="tomato", line_dash="8 8", line_width=2)

# Создание легенды
p.legend.title = f"Данные по опциону:\n\
Цена БА на на время покупки: {price_BA_date_purchase}\n\
Цена последней сделки БА: {price_BA_last_transaction}\n\
Динамика: \
{((price_BA_last_transaction - price_BA_date_purchase) / price_BA_date_purchase * 100):.2f}\
%\n\
Цена одного опциона: {premium_price}\n\
Лотность БА: {df_share['Лотность'][ind_tik_BA[0]]}\n\
Лотность опциона: {lots_opt}\n\
Соотношение БА к опциону: 1 к {number_opt}\n\
Цена БА неубыточности: {break_even_price_BA:.2f}"
p.legend.title_text_font_size = "16px"
p.legend.location = "top_left"
p.legend.title_text_color = 'orange'
print(break_even_price_opt)
print(break_even_price_BA)
# Активация Bokeh и вывод графика
output_notebook()
show(p)

304.64
326.46000000000004


Loading BokehJS ...

/home/sergo/TinvestOptions/venv/lib/python3.11/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


# РосНефть PUT 475 04.12

In [15]:
import os

import pandas as pd

from datetime import timedelta, datetime
import time

from math import floor, log10

from tinkoff.invest import AsyncClient
from tinkoff.invest.constants import INVEST_GRPC_API_SANDBOX
from tinkoff.invest.schemas import (
    Deviation,
    GetTechAnalysisRequest,
    IndicatorInterval,
    CandleInterval,
    CandleSource,
    IndicatorType,
    Smoothing,
    TypeOfPrice,
)
from tinkoff.invest.utils import decimal_to_quotation, now

from dotenv import load_dotenv

from bokeh.models import HoverTool
from bokeh.plotting import figure, show
# Активация Bokeh
from bokeh.io import output_notebook, curdoc

# Токен санбокс т-инвестиции
load_dotenv()
TOKEN_TIN_SanBox = os.getenv("TOKEN_TIN_SanBox")

# Пеобразование тин-чисел в флоат
def format_T_num_in_float(units, nano):
    if nano < 0:
        nano = abs(nano)
    if nano == 0:
        return float(units)
    return units + nano * 10**-(floor(log10(nano))+1)

# Введите данные по условию
# Тикер БА
ticker_BA = 'ROSN'
# Дата исполнения
date_execution = pd.Timestamp(2024, 12, 4)
# Дата покупки(начала пролонгирования)
date_purchase = pd.Timestamp(2024, 11, 19)
# Страйк
option_strike = 475
# Цена премии опциона
premium_price = 18.33
# Лотность опциона
lots_opt = 1
# Количество покупаемых опционов на одну акцию
number_opt = 1
# Стоимость БА на момент покупки(пролонгирования)
price_BA_date_purchase = 462.9
# Цена безубыточности опциона
break_even_price_opt = 456.12

# Запрос дневных свечей за год
async def get_history_candles(
        period, interval, figi='figi', uid='uid'
):
    try:
        async with AsyncClient(TOKEN_TIN_SanBox,
                               target=INVEST_GRPC_API_SANDBOX) as client:
            historical_candles = await client.market_data.get_candles(
                figi=figi,
                from_=now() - timedelta(days=period),
                to=now(),
                interval=interval,
                instrument_id=uid,
                candle_source_type=CandleSource.CANDLE_SOURCE_EXCHANGE
            )
            historical_candles_array = {
                'Время свечи': [x.time for x in historical_candles.candles],
                'Цена открытия': [x.open for x in historical_candles.candles],
                'Макс цена': [x.high for x in historical_candles.candles],
                'Мин цена': [x.low for x in historical_candles.candles],
                'Цена закрытия': [format_T_num_in_float(units=x.close.units, nano=x.close.nano) for x in historical_candles.candles],
                'Сформированость': [x.is_complete for x in historical_candles.candles],
                'Объём торгов': [x.volume for x in historical_candles.candles],
                # 'Тип источника': [x.candle_source.name for x in historical_candles.candles],
            }
            df = pd.DataFrame.from_dict(historical_candles_array)
            return df
    except Exception as e:
        print(
            f"Тип исключения: {type(e).__name__}, сообщение: {str(e)}"
        )
        return None
# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
# Получить опционы из таблицы
df_opt = pd.read_excel(io='all_options.xlsx', engine='openpyxl')
# Получить индекс БА по тикеру инструмента
def index_by_ticker(tiker='tiker'):
    for tik, ind in zip(df_share['Тикер'], df_share['Unnamed: 0']):
        if tik == tiker:
            return [ind, tik]
# Получить массив исторических свеч по тикеру инструмента
ind_tik_BA = (index_by_ticker(tiker=ticker_BA))
df_candles_BA = await get_history_candles(365, CandleInterval.CANDLE_INTERVAL_DAY, figi=df_share['FIGI'][ind_tik_BA[0]], uid=df_share['UID'][ind_tik_BA[0]])

# Цена БА последней сделки
price_BA_last_transaction = df_candles_BA['Цена закрытия'][(len(df_candles_BA['Цена закрытия']) - 1)]

# Цена неубыточности БА
break_even_price_BA = price_BA_date_purchase + (premium_price * number_opt) 

# Тема графика
curdoc().theme = "dark_minimal"
# Панель инструментов
TOOLS = "pan,wheel_zoom,box_zoom,reset, crosshair"
# Данные для изображения в hover
data_in_hover = HoverTool(
    tooltips=[
        ( 'Дата',   '@x{%F}'),
        ( 'Цена закрытия',  '@y{%0.2f} руб' ),
    ],
    formatters={
        '@x': 'datetime',
        '@y': 'printf',
    },
)

# Создание поля графика
p = figure(title=f"{df_share['Название'][ind_tik_BA[0]]} PUT {option_strike} {date_execution:%d.%m.%Y}", x_axis_type="datetime", sizing_mode="stretch_width", tools=[data_in_hover, TOOLS])
# Создание графика исторических свечей
p.line(df_candles_BA['Время свечи'], df_candles_BA['Цена закрытия'], color="blue", line_width=4, legend_label=f"БА: {df_share['Название'][ind_tik_BA[0]]}")

# Создание графика-box убыточности
# Справа
p.segment(x0=date_purchase, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_BA, legend_label="Дата покупки", line_color="gold", line_dash="8 8", line_width=2)
# Сверху
p.segment(x0=date_purchase, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_BA, legend_label="Цена неубыточности БА", line_color="mediumseagreen", line_dash="8 8", line_width=2)
# Слева
p.segment(x0=date_execution, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_opt, legend_label="Дата исполнения", line_color="navajowhite", line_dash="8 8", line_width=2)
# Снизу
p.segment(x0=date_execution, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_opt, legend_label="Цена неубыточности опциона", line_color="tomato", line_dash="8 8", line_width=2)

# Создание легенды
p.legend.title = f"Данные по опциону:\n\
Цена БА на на время покупки: {price_BA_date_purchase}\n\
Цена последней сделки БА: {price_BA_last_transaction}\n\
Динамика: \
{((price_BA_last_transaction - price_BA_date_purchase) / price_BA_date_purchase * 100):.2f}\
%\n\
Цена одного опциона: {premium_price}\n\
Лотность БА: {df_share['Лотность'][ind_tik_BA[0]]}\n\
Лотность опциона: {lots_opt}\n\
Соотношение БА к опциону: 1 к {number_opt}\n\
Цена БА неубыточности: {break_even_price_BA:.2f}"
p.legend.title_text_font_size = "16px"
p.legend.location = "top_left"
p.legend.title_text_color = 'orange'
print(break_even_price_opt)
print(break_even_price_BA)
# Активация Bokeh и вывод графика
output_notebook()
show(p)

456.12
481.22999999999996


Loading BokehJS ...

/home/sergo/TinvestOptions/venv/lib/python3.11/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


# Аэрофлот PUT 61.5 27.11

In [16]:
import os

import pandas as pd

from datetime import timedelta, datetime
import time

from math import floor, log10

from tinkoff.invest import AsyncClient
from tinkoff.invest.constants import INVEST_GRPC_API_SANDBOX
from tinkoff.invest.schemas import (
    Deviation,
    GetTechAnalysisRequest,
    IndicatorInterval,
    CandleInterval,
    CandleSource,
    IndicatorType,
    Smoothing,
    TypeOfPrice,
)
from tinkoff.invest.utils import decimal_to_quotation, now

from dotenv import load_dotenv

from bokeh.models import HoverTool
from bokeh.plotting import figure, show
# Активация Bokeh
from bokeh.io import output_notebook, curdoc

# Токен санбокс т-инвестиции
load_dotenv()
TOKEN_TIN_SanBox = os.getenv("TOKEN_TIN_SanBox")

# Пеобразование тин-чисел в флоат
def format_T_num_in_float(units, nano):
    if nano < 0:
        nano = abs(nano)
    if nano == 0:
        return float(units)
    return units + nano * 10**-(floor(log10(nano))+1)

# Введите данные по условию
# Тикер БА
ticker_BA = 'AFLT'
# Дата исполнения
date_execution = pd.Timestamp(2024, 11, 27)
# Дата покупки(начала пролонгирования)
date_purchase = pd.Timestamp(2024, 11, 19)
# Страйк
option_strike = 61.5
# Цена премии опциона
premium_price = 1.81
# Лотность опциона
lots_opt = 10
# Количество покупаемых опционов на одну акцию
number_opt = 1
# Стоимость БА на момент покупки(пролонгирования)
price_BA_date_purchase = 60.8
# Цена безубыточности опциона
break_even_price_opt = 59.66

# Запрос дневных свечей за год
async def get_history_candles(
        period, interval, figi='figi', uid='uid'
):
    try:
        async with AsyncClient(TOKEN_TIN_SanBox,
                               target=INVEST_GRPC_API_SANDBOX) as client:
            historical_candles = await client.market_data.get_candles(
                figi=figi,
                from_=now() - timedelta(days=period),
                to=now(),
                interval=interval,
                instrument_id=uid,
                candle_source_type=CandleSource.CANDLE_SOURCE_EXCHANGE
            )
            historical_candles_array = {
                'Время свечи': [x.time for x in historical_candles.candles],
                'Цена открытия': [x.open for x in historical_candles.candles],
                'Макс цена': [x.high for x in historical_candles.candles],
                'Мин цена': [x.low for x in historical_candles.candles],
                'Цена закрытия': [format_T_num_in_float(units=x.close.units, nano=x.close.nano) for x in historical_candles.candles],
                'Сформированость': [x.is_complete for x in historical_candles.candles],
                'Объём торгов': [x.volume for x in historical_candles.candles],
                # 'Тип источника': [x.candle_source.name for x in historical_candles.candles],
            }
            df = pd.DataFrame.from_dict(historical_candles_array)
            return df
    except Exception as e:
        print(
            f"Тип исключения: {type(e).__name__}, сообщение: {str(e)}"
        )
        return None
# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
# Получить опционы из таблицы
df_opt = pd.read_excel(io='all_options.xlsx', engine='openpyxl')
# Получить индекс БА по тикеру инструмента
def index_by_ticker(tiker='tiker'):
    for tik, ind in zip(df_share['Тикер'], df_share['Unnamed: 0']):
        if tik == tiker:
            return [ind, tik]
# Получить массив исторических свеч по тикеру инструмента
ind_tik_BA = (index_by_ticker(tiker=ticker_BA))
df_candles_BA = await get_history_candles(365, CandleInterval.CANDLE_INTERVAL_DAY, figi=df_share['FIGI'][ind_tik_BA[0]], uid=df_share['UID'][ind_tik_BA[0]])

# Цена БА последней сделки
price_BA_last_transaction = df_candles_BA['Цена закрытия'][(len(df_candles_BA['Цена закрытия']) - 1)]

# Цена неубыточности БА
break_even_price_BA = price_BA_date_purchase + (premium_price * number_opt) 

# Тема графика
curdoc().theme = "dark_minimal"
# Панель инструментов
TOOLS = "pan,wheel_zoom,box_zoom,reset, crosshair"
# Данные для изображения в hover
data_in_hover = HoverTool(
    tooltips=[
        ( 'Дата',   '@x{%F}'),
        ( 'Цена закрытия',  '@y{%0.2f} руб' ),
    ],
    formatters={
        '@x': 'datetime',
        '@y': 'printf',
    },
)

# Создание поля графика
p = figure(title=f"{df_share['Название'][ind_tik_BA[0]]} PUT {option_strike} {date_execution:%d.%m.%Y}", x_axis_type="datetime", sizing_mode="stretch_width", tools=[data_in_hover, TOOLS])
# Создание графика исторических свечей
p.line(df_candles_BA['Время свечи'], df_candles_BA['Цена закрытия'], color="blue", line_width=4, legend_label=f"БА: {df_share['Название'][ind_tik_BA[0]]}")

# Создание графика-box убыточности
# Справа
p.segment(x0=date_purchase, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_BA, legend_label="Дата покупки", line_color="gold", line_dash="8 8", line_width=2)
# Сверху
p.segment(x0=date_purchase, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_BA, legend_label="Цена неубыточности БА", line_color="mediumseagreen", line_dash="8 8", line_width=2)
# Слева
p.segment(x0=date_execution, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_opt, legend_label="Дата исполнения", line_color="navajowhite", line_dash="8 8", line_width=2)
# Снизу
p.segment(x0=date_execution, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_opt, legend_label="Цена неубыточности опциона", line_color="tomato", line_dash="8 8", line_width=2)

# Создание легенды
p.legend.title = f"Данные по опциону:\n\
Цена БА на на время покупки: {price_BA_date_purchase}\n\
Цена последней сделки БА: {price_BA_last_transaction}\n\
Динамика: \
{((price_BA_last_transaction - price_BA_date_purchase) / price_BA_date_purchase * 100):.2f}\
%\n\
Цена одного опциона: {premium_price}\n\
Лотность БА: {df_share['Лотность'][ind_tik_BA[0]]}\n\
Лотность опциона: {lots_opt}\n\
Соотношение БА к опциону: 1 к {number_opt}\n\
Цена БА неубыточности: {break_even_price_BA:.2f}"
p.legend.title_text_font_size = "16px"
p.legend.location = "top_left"
p.legend.title_text_color = 'orange'
print(break_even_price_opt)
print(break_even_price_BA)
# Активация Bokeh и вывод графика
output_notebook()
show(p)

59.66
62.61


Loading BokehJS ...

/home/sergo/TinvestOptions/venv/lib/python3.11/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


# Интер РАО ЕЭС PUT 4Р 18.12

In [5]:
import pandas as pd

# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
for indx in df_share['Unnamed: 0']:
    if (df_share['Лотность'][indx] * df_share['Цена последней сделки'][indx]) < 600:
        print(f"{df_share['Тикер'][indx]}\t{(df_share['Цена последней сделки'][indx] * df_share['Лотность'][indx]):.2f}\t{df_share['Название'][indx]}")

IRAO	375.70	Интер РАО ЕЭС
SNGSP	547.70	Сургутнефтегаз - привилегированные акции
ASTR	504.55	Группа Астра
ROSN	437.85	Роснефть
MTLR	96.63	Мечел
TATN	548.00	Татнефть
TATNP	546.10	Татнефть - привилегированные акции
VKCO	297.40	ВК
MAGN	365.45	Магнитогорский металлургический комбинат
VTBR	76.21	Банк ВТБ
RUAL	337.45	РУСАЛ
ALRS	475.20	АЛРОСА
PIKK	569.10	ПИК
AFLT	544.00	Аэрофлот


In [17]:
import os

import pandas as pd

from datetime import timedelta, datetime
import time

from math import floor, log10

from tinkoff.invest import AsyncClient
from tinkoff.invest.constants import INVEST_GRPC_API_SANDBOX
from tinkoff.invest.schemas import (
    Deviation,
    GetTechAnalysisRequest,
    IndicatorInterval,
    CandleInterval,
    CandleSource,
    IndicatorType,
    Smoothing,
    TypeOfPrice,
)
from tinkoff.invest.utils import decimal_to_quotation, now

from dotenv import load_dotenv

from bokeh.models import HoverTool
from bokeh.plotting import figure, show
# Активация Bokeh
from bokeh.io import output_notebook, curdoc

# Токен санбокс т-инвестиции
load_dotenv()
TOKEN_TIN_SanBox = os.getenv("TOKEN_TIN_SanBox")

# Пеобразование тин-чисел в флоат
def format_T_num_in_float(units, nano):
    if nano < 0:
        nano = abs(nano)
    if nano == 0:
        return float(units)
    return units + nano * 10**-(floor(log10(nano))+1)

# Введите данные по условию
# Тикер БА
ticker_BA = 'IRAO'
# Дата исполнения
date_execution = pd.Timestamp(2024, 12, 18)
# Дата покупки(начала пролонгирования)
date_purchase = pd.Timestamp(2024, 11, 25)
# Страйк
option_strike = 4
# Цена премии опциона
premium_price = 0.189
# Лотность опциона
lots_opt = 100
# Количество покупаемых опционов на один лот БА
number_opt = 1
# Стоимость БА на момент покупки(пролонгирования)
price_BA_date_purchase = 3.853
# Цена безубыточности опциона
break_even_price_opt = 3.81

# Запрос дневных свечей за год
async def get_history_candles(
        period, interval, figi='figi', uid='uid'
):
    try:
        async with AsyncClient(TOKEN_TIN_SanBox,
                               target=INVEST_GRPC_API_SANDBOX) as client:
            historical_candles = await client.market_data.get_candles(
                figi=figi,
                from_=now() - timedelta(days=period),
                to=now(),
                interval=interval,
                instrument_id=uid,
                candle_source_type=CandleSource.CANDLE_SOURCE_EXCHANGE
            )
            historical_candles_array = {
                'Время свечи': [x.time for x in historical_candles.candles],
                'Цена открытия': [x.open for x in historical_candles.candles],
                'Макс цена': [x.high for x in historical_candles.candles],
                'Мин цена': [x.low for x in historical_candles.candles],
                'Цена закрытия': [format_T_num_in_float(units=x.close.units, nano=x.close.nano) for x in historical_candles.candles],
                'Сформированость': [x.is_complete for x in historical_candles.candles],
                'Объём торгов': [x.volume for x in historical_candles.candles],
                # 'Тип источника': [x.candle_source.name for x in historical_candles.candles],
            }
            df = pd.DataFrame.from_dict(historical_candles_array)
            return df
    except Exception as e:
        print(
            f"Тип исключения: {type(e).__name__}, сообщение: {str(e)}"
        )
        return None
# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
# Получить опционы из таблицы
df_opt = pd.read_excel(io='all_options.xlsx', engine='openpyxl')
# Получить индекс БА по тикеру инструмента
def index_by_ticker(tiker='tiker'):
    for tik, ind in zip(df_share['Тикер'], df_share['Unnamed: 0']):
        if tik == tiker:
            return [ind, tik]
# Получить массив исторических свеч по тикеру инструмента
ind_tik_BA = (index_by_ticker(tiker=ticker_BA))
df_candles_BA = await get_history_candles(365, CandleInterval.CANDLE_INTERVAL_DAY, figi=df_share['FIGI'][ind_tik_BA[0]], uid=df_share['UID'][ind_tik_BA[0]])

# Цена БА последней сделки
price_BA_last_transaction = df_candles_BA['Цена закрытия'][(len(df_candles_BA['Цена закрытия']) - 1)]

# Цена неубыточности БА
break_even_price_BA = price_BA_date_purchase + (premium_price * number_opt) 

# Тема графика
curdoc().theme = "dark_minimal"
# Панель инструментов
TOOLS = "pan,wheel_zoom,box_zoom,reset, crosshair"
# Данные для изображения в hover
data_in_hover = HoverTool(
    tooltips=[
        ( 'Дата',   '@x{%F}'),
        ( 'Цена закрытия',  '@y{%0.2f} руб' ),
    ],
    formatters={
        '@x': 'datetime',
        '@y': 'printf',
    },
)

# Создание поля графика
p = figure(title=f"{df_share['Название'][ind_tik_BA[0]]} PUT {option_strike} {date_execution:%d.%m.%Y}", x_axis_type="datetime", sizing_mode="stretch_width", tools=[data_in_hover, TOOLS])
# Создание графика исторических свечей
p.line(df_candles_BA['Время свечи'], df_candles_BA['Цена закрытия'], color="blue", line_width=4, legend_label=f"БА: {df_share['Название'][ind_tik_BA[0]]}")

# Создание графика-box убыточности
# Справа
p.segment(x0=date_purchase, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_BA, legend_label="Дата покупки", line_color="gold", line_dash="8 8", line_width=2)
# Сверху
p.segment(x0=date_purchase, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_BA, legend_label="Цена неубыточности БА", line_color="mediumseagreen", line_dash="8 8", line_width=2)
# Слева
p.segment(x0=date_execution, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_opt, legend_label="Дата исполнения", line_color="navajowhite", line_dash="8 8", line_width=2)
# Снизу
p.segment(x0=date_execution, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_opt, legend_label="Цена неубыточности опциона", line_color="tomato", line_dash="8 8", line_width=2)

# Создание легенды
p.legend.title = f"Данные по опциону:\n\
Цена БА на на время покупки: {price_BA_date_purchase}\n\
Цена последней сделки БА: {price_BA_last_transaction}\n\
Динамика: \
{((price_BA_last_transaction - price_BA_date_purchase) / price_BA_date_purchase * 100):.2f}\
%\n\
Цена одного опциона: {premium_price}\n\
Лотность БА: {df_share['Лотность'][ind_tik_BA[0]]}\n\
Лотность опциона: {lots_opt}\n\
Соотношение БА к опциону: 1 к {number_opt}\n\
Цена БА неубыточности: {break_even_price_BA:.2f}"
p.legend.title_text_font_size = "16px"
p.legend.location = "top_left"
p.legend.title_text_color = 'orange'
print(break_even_price_opt)
print(break_even_price_BA)
# Активация Bokeh и вывод графика
output_notebook()
show(p)

3.81
4.042


Loading BokehJS ...

/home/sergo/TinvestOptions/venv/lib/python3.11/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


# РУСАЛ PUT 36 18.12

In [18]:
import os

import pandas as pd

from datetime import timedelta, datetime
import time

from math import floor, log10

from tinkoff.invest import AsyncClient
from tinkoff.invest.constants import INVEST_GRPC_API_SANDBOX
from tinkoff.invest.schemas import (
    Deviation,
    GetTechAnalysisRequest,
    IndicatorInterval,
    CandleInterval,
    CandleSource,
    IndicatorType,
    Smoothing,
    TypeOfPrice,
)
from tinkoff.invest.utils import decimal_to_quotation, now

from dotenv import load_dotenv

from bokeh.models import HoverTool
from bokeh.plotting import figure, show
# Активация Bokeh
from bokeh.io import output_notebook, curdoc

# Токен санбокс т-инвестиции
load_dotenv()
TOKEN_TIN_SanBox = os.getenv("TOKEN_TIN_SanBox")

# Пеобразование тин-чисел в флоат
def format_T_num_in_float(units, nano):
    if nano < 0:
        nano = abs(nano)
    if nano == 0:
        return float(units)
    return units + nano * 10**-(floor(log10(nano))+1)

# Введите данные по условию
# Тикер БА
ticker_BA = 'RUAL'
# Дата исполнения
date_execution = pd.Timestamp(2024, 12, 18)
# Дата покупки(начала пролонгирования)
date_purchase = pd.Timestamp(2024, 11, 25)
# Страйк
option_strike = 36
# Цена премии опциона
premium_price = 1.93
# Лотность опциона
lots_opt = 10
# Количество покупаемых опционов на один лот БА
number_opt = 1
# Стоимость БА на момент покупки(пролонгирования)
price_BA_date_purchase = 34.485
# Цена безубыточности опциона
break_even_price_opt = 33.96

# Запрос дневных свечей за год
async def get_history_candles(
        period, interval, figi='figi', uid='uid'
):
    try:
        async with AsyncClient(TOKEN_TIN_SanBox,
                               target=INVEST_GRPC_API_SANDBOX) as client:
            historical_candles = await client.market_data.get_candles(
                figi=figi,
                from_=now() - timedelta(days=period),
                to=now(),
                interval=interval,
                instrument_id=uid,
                candle_source_type=CandleSource.CANDLE_SOURCE_EXCHANGE
            )
            historical_candles_array = {
                'Время свечи': [x.time for x in historical_candles.candles],
                'Цена открытия': [x.open for x in historical_candles.candles],
                'Макс цена': [x.high for x in historical_candles.candles],
                'Мин цена': [x.low for x in historical_candles.candles],
                'Цена закрытия': [format_T_num_in_float(units=x.close.units, nano=x.close.nano) for x in historical_candles.candles],
                'Сформированость': [x.is_complete for x in historical_candles.candles],
                'Объём торгов': [x.volume for x in historical_candles.candles],
                # 'Тип источника': [x.candle_source.name for x in historical_candles.candles],
            }
            df = pd.DataFrame.from_dict(historical_candles_array)
            return df
    except Exception as e:
        print(
            f"Тип исключения: {type(e).__name__}, сообщение: {str(e)}"
        )
        return None
# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
# Получить опционы из таблицы
df_opt = pd.read_excel(io='all_options.xlsx', engine='openpyxl')
# Получить индекс БА по тикеру инструмента
def index_by_ticker(tiker='tiker'):
    for tik, ind in zip(df_share['Тикер'], df_share['Unnamed: 0']):
        if tik == tiker:
            return [ind, tik]
# Получить массив исторических свеч по тикеру инструмента
ind_tik_BA = (index_by_ticker(tiker=ticker_BA))
df_candles_BA = await get_history_candles(365, CandleInterval.CANDLE_INTERVAL_DAY, figi=df_share['FIGI'][ind_tik_BA[0]], uid=df_share['UID'][ind_tik_BA[0]])

# Цена БА последней сделки
price_BA_last_transaction = df_candles_BA['Цена закрытия'][(len(df_candles_BA['Цена закрытия']) - 1)]

# Цена неубыточности БА
break_even_price_BA = price_BA_date_purchase + (premium_price * number_opt) 

# Тема графика
curdoc().theme = "dark_minimal"
# Панель инструментов
TOOLS = "pan,wheel_zoom,box_zoom,reset, crosshair"
# Данные для изображения в hover
data_in_hover = HoverTool(
    tooltips=[
        ( 'Дата',   '@x{%F}'),
        ( 'Цена закрытия',  '@y{%0.2f} руб' ),
    ],
    formatters={
        '@x': 'datetime',
        '@y': 'printf',
    },
)

# Создание поля графика
p = figure(title=f"{df_share['Название'][ind_tik_BA[0]]} PUT {option_strike} {date_execution:%d.%m.%Y}", x_axis_type="datetime", sizing_mode="stretch_width", tools=[data_in_hover, TOOLS])
# Создание графика исторических свечей
p.line(df_candles_BA['Время свечи'], df_candles_BA['Цена закрытия'], color="blue", line_width=4, legend_label=f"БА: {df_share['Название'][ind_tik_BA[0]]}")

# Создание графика-box убыточности
# Справа
p.segment(x0=date_purchase, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_BA, legend_label="Дата покупки", line_color="gold", line_dash="8 8", line_width=2)
# Сверху
p.segment(x0=date_purchase, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_BA, legend_label="Цена неубыточности БА", line_color="mediumseagreen", line_dash="8 8", line_width=2)
# Слева
p.segment(x0=date_execution, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_opt, legend_label="Дата исполнения", line_color="navajowhite", line_dash="8 8", line_width=2)
# Снизу
p.segment(x0=date_execution, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_opt, legend_label="Цена неубыточности опциона", line_color="tomato", line_dash="8 8", line_width=2)

# Создание легенды
p.legend.title = f"Данные по опциону:\n\
Цена БА на на время покупки: {price_BA_date_purchase}\n\
Цена последней сделки БА: {price_BA_last_transaction}\n\
Динамика: \
{((price_BA_last_transaction - price_BA_date_purchase) / price_BA_date_purchase * 100):.2f}\
%\n\
Цена одного опциона: {premium_price}\n\
Лотность БА: {df_share['Лотность'][ind_tik_BA[0]]}\n\
Лотность опциона: {lots_opt}\n\
Соотношение БА к опциону: 1 к {number_opt}\n\
Цена БА неубыточности: {break_even_price_BA:.2f}"
p.legend.title_text_font_size = "16px"
p.legend.location = "top_left"
p.legend.title_text_color = 'orange'
print(break_even_price_opt)
print(break_even_price_BA)
# Активация Bokeh и вывод графика
output_notebook()
show(p)

33.96
36.415


Loading BokehJS ...

/home/sergo/TinvestOptions/venv/lib/python3.11/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


# Проверка стратегии stragl 
----

## Описание
>Вместо покупки БА в противовес PUT будет покупаться CALL с тем же сроком исполнения, окупаемость верхней границы: прибыльность опциона CALL + премия опциона PUT(контрпозиции); окупаемость нижней границы: прибыльность опциона PUT - премия опциона CALL(контрпозиция).

In [18]:
import pandas as pd

# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
for indx in df_share['Unnamed: 0']:
    if (df_share['Лотность'][indx] * df_share['Цена последней сделки'][indx]) < 600:
        print(f"{df_share['Тикер'][indx]}\t{(df_share['Цена последней сделки'][indx] * df_share['Лотность'][indx]):.2f}\t{df_share['Название'][indx]}")

IRAO	375.70	Интер РАО ЕЭС
SNGSP	547.70	Сургутнефтегаз - привилегированные акции
ASTR	504.55	Группа Астра
ROSN	437.85	Роснефть
MTLR	96.63	Мечел
TATN	548.00	Татнефть
TATNP	546.10	Татнефть - привилегированные акции
VKCO	297.40	ВК
MAGN	365.45	Магнитогорский металлургический комбинат
VTBR	76.21	Банк ВТБ
RUAL	337.45	РУСАЛ
ALRS	475.20	АЛРОСА
PIKK	569.10	ПИК
AFLT	544.00	Аэрофлот


In [24]:
import os

import pandas as pd

from datetime import timedelta, datetime
import time

from math import floor, log10

from tinkoff.invest import AsyncClient
from tinkoff.invest.constants import INVEST_GRPC_API_SANDBOX
from tinkoff.invest.schemas import (
    Deviation,
    GetTechAnalysisRequest,
    IndicatorInterval,
    CandleInterval,
    CandleSource,
    IndicatorType,
    Smoothing,
    TypeOfPrice,
)
from tinkoff.invest.utils import decimal_to_quotation, now

from dotenv import load_dotenv

from bokeh.models import HoverTool
from bokeh.plotting import figure, show
# Активация Bokeh
from bokeh.io import output_notebook, curdoc

# Токен санбокс т-инвестиции
load_dotenv()
TOKEN_TIN_SanBox = os.getenv("TOKEN_TIN_SanBox")

# Пеобразование тин-чисел в флоат
def format_T_num_in_float(units, nano):
    if nano < 0:
        nano = abs(nano)
    if nano == 0:
        return float(units)
    return units + nano * 10**-(floor(log10(nano))+1)

# Введите данные по условию
# Тикер БА
ticker_BA = 'SNGSP'
# Дата исполнения
date_execution = pd.Timestamp(2024, 12, 18)
# Дата покупки(начала пролонгирования)
date_purchase = pd.Timestamp(2024, 11, 27)
# Страйк CALL
option_strike_CALL = 66
# Страйк PUT
option_strike_PUT = 56
# Цена премии опциона CALL
premium_price_CALL = 0.44
# Цена премии опциона PUT
premium_price_PUT = 0.72
# Лотность опциона
lots_opt = 1
# Количество покупаемых опционов на один лот БА
number_opt = 1
# Стоимость БА на момент покупки(пролонгирования)
price_BA_date_purchase = 59.555
# Цена безубыточности опциона CALL
break_even_price_opt_CALL = 66.49
# Цена безубыточности опциона PUT
break_even_price_opt_PUT = 55.23

# Запрос дневных свечей за год
async def get_history_candles(
        period, interval, figi='figi', uid='uid'
):
    try:
        async with AsyncClient(TOKEN_TIN_SanBox,
                               target=INVEST_GRPC_API_SANDBOX) as client:
            historical_candles = await client.market_data.get_candles(
                figi=figi,
                from_=now() - timedelta(days=period),
                to=now(),
                interval=interval,
                instrument_id=uid,
                candle_source_type=CandleSource.CANDLE_SOURCE_EXCHANGE
            )
            historical_candles_array = {
                'Время свечи': [x.time for x in historical_candles.candles],
                'Цена открытия': [x.open for x in historical_candles.candles],
                'Макс цена': [x.high for x in historical_candles.candles],
                'Мин цена': [x.low for x in historical_candles.candles],
                'Цена закрытия': [format_T_num_in_float(units=x.close.units, nano=x.close.nano) for x in historical_candles.candles],
                'Сформированость': [x.is_complete for x in historical_candles.candles],
                'Объём торгов': [x.volume for x in historical_candles.candles],
                # 'Тип источника': [x.candle_source.name for x in historical_candles.candles],
            }
            df = pd.DataFrame.from_dict(historical_candles_array)
            return df
    except Exception as e:
        print(
            f"Тип исключения: {type(e).__name__}, сообщение: {str(e)}"
        )
        return None
# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
# Получить опционы из таблицы
# df_opt = pd.read_excel(io='all_options.xlsx', engine='openpyxl')
# Получить индекс БА по тикеру инструмента
def index_by_ticker(tiker='tiker'):
    for tik, ind in zip(df_share['Тикер'], df_share['Unnamed: 0']):
        if tik == tiker:
            return [ind, tik]
# Получить массив исторических свеч по тикеру инструмента
ind_tik_BA = (index_by_ticker(tiker=ticker_BA))
df_candles_BA = await get_history_candles(365, CandleInterval.CANDLE_INTERVAL_DAY, figi=df_share['FIGI'][ind_tik_BA[0]], uid=df_share['UID'][ind_tik_BA[0]])

# Цена БА последней сделки
price_BA_last_transaction = df_candles_BA['Цена закрытия'][(len(df_candles_BA['Цена закрытия']) - 1)]

# Цена неубыточности CALL(верхняя граница)
break_CALL = break_even_price_opt_CALL + (premium_price_PUT ) 

# Цена неубыточности PUT(нижняя граница)
break_PUT = break_even_price_opt_PUT - (premium_price_CALL)

# Тема графика
curdoc().theme = "dark_minimal"
# Панель инструментов
TOOLS = "pan,wheel_zoom,box_zoom,reset, crosshair"
# Данные для изображения в hover
data_in_hover = HoverTool(
    tooltips=[
        ( 'Дата',   '@x{%F}'),
        ( 'Цена закрытия',  '@y{%0.2f} руб' ),
    ],
    formatters={
        '@x': 'datetime',
        '@y': 'printf',
    },
)

# Создание поля графика
p = figure(title=f"{df_share['Название'][ind_tik_BA[0]]} PUT {option_strike_CALL} {date_execution:%d.%m.%Y}", x_axis_type="datetime", sizing_mode="stretch_width", tools=[data_in_hover, TOOLS])
# Создание графика исторических свечей
p.line(df_candles_BA['Время свечи'], df_candles_BA['Цена закрытия'], color="blue", line_width=4, legend_label=f"БА: {df_share['Название'][ind_tik_BA[0]]}")

# Создание графика-box убыточности
# Справа
p.segment(
    x0=date_purchase, y0=break_PUT, x1=date_purchase, y1=break_CALL,
    legend_label="Дата покупки", line_color="gold", line_dash="8 8", line_width=2
)
# Сверху
p.segment(
    x0=date_purchase, y0=break_CALL, x1=date_execution, y1=break_CALL,
    legend_label="Цена неубыточности БА", line_color="mediumseagreen", line_dash="8 8",
    line_width=2
)
# Слева
p.segment(
    x0=date_execution, y0=break_CALL, x1=date_execution, y1=break_PUT,
    legend_label="Дата исполнения", line_color="navajowhite", line_dash="8 8", line_width=2
)
# Снизу
p.segment(
    x0=date_execution, y0=break_PUT, x1=date_purchase, y1=break_PUT,
    legend_label="Цена неубыточности опциона", line_color="tomato", line_dash="8 8",
    line_width=2
)

# Создание легенды
p.legend.title = f"{break_CALL} - {break_PUT}\n{premium_price_CALL} - {premium_price_PUT}"
# p.legend.title = f"Данные по опциону:\n\
# Цена БА на на время покупки: {price_BA_date_purchase}\n\
# Цена последней сделки БА: {price_BA_last_transaction}\n\
# Динамика: \
# {((price_BA_last_transaction - price_BA_date_purchase) / price_BA_date_purchase * 100):.2f}\
# %\n\
# Цена одного опциона: {premium_price}\n\
# Лотность БА: {df_share['Лотность'][ind_tik_BA[0]]}\n\
# Лотность опциона: {lots_opt}\n\
# Соотношение БА к опциону: 1 к {number_opt}\n\
# Цена БА неубыточности: {break_even_price_BA:.2f}"
p.legend.title_text_font_size = "16px"
p.legend.location = "top_left"
p.legend.title_text_color = 'orange'

# Активация Bokeh и вывод графика
output_notebook()
show(p)

Loading BokehJS ...

/home/sergo/TinvestOptions/venv/lib/python3.11/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


Заключение: даннаю стратегию можно ещё раз опробовать после стабилизации рынка - хотя не очень верится в её успех, так как даже на сегодняшний день с большой волотильностью и непредсказуемостью поведения БА нашего рынка стратегия мало просчитываема, а с учетом добавления позиции окупаемости на премию контропциона наврядли будет прибыльна, если её гипотетически перложить на более спокойное время, то доход на ней может быть только на крайне рисковых позициях - короче КАЗИНО! 

# АЛРОСА PUT 50 4.12

In [5]:
import pandas as pd

# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
for indx in df_share['Unnamed: 0']:
    if (df_share['Лотность'][indx] * df_share['Цена последней сделки'][indx]) < 600:
        print(f"{df_share['Тикер'][indx]}\t{(df_share['Цена последней сделки'][indx] * df_share['Лотность'][indx]):.2f}\t{df_share['Название'][indx]}")

IRAO	375.70	Интер РАО ЕЭС
SNGSP	547.70	Сургутнефтегаз - привилегированные акции
ASTR	504.55	Группа Астра
ROSN	437.85	Роснефть
MTLR	96.63	Мечел
TATN	548.00	Татнефть
TATNP	546.10	Татнефть - привилегированные акции
VKCO	297.40	ВК
MAGN	365.45	Магнитогорский металлургический комбинат
VTBR	76.21	Банк ВТБ
RUAL	337.45	РУСАЛ
ALRS	475.20	АЛРОСА
PIKK	569.10	ПИК
AFLT	544.00	Аэрофлот


In [19]:
import os

import pandas as pd

from datetime import timedelta, datetime
import time

from math import floor, log10

from tinkoff.invest import AsyncClient
from tinkoff.invest.constants import INVEST_GRPC_API_SANDBOX
from tinkoff.invest.schemas import (
    Deviation,
    GetTechAnalysisRequest,
    IndicatorInterval,
    CandleInterval,
    CandleSource,
    IndicatorType,
    Smoothing,
    TypeOfPrice,
)
from tinkoff.invest.utils import decimal_to_quotation, now

from dotenv import load_dotenv

from bokeh.models import HoverTool
from bokeh.plotting import figure, show
# Активация Bokeh
from bokeh.io import output_notebook, curdoc

# Токен санбокс т-инвестиции
load_dotenv()
TOKEN_TIN_SanBox = os.getenv("TOKEN_TIN_SanBox")

# Пеобразование тин-чисел в флоат
def format_T_num_in_float(units, nano):
    if nano < 0:
        nano = abs(nano)
    if nano == 0:
        return float(units)
    return units + nano * 10**-(floor(log10(nano))+1)

# Введите данные по условию
# Тикер БА
ticker_BA = 'ALRS'
# Дата исполнения
date_execution = pd.Timestamp(2024, 12, 4)
# Дата покупки(начала пролонгирования)
date_purchase = pd.Timestamp(2024, 11, 28)
# Страйк
option_strike = 50
# Цена премии опциона
premium_price = 1.54
# Лотность опциона
lots_opt = 1
# Количество покупаемых опционов на один лот БА
number_opt = 1
# Стоимость БА на момент покупки(пролонгирования)
price_BA_date_purchase = 48.99
# Цена безубыточности опциона
break_even_price_opt = 48.15

# Запрос дневных свечей за год
async def get_history_candles(
        period, interval, figi='figi', uid='uid'
):
    try:
        async with AsyncClient(TOKEN_TIN_SanBox,
                               target=INVEST_GRPC_API_SANDBOX) as client:
            historical_candles = await client.market_data.get_candles(
                figi=figi,
                from_=now() - timedelta(days=period),
                to=now(),
                interval=interval,
                instrument_id=uid,
                candle_source_type=CandleSource.CANDLE_SOURCE_EXCHANGE
            )
            historical_candles_array = {
                'Время свечи': [x.time for x in historical_candles.candles],
                'Цена открытия': [x.open for x in historical_candles.candles],
                'Макс цена': [x.high for x in historical_candles.candles],
                'Мин цена': [x.low for x in historical_candles.candles],
                'Цена закрытия': [format_T_num_in_float(units=x.close.units, nano=x.close.nano) for x in historical_candles.candles],
                'Сформированость': [x.is_complete for x in historical_candles.candles],
                'Объём торгов': [x.volume for x in historical_candles.candles],
                # 'Тип источника': [x.candle_source.name for x in historical_candles.candles],
            }
            df = pd.DataFrame.from_dict(historical_candles_array)
            return df
    except Exception as e:
        print(
            f"Тип исключения: {type(e).__name__}, сообщение: {str(e)}"
        )
        return None
# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
# Получить опционы из таблицы
df_opt = pd.read_excel(io='all_options.xlsx', engine='openpyxl')
# Получить индекс БА по тикеру инструмента
def index_by_ticker(tiker='tiker'):
    for tik, ind in zip(df_share['Тикер'], df_share['Unnamed: 0']):
        if tik == tiker:
            return [ind, tik]
# Получить массив исторических свеч по тикеру инструмента
ind_tik_BA = (index_by_ticker(tiker=ticker_BA))
df_candles_BA = await get_history_candles(
    365, CandleInterval.CANDLE_INTERVAL_DAY, figi=df_share['FIGI'][ind_tik_BA[0]],
    uid=df_share['UID'][ind_tik_BA[0]]
)

# Цена БА последней сделки
price_BA_last_transaction = df_candles_BA['Цена закрытия'][(len(df_candles_BA['Цена закрытия']) - 1)]

# Цена неубыточности БА
break_even_price_BA = price_BA_date_purchase + (premium_price * number_opt) 

# Тема графика
curdoc().theme = "dark_minimal"
# Панель инструментов
TOOLS = "pan,wheel_zoom,box_zoom,reset, crosshair"
# Данные для изображения в hover
data_in_hover = HoverTool(
    tooltips=[
        ( 'Дата',   '@x{%F}'),
        ( 'Цена закрытия',  '@y{%0.2f} руб' ),
    ],
    formatters={
        '@x': 'datetime',
        '@y': 'printf',
    },
)

# Создание поля графика
p = figure(title=f"{df_share['Название'][ind_tik_BA[0]]} PUT {option_strike} {date_execution:%d.%m.%Y}", x_axis_type="datetime", sizing_mode="stretch_width", tools=[data_in_hover, TOOLS])
# Создание графика исторических свечей
p.line(df_candles_BA['Время свечи'], df_candles_BA['Цена закрытия'], color="blue", line_width=4, legend_label=f"БА: {df_share['Название'][ind_tik_BA[0]]}")

# Создание графика-box убыточности
# Справа
p.segment(x0=date_purchase, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_BA, legend_label="Дата покупки", line_color="gold", line_dash="8 8", line_width=2)
# Сверху
p.segment(x0=date_purchase, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_BA, legend_label="Цена неубыточности БА", line_color="mediumseagreen", line_dash="8 8", line_width=2)
# Слева
p.segment(x0=date_execution, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_opt, legend_label="Дата исполнения", line_color="navajowhite", line_dash="8 8", line_width=2)
# Снизу
p.segment(x0=date_execution, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_opt, legend_label="Цена неубыточности опциона", line_color="tomato", line_dash="8 8", line_width=2)

# Создание легенды
p.legend.title = f"Данные по опциону:\n\
Цена БА на на время покупки: {price_BA_date_purchase}\n\
Цена последней сделки БА: {price_BA_last_transaction}\n\
Динамика: \
{((price_BA_last_transaction - price_BA_date_purchase) / price_BA_date_purchase * 100):.2f}\
%\n\
Цена одного опциона: {premium_price}\n\
Лотность БА: {df_share['Лотность'][ind_tik_BA[0]]}\n\
Лотность опциона: {lots_opt}\n\
Соотношение БА к опциону: 1 к {number_opt}\n\
Цена БА неубыточности: {break_even_price_BA:.2f}"
p.legend.title_text_font_size = "16px"
p.legend.location = "top_left"
p.legend.title_text_color = 'orange'
print(break_even_price_opt)
print(break_even_price_BA)
# Активация Bokeh и вывод графика
output_notebook()
show(p)

48.15
50.53


Loading BokehJS ...

/home/sergo/TinvestOptions/venv/lib/python3.11/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


# ВК PUT 280 18.12

In [20]:
import os

import pandas as pd

from datetime import timedelta, datetime
import time

from math import floor, log10

from tinkoff.invest import AsyncClient
from tinkoff.invest.constants import INVEST_GRPC_API_SANDBOX
from tinkoff.invest.schemas import (
    Deviation,
    GetTechAnalysisRequest,
    IndicatorInterval,
    CandleInterval,
    CandleSource,
    IndicatorType,
    Smoothing,
    TypeOfPrice,
)
from tinkoff.invest.utils import decimal_to_quotation, now

from dotenv import load_dotenv

from bokeh.models import HoverTool
from bokeh.plotting import figure, show
# Активация Bokeh
from bokeh.io import output_notebook, curdoc

# Токен санбокс т-инвестиции
load_dotenv()
TOKEN_TIN_SanBox = os.getenv("TOKEN_TIN_SanBox")

# Пеобразование тин-чисел в флоат
def format_T_num_in_float(units, nano):
    if nano < 0:
        nano = abs(nano)
    if nano == 0:
        return float(units)
    return units + nano * 10**-(floor(log10(nano))+1)

# Введите данные по условию
# Тикер БА
ticker_BA = 'VKCO'
# Дата исполнения
date_execution = pd.Timestamp(2024, 12, 18)
# Дата покупки(начала пролонгирования)
date_purchase = pd.Timestamp(2024, 12, 3)
# Страйк
option_strike = 280
# Цена премии опциона
premium_price = 18.67
# Лотность опциона
lots_opt = 1
# Количество покупаемых опционов на один лот БА
number_opt = 1
# Стоимость БА на момент покупки(пролонгирования)
price_BA_date_purchase = 268.4
# Цена безубыточности опциона
break_even_price_opt = 260.77

# Запрос дневных свечей за год
async def get_history_candles(
        period, interval, figi='figi', uid='uid'
):
    try:
        async with AsyncClient(TOKEN_TIN_SanBox,
                               target=INVEST_GRPC_API_SANDBOX) as client:
            historical_candles = await client.market_data.get_candles(
                figi=figi,
                from_=now() - timedelta(days=period),
                to=now(),
                interval=interval,
                instrument_id=uid,
                candle_source_type=CandleSource.CANDLE_SOURCE_EXCHANGE
            )
            historical_candles_array = {
                'Время свечи': [x.time for x in historical_candles.candles],
                'Цена открытия': [x.open for x in historical_candles.candles],
                'Макс цена': [x.high for x in historical_candles.candles],
                'Мин цена': [x.low for x in historical_candles.candles],
                'Цена закрытия': [format_T_num_in_float(units=x.close.units, nano=x.close.nano) for x in historical_candles.candles],
                'Сформированость': [x.is_complete for x in historical_candles.candles],
                'Объём торгов': [x.volume for x in historical_candles.candles],
                # 'Тип источника': [x.candle_source.name for x in historical_candles.candles],
            }
            df = pd.DataFrame.from_dict(historical_candles_array)
            return df
    except Exception as e:
        print(
            f"Тип исключения: {type(e).__name__}, сообщение: {str(e)}"
        )
        return None
# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
# Получить опционы из таблицы
df_opt = pd.read_excel(io='all_options.xlsx', engine='openpyxl')
# Получить индекс БА по тикеру инструмента
def index_by_ticker(tiker='tiker'):
    for tik, ind in zip(df_share['Тикер'], df_share['Unnamed: 0']):
        if tik == tiker:
            return [ind, tik]
# Получить массив исторических свеч по тикеру инструмента
ind_tik_BA = (index_by_ticker(tiker=ticker_BA))
df_candles_BA = await get_history_candles(
    365, CandleInterval.CANDLE_INTERVAL_DAY, figi=df_share['FIGI'][ind_tik_BA[0]],
    uid=df_share['UID'][ind_tik_BA[0]]
)

# Цена БА последней сделки
price_BA_last_transaction = df_candles_BA['Цена закрытия'][(len(df_candles_BA['Цена закрытия']) - 1)]

# Цена неубыточности БА
break_even_price_BA = price_BA_date_purchase + (premium_price * number_opt) 

# Тема графика
curdoc().theme = "dark_minimal"
# Панель инструментов
TOOLS = "pan,wheel_zoom,box_zoom,reset, crosshair"
# Данные для изображения в hover
data_in_hover = HoverTool(
    tooltips=[
        ( 'Дата',   '@x{%F}'),
        ( 'Цена закрытия',  '@y{%0.2f} руб' ),
    ],
    formatters={
        '@x': 'datetime',
        '@y': 'printf',
    },
)

# Создание поля графика
p = figure(title=f"{df_share['Название'][ind_tik_BA[0]]} PUT {option_strike} {date_execution:%d.%m.%Y}", x_axis_type="datetime", sizing_mode="stretch_width", tools=[data_in_hover, TOOLS])
# Создание графика исторических свечей
p.line(df_candles_BA['Время свечи'], df_candles_BA['Цена закрытия'], color="blue", line_width=4, legend_label=f"БА: {df_share['Название'][ind_tik_BA[0]]}")

# Создание графика-box убыточности
# Справа
p.segment(x0=date_purchase, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_BA, legend_label="Дата покупки", line_color="gold", line_dash="8 8", line_width=2)
# Сверху
p.segment(x0=date_purchase, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_BA, legend_label="Цена неубыточности БА", line_color="mediumseagreen", line_dash="8 8", line_width=2)
# Слева
p.segment(x0=date_execution, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_opt, legend_label="Дата исполнения", line_color="navajowhite", line_dash="8 8", line_width=2)
# Снизу
p.segment(x0=date_execution, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_opt, legend_label="Цена неубыточности опциона", line_color="tomato", line_dash="8 8", line_width=2)

# Создание легенды
p.legend.title = f"Данные по опциону:\n\
Цена БА на на время покупки: {price_BA_date_purchase}\n\
Цена последней сделки БА: {price_BA_last_transaction}\n\
Динамика: \
{((price_BA_last_transaction - price_BA_date_purchase) / price_BA_date_purchase * 100):.2f}\
%\n\
Цена одного опциона: {premium_price}\n\
Лотность БА: {df_share['Лотность'][ind_tik_BA[0]]}\n\
Лотность опциона: {lots_opt}\n\
Соотношение БА к опциону: 1 к {number_opt}\n\
Цена БА неубыточности: {break_even_price_BA:.2f}"
p.legend.title_text_font_size = "16px"
p.legend.location = "top_left"
p.legend.title_text_color = 'orange'
print(break_even_price_opt)
print(break_even_price_BA)
# Активация Bokeh и вывод графика
output_notebook()
show(p)

260.77
287.07


Loading BokehJS ...

/home/sergo/TinvestOptions/venv/lib/python3.11/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


# Мечел PUT 100 18.12

In [21]:
import os

import pandas as pd

from datetime import timedelta, datetime
import time

from math import floor, log10

from tinkoff.invest import AsyncClient
from tinkoff.invest.constants import INVEST_GRPC_API_SANDBOX
from tinkoff.invest.schemas import (
    Deviation,
    GetTechAnalysisRequest,
    IndicatorInterval,
    CandleInterval,
    CandleSource,
    IndicatorType,
    Smoothing,
    TypeOfPrice,
)
from tinkoff.invest.utils import decimal_to_quotation, now

from dotenv import load_dotenv

from bokeh.models import HoverTool
from bokeh.plotting import figure, show
# Активация Bokeh
from bokeh.io import output_notebook, curdoc

# Токен санбокс т-инвестиции
load_dotenv()
TOKEN_TIN_SanBox = os.getenv("TOKEN_TIN_SanBox")

# Пеобразование тин-чисел в флоат
def format_T_num_in_float(units, nano):
    if nano < 0:
        nano = abs(nano)
    if nano == 0:
        return float(units)
    return units + nano * 10**-(floor(log10(nano))+1)

# Введите данные по условию
# Тикер БА
ticker_BA = 'MTLR'
# Дата исполнения
date_execution = pd.Timestamp(2024, 12, 18)
# Дата покупки(начала пролонгирования)
date_purchase = pd.Timestamp(2024, 12, 3)
# Страйк
option_strike = 100
# Цена премии опциона
premium_price = 17.75
# Лотность опциона
lots_opt = 1
# Количество покупаемых опционов на один лот БА
number_opt = 1
# Стоимость БА на момент покупки(пролонгирования)
price_BA_date_purchase = 82.58
# Цена безубыточности опциона
break_even_price_opt = 81.72

# Запрос дневных свечей за год
async def get_history_candles(
        period, interval, figi='figi', uid='uid'
):
    try:
        async with AsyncClient(TOKEN_TIN_SanBox,
                               target=INVEST_GRPC_API_SANDBOX) as client:
            historical_candles = await client.market_data.get_candles(
                figi=figi,
                from_=now() - timedelta(days=period),
                to=now(),
                interval=interval,
                instrument_id=uid,
                candle_source_type=CandleSource.CANDLE_SOURCE_EXCHANGE
            )
            historical_candles_array = {
                'Время свечи': [x.time for x in historical_candles.candles],
                'Цена открытия': [x.open for x in historical_candles.candles],
                'Макс цена': [x.high for x in historical_candles.candles],
                'Мин цена': [x.low for x in historical_candles.candles],
                'Цена закрытия': [format_T_num_in_float(units=x.close.units, nano=x.close.nano) for x in historical_candles.candles],
                'Сформированость': [x.is_complete for x in historical_candles.candles],
                'Объём торгов': [x.volume for x in historical_candles.candles],
                # 'Тип источника': [x.candle_source.name for x in historical_candles.candles],
            }
            df = pd.DataFrame.from_dict(historical_candles_array)
            return df
    except Exception as e:
        print(
            f"Тип исключения: {type(e).__name__}, сообщение: {str(e)}"
        )
        return None
# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
# Получить опционы из таблицы
df_opt = pd.read_excel(io='all_options.xlsx', engine='openpyxl')
# Получить индекс БА по тикеру инструмента
def index_by_ticker(tiker='tiker'):
    for tik, ind in zip(df_share['Тикер'], df_share['Unnamed: 0']):
        if tik == tiker:
            return [ind, tik]
# Получить массив исторических свеч по тикеру инструмента
ind_tik_BA = (index_by_ticker(tiker=ticker_BA))
df_candles_BA = await get_history_candles(
    365, CandleInterval.CANDLE_INTERVAL_DAY, figi=df_share['FIGI'][ind_tik_BA[0]],
    uid=df_share['UID'][ind_tik_BA[0]]
)

# Цена БА последней сделки
price_BA_last_transaction = df_candles_BA['Цена закрытия'][(len(df_candles_BA['Цена закрытия']) - 1)]

# Цена неубыточности БА
break_even_price_BA = price_BA_date_purchase + (premium_price * number_opt) 

# Тема графика
curdoc().theme = "dark_minimal"
# Панель инструментов
TOOLS = "pan,wheel_zoom,box_zoom,reset, crosshair"
# Данные для изображения в hover
data_in_hover = HoverTool(
    tooltips=[
        ( 'Дата',   '@x{%F}'),
        ( 'Цена закрытия',  '@y{%0.2f} руб' ),
    ],
    formatters={
        '@x': 'datetime',
        '@y': 'printf',
    },
)

# Создание поля графика
p = figure(title=f"{df_share['Название'][ind_tik_BA[0]]} PUT {option_strike} {date_execution:%d.%m.%Y}", x_axis_type="datetime", sizing_mode="stretch_width", tools=[data_in_hover, TOOLS])
# Создание графика исторических свечей
p.line(df_candles_BA['Время свечи'], df_candles_BA['Цена закрытия'], color="blue", line_width=4, legend_label=f"БА: {df_share['Название'][ind_tik_BA[0]]}")

# Создание графика-box убыточности
# Справа
p.segment(x0=date_purchase, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_BA, legend_label="Дата покупки", line_color="gold", line_dash="8 8", line_width=2)
# Сверху
p.segment(x0=date_purchase, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_BA, legend_label="Цена неубыточности БА", line_color="mediumseagreen", line_dash="8 8", line_width=2)
# Слева
p.segment(x0=date_execution, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_opt, legend_label="Дата исполнения", line_color="navajowhite", line_dash="8 8", line_width=2)
# Снизу
p.segment(x0=date_execution, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_opt, legend_label="Цена неубыточности опциона", line_color="tomato", line_dash="8 8", line_width=2)

# Создание легенды
p.legend.title = f"Данные по опциону:\n\
Цена БА на на время покупки: {price_BA_date_purchase}\n\
Цена последней сделки БА: {price_BA_last_transaction}\n\
Динамика: \
{((price_BA_last_transaction - price_BA_date_purchase) / price_BA_date_purchase * 100):.2f}\
%\n\
Цена одного опциона: {premium_price}\n\
Лотность БА: {df_share['Лотность'][ind_tik_BA[0]]}\n\
Лотность опциона: {lots_opt}\n\
Соотношение БА к опциону: 1 к {number_opt}\n\
Цена БА неубыточности: {break_even_price_BA:.2f}"
p.legend.title_text_font_size = "16px"
p.legend.location = "top_left"
p.legend.title_text_color = 'orange'
print(break_even_price_opt)
print(break_even_price_BA)
# Активация Bokeh и вывод графика
output_notebook()
show(p)

81.72
100.33


Loading BokehJS ...

/home/sergo/TinvestOptions/venv/lib/python3.11/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


# Татнефть PUT 550 18.12

In [22]:
import os

import pandas as pd

from datetime import timedelta, datetime
import time

from math import floor, log10

from tinkoff.invest import AsyncClient
from tinkoff.invest.constants import INVEST_GRPC_API_SANDBOX
from tinkoff.invest.schemas import (
    Deviation,
    GetTechAnalysisRequest,
    IndicatorInterval,
    CandleInterval,
    CandleSource,
    IndicatorType,
    Smoothing,
    TypeOfPrice,
)
from tinkoff.invest.utils import decimal_to_quotation, now

from dotenv import load_dotenv

from bokeh.models import HoverTool
from bokeh.plotting import figure, show
# Активация Bokeh
from bokeh.io import output_notebook, curdoc

# Токен санбокс т-инвестиции
load_dotenv()
TOKEN_TIN_SanBox = os.getenv("TOKEN_TIN_SanBox")

# Пеобразование тин-чисел в флоат
def format_T_num_in_float(units, nano):
    if nano < 0:
        nano = abs(nano)
    if nano == 0:
        return float(units)
    return units + nano * 10**-(floor(log10(nano))+1)

# Введите данные по условию
# Тикер БА
ticker_BA = 'TATN'
# Дата исполнения
date_execution = pd.Timestamp(2024, 12, 18)
# Дата покупки(начала пролонгирования)
date_purchase = pd.Timestamp(2024, 12, 3)
# Страйк
option_strike = 550
# Цена премии опциона
premium_price = 18.79
# Лотность опциона
lots_opt = 1
# Количество покупаемых опционов на один лот БА
number_opt = 1
# Стоимость БА на момент покупки(пролонгирования)
price_BA_date_purchase = 545.7
# Цена безубыточности опциона
break_even_price_opt = 530.65

# Запрос дневных свечей за год
async def get_history_candles(
        period, interval, figi='figi', uid='uid'
):
    try:
        async with AsyncClient(TOKEN_TIN_SanBox,
                               target=INVEST_GRPC_API_SANDBOX) as client:
            historical_candles = await client.market_data.get_candles(
                figi=figi,
                from_=now() - timedelta(days=period),
                to=now(),
                interval=interval,
                instrument_id=uid,
                candle_source_type=CandleSource.CANDLE_SOURCE_EXCHANGE
            )
            historical_candles_array = {
                'Время свечи': [x.time for x in historical_candles.candles],
                'Цена открытия': [x.open for x in historical_candles.candles],
                'Макс цена': [x.high for x in historical_candles.candles],
                'Мин цена': [x.low for x in historical_candles.candles],
                'Цена закрытия': [format_T_num_in_float(units=x.close.units, nano=x.close.nano) for x in historical_candles.candles],
                'Сформированость': [x.is_complete for x in historical_candles.candles],
                'Объём торгов': [x.volume for x in historical_candles.candles],
                # 'Тип источника': [x.candle_source.name for x in historical_candles.candles],
            }
            df = pd.DataFrame.from_dict(historical_candles_array)
            return df
    except Exception as e:
        print(
            f"Тип исключения: {type(e).__name__}, сообщение: {str(e)}"
        )
        return None
# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
# Получить опционы из таблицы
df_opt = pd.read_excel(io='all_options.xlsx', engine='openpyxl')
# Получить индекс БА по тикеру инструмента
def index_by_ticker(tiker='tiker'):
    for tik, ind in zip(df_share['Тикер'], df_share['Unnamed: 0']):
        if tik == tiker:
            return [ind, tik]
# Получить массив исторических свеч по тикеру инструмента
ind_tik_BA = (index_by_ticker(tiker=ticker_BA))
df_candles_BA = await get_history_candles(
    365, CandleInterval.CANDLE_INTERVAL_DAY, figi=df_share['FIGI'][ind_tik_BA[0]],
    uid=df_share['UID'][ind_tik_BA[0]]
)

# Цена БА последней сделки
price_BA_last_transaction = df_candles_BA['Цена закрытия'][(len(df_candles_BA['Цена закрытия']) - 1)]

# Цена неубыточности БА
break_even_price_BA = price_BA_date_purchase + (premium_price * number_opt) 

# Тема графика
curdoc().theme = "dark_minimal"
# Панель инструментов
TOOLS = "pan,wheel_zoom,box_zoom,reset, crosshair"
# Данные для изображения в hover
data_in_hover = HoverTool(
    tooltips=[
        ( 'Дата',   '@x{%F}'),
        ( 'Цена закрытия',  '@y{%0.2f} руб' ),
    ],
    formatters={
        '@x': 'datetime',
        '@y': 'printf',
    },
)

# Создание поля графика
p = figure(title=f"{df_share['Название'][ind_tik_BA[0]]} PUT {option_strike} {date_execution:%d.%m.%Y}", x_axis_type="datetime", sizing_mode="stretch_width", tools=[data_in_hover, TOOLS])
# Создание графика исторических свечей
p.line(df_candles_BA['Время свечи'], df_candles_BA['Цена закрытия'], color="blue", line_width=4, legend_label=f"БА: {df_share['Название'][ind_tik_BA[0]]}")

# Создание графика-box убыточности
# Справа
p.segment(x0=date_purchase, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_BA, legend_label="Дата покупки", line_color="gold", line_dash="8 8", line_width=2)
# Сверху
p.segment(x0=date_purchase, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_BA, legend_label="Цена неубыточности БА", line_color="mediumseagreen", line_dash="8 8", line_width=2)
# Слева
p.segment(x0=date_execution, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_opt, legend_label="Дата исполнения", line_color="navajowhite", line_dash="8 8", line_width=2)
# Снизу
p.segment(x0=date_execution, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_opt, legend_label="Цена неубыточности опциона", line_color="tomato", line_dash="8 8", line_width=2)

# Создание легенды
p.legend.title = f"Данные по опциону:\n\
Цена БА на на время покупки: {price_BA_date_purchase}\n\
Цена последней сделки БА: {price_BA_last_transaction}\n\
Динамика: \
{((price_BA_last_transaction - price_BA_date_purchase) / price_BA_date_purchase * 100):.2f}\
%\n\
Цена одного опциона: {premium_price}\n\
Лотность БА: {df_share['Лотность'][ind_tik_BA[0]]}\n\
Лотность опциона: {lots_opt}\n\
Соотношение БА к опциону: 1 к {number_opt}\n\
Цена БА неубыточности: {break_even_price_BA:.2f}"
p.legend.title_text_font_size = "16px"
p.legend.location = "top_left"
p.legend.title_text_color = 'orange'
print(break_even_price_opt)
print(break_even_price_BA)
# Активация Bokeh и вывод графика
output_notebook()
show(p)

530.65
564.49


Loading BokehJS ...

/home/sergo/TinvestOptions/venv/lib/python3.11/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


# Роснефть PUT 475 11.12

In [23]:
import os

import pandas as pd

from datetime import timedelta, datetime
import time

from math import floor, log10

from tinkoff.invest import AsyncClient
from tinkoff.invest.constants import INVEST_GRPC_API_SANDBOX
from tinkoff.invest.schemas import (
    Deviation,
    GetTechAnalysisRequest,
    IndicatorInterval,
    CandleInterval,
    CandleSource,
    IndicatorType,
    Smoothing,
    TypeOfPrice,
)
from tinkoff.invest.utils import decimal_to_quotation, now

from dotenv import load_dotenv

from bokeh.models import HoverTool
from bokeh.plotting import figure, show
# Активация Bokeh
from bokeh.io import output_notebook, curdoc

# Токен санбокс т-инвестиции
load_dotenv()
TOKEN_TIN_SanBox = os.getenv("TOKEN_TIN_SanBox")

# Пеобразование тин-чисел в флоат
def format_T_num_in_float(units, nano):
    if nano < 0:
        nano = abs(nano)
    if nano == 0:
        return float(units)
    return units + nano * 10**-(floor(log10(nano))+1)

# Введите данные по условию
# Тикер БА
ticker_BA = 'ROSN'
# Дата исполнения
date_execution = pd.Timestamp(2024, 12, 11)
# Дата покупки(начала пролонгирования)
date_purchase = pd.Timestamp(2024, 12, 3)
# Страйк
option_strike = 475
# Цена премии опциона
premium_price = 7.02
# Лотность опциона
lots_opt = 1
# Количество покупаемых опционов на один лот БА
number_opt = 1
# Стоимость БА на момент покупки(пролонгирования)
price_BA_date_purchase = 487.75
# Цена безубыточности опциона
break_even_price_opt = 467.48

# Запрос дневных свечей за год
async def get_history_candles(
        period, interval, figi='figi', uid='uid'
):
    try:
        async with AsyncClient(TOKEN_TIN_SanBox,
                               target=INVEST_GRPC_API_SANDBOX) as client:
            historical_candles = await client.market_data.get_candles(
                figi=figi,
                from_=now() - timedelta(days=period),
                to=now(),
                interval=interval,
                instrument_id=uid,
                candle_source_type=CandleSource.CANDLE_SOURCE_EXCHANGE
            )
            historical_candles_array = {
                'Время свечи': [x.time for x in historical_candles.candles],
                'Цена открытия': [x.open for x in historical_candles.candles],
                'Макс цена': [x.high for x in historical_candles.candles],
                'Мин цена': [x.low for x in historical_candles.candles],
                'Цена закрытия': [format_T_num_in_float(units=x.close.units, nano=x.close.nano) for x in historical_candles.candles],
                'Сформированость': [x.is_complete for x in historical_candles.candles],
                'Объём торгов': [x.volume for x in historical_candles.candles],
                # 'Тип источника': [x.candle_source.name for x in historical_candles.candles],
            }
            df = pd.DataFrame.from_dict(historical_candles_array)
            return df
    except Exception as e:
        print(
            f"Тип исключения: {type(e).__name__}, сообщение: {str(e)}"
        )
        return None
# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
# Получить опционы из таблицы
df_opt = pd.read_excel(io='all_options.xlsx', engine='openpyxl')
# Получить индекс БА по тикеру инструмента
def index_by_ticker(tiker='tiker'):
    for tik, ind in zip(df_share['Тикер'], df_share['Unnamed: 0']):
        if tik == tiker:
            return [ind, tik]
# Получить массив исторических свеч по тикеру инструмента
ind_tik_BA = (index_by_ticker(tiker=ticker_BA))
df_candles_BA = await get_history_candles(
    365, CandleInterval.CANDLE_INTERVAL_DAY, figi=df_share['FIGI'][ind_tik_BA[0]],
    uid=df_share['UID'][ind_tik_BA[0]]
)

# Цена БА последней сделки
price_BA_last_transaction = df_candles_BA['Цена закрытия'][(len(df_candles_BA['Цена закрытия']) - 1)]

# Цена неубыточности БА
break_even_price_BA = price_BA_date_purchase + (premium_price * number_opt) 

# Тема графика
curdoc().theme = "dark_minimal"
# Панель инструментов
TOOLS = "pan,wheel_zoom,box_zoom,reset, crosshair"
# Данные для изображения в hover
data_in_hover = HoverTool(
    tooltips=[
        ( 'Дата',   '@x{%F}'),
        ( 'Цена закрытия',  '@y{%0.2f} руб' ),
    ],
    formatters={
        '@x': 'datetime',
        '@y': 'printf',
    },
)

# Создание поля графика
p = figure(title=f"{df_share['Название'][ind_tik_BA[0]]} PUT {option_strike} {date_execution:%d.%m.%Y}", x_axis_type="datetime", sizing_mode="stretch_width", tools=[data_in_hover, TOOLS])
# Создание графика исторических свечей
p.line(df_candles_BA['Время свечи'], df_candles_BA['Цена закрытия'], color="blue", line_width=4, legend_label=f"БА: {df_share['Название'][ind_tik_BA[0]]}")

# Создание графика-box убыточности
# Справа
p.segment(x0=date_purchase, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_BA, legend_label="Дата покупки", line_color="gold", line_dash="8 8", line_width=2)
# Сверху
p.segment(x0=date_purchase, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_BA, legend_label="Цена неубыточности БА", line_color="mediumseagreen", line_dash="8 8", line_width=2)
# Слева
p.segment(x0=date_execution, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_opt, legend_label="Дата исполнения", line_color="navajowhite", line_dash="8 8", line_width=2)
# Снизу
p.segment(x0=date_execution, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_opt, legend_label="Цена неубыточности опциона", line_color="tomato", line_dash="8 8", line_width=2)

# Создание легенды
p.legend.title = f"Данные по опциону:\n\
Цена БА на на время покупки: {price_BA_date_purchase}\n\
Цена последней сделки БА: {price_BA_last_transaction}\n\
Динамика: \
{((price_BA_last_transaction - price_BA_date_purchase) / price_BA_date_purchase * 100):.2f}\
%\n\
Цена одного опциона: {premium_price}\n\
Лотность БА: {df_share['Лотность'][ind_tik_BA[0]]}\n\
Лотность опциона: {lots_opt}\n\
Соотношение БА к опциону: 1 к {number_opt}\n\
Цена БА неубыточности: {break_even_price_BA:.2f}"
p.legend.title_text_font_size = "16px"
p.legend.location = "top_left"
p.legend.title_text_color = 'orange'
print(break_even_price_opt)
print(break_even_price_BA)
# Активация Bokeh и вывод графика
output_notebook()
show(p)

467.48
494.77


Loading BokehJS ...

/home/sergo/TinvestOptions/venv/lib/python3.11/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


# ММК PUT 32 18.12

In [24]:
import os

import pandas as pd

from datetime import timedelta, datetime
import time

from math import floor, log10

from tinkoff.invest import AsyncClient
from tinkoff.invest.constants import INVEST_GRPC_API_SANDBOX
from tinkoff.invest.schemas import (
    Deviation,
    GetTechAnalysisRequest,
    IndicatorInterval,
    CandleInterval,
    CandleSource,
    IndicatorType,
    Smoothing,
    TypeOfPrice,
)
from tinkoff.invest.utils import decimal_to_quotation, now

from dotenv import load_dotenv

from bokeh.models import HoverTool
from bokeh.plotting import figure, show
# Активация Bokeh
from bokeh.io import output_notebook, curdoc

# Токен санбокс т-инвестиции
load_dotenv()
TOKEN_TIN_SanBox = os.getenv("TOKEN_TIN_SanBox")

# Пеобразование тин-чисел в флоат
def format_T_num_in_float(units, nano):
    if nano < 0:
        nano = abs(nano)
    if nano == 0:
        return float(units)
    return units + nano * 10**-(floor(log10(nano))+1)

# Введите данные по условию
# Тикер БА
ticker_BA = 'MAGN'
# Дата исполнения
date_execution = pd.Timestamp(2024, 12, 18)
# Дата покупки(начала пролонгирования)
date_purchase = pd.Timestamp(2024, 12, 3)
# Страйк
option_strike = 32
# Цена премии опциона
premium_price = 0.89
# Лотность опциона
lots_opt = 1
# Количество покупаемых опционов на один лот БА
number_opt = 1
# Стоимость БА на момент покупки(пролонгирования)
price_BA_date_purchase = 32.745
# Цена безубыточности опциона
break_even_price_opt = 30.93

# Запрос дневных свечей за год
async def get_history_candles(
        period, interval, figi='figi', uid='uid'
):
    try:
        async with AsyncClient(TOKEN_TIN_SanBox,
                               target=INVEST_GRPC_API_SANDBOX) as client:
            historical_candles = await client.market_data.get_candles(
                figi=figi,
                from_=now() - timedelta(days=period),
                to=now(),
                interval=interval,
                instrument_id=uid,
                candle_source_type=CandleSource.CANDLE_SOURCE_EXCHANGE
            )
            historical_candles_array = {
                'Время свечи': [x.time for x in historical_candles.candles],
                'Цена открытия': [x.open for x in historical_candles.candles],
                'Макс цена': [x.high for x in historical_candles.candles],
                'Мин цена': [x.low for x in historical_candles.candles],
                'Цена закрытия': [format_T_num_in_float(units=x.close.units, nano=x.close.nano) for x in historical_candles.candles],
                'Сформированость': [x.is_complete for x in historical_candles.candles],
                'Объём торгов': [x.volume for x in historical_candles.candles],
                # 'Тип источника': [x.candle_source.name for x in historical_candles.candles],
            }
            df = pd.DataFrame.from_dict(historical_candles_array)
            return df
    except Exception as e:
        print(
            f"Тип исключения: {type(e).__name__}, сообщение: {str(e)}"
        )
        return None
# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
# Получить опционы из таблицы
df_opt = pd.read_excel(io='all_options.xlsx', engine='openpyxl')
# Получить индекс БА по тикеру инструмента
def index_by_ticker(tiker='tiker'):
    for tik, ind in zip(df_share['Тикер'], df_share['Unnamed: 0']):
        if tik == tiker:
            return [ind, tik]
# Получить массив исторических свеч по тикеру инструмента
ind_tik_BA = (index_by_ticker(tiker=ticker_BA))
df_candles_BA = await get_history_candles(
    365, CandleInterval.CANDLE_INTERVAL_DAY, figi=df_share['FIGI'][ind_tik_BA[0]],
    uid=df_share['UID'][ind_tik_BA[0]]
)

# Цена БА последней сделки
price_BA_last_transaction = df_candles_BA['Цена закрытия'][(len(df_candles_BA['Цена закрытия']) - 1)]

# Цена неубыточности БА
break_even_price_BA = price_BA_date_purchase + (premium_price * number_opt) 

# Тема графика
curdoc().theme = "dark_minimal"
# Панель инструментов
TOOLS = "pan,wheel_zoom,box_zoom,reset, crosshair"
# Данные для изображения в hover
data_in_hover = HoverTool(
    tooltips=[
        ( 'Дата',   '@x{%F}'),
        ( 'Цена закрытия',  '@y{%0.2f} руб' ),
    ],
    formatters={
        '@x': 'datetime',
        '@y': 'printf',
    },
)

# Создание поля графика
p = figure(title=f"{df_share['Название'][ind_tik_BA[0]]} PUT {option_strike} {date_execution:%d.%m.%Y}", x_axis_type="datetime", sizing_mode="stretch_width", tools=[data_in_hover, TOOLS])
# Создание графика исторических свечей
p.line(df_candles_BA['Время свечи'], df_candles_BA['Цена закрытия'], color="blue", line_width=4, legend_label=f"БА: {df_share['Название'][ind_tik_BA[0]]}")

# Создание графика-box убыточности
# Справа
p.segment(x0=date_purchase, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_BA, legend_label="Дата покупки", line_color="gold", line_dash="8 8", line_width=2)
# Сверху
p.segment(x0=date_purchase, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_BA, legend_label="Цена неубыточности БА", line_color="mediumseagreen", line_dash="8 8", line_width=2)
# Слева
p.segment(x0=date_execution, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_opt, legend_label="Дата исполнения", line_color="navajowhite", line_dash="8 8", line_width=2)
# Снизу
p.segment(x0=date_execution, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_opt, legend_label="Цена неубыточности опциона", line_color="tomato", line_dash="8 8", line_width=2)

# Создание легенды
p.legend.title = f"Данные по опциону:\n\
Цена БА на на время покупки: {price_BA_date_purchase}\n\
Цена последней сделки БА: {price_BA_last_transaction}\n\
Динамика: \
{((price_BA_last_transaction - price_BA_date_purchase) / price_BA_date_purchase * 100):.2f}\
%\n\
Цена одного опциона: {premium_price}\n\
Лотность БА: {df_share['Лотность'][ind_tik_BA[0]]}\n\
Лотность опциона: {lots_opt}\n\
Соотношение БА к опциону: 1 к {number_opt}\n\
Цена БА неубыточности: {break_even_price_BA:.2f}"
p.legend.title_text_font_size = "16px"
p.legend.location = "top_left"
p.legend.title_text_color = 'orange'
print(break_even_price_opt)
print(break_even_price_BA)
# Активация Bokeh и вывод графика
output_notebook()
show(p)

30.93
33.635


Loading BokehJS ...

/home/sergo/TinvestOptions/venv/lib/python3.11/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


# Роснефть PUT 500 25.12

In [10]:
import pandas as pd

# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
for indx in df_share['Unnamed: 0']:
    if (df_share['Лотность'][indx] * df_share['Цена последней сделки'][indx]) < 2000:
        print(f"{df_share['Тикер'][indx]}\t{(df_share['Цена последней сделки'][indx] * df_share['Лотность'][indx]):.2f}\t{df_share['Название'][indx]}")

IRAO	375.70	Интер РАО ЕЭС
SNGSP	547.70	Сургутнефтегаз - привилегированные акции
ABIO	826.40	Артген
ASTR	504.55	Группа Астра
ROSN	437.85	Роснефть
MTLR	96.63	Мечел
TATN	548.00	Татнефть
AFKS	1281.90	АФК Система
TATNP	546.10	Татнефть - привилегированные акции
GAZP	1235.90	Газпром
VKCO	297.40	ВК
RTKM	611.50	Ростелеком
CHMF	1112.00	Северсталь
MAGN	365.45	Магнитогорский металлургический комбинат
VTBR	76.21	Банк ВТБ
RUAL	337.45	РУСАЛ
ALRS	475.20	АЛРОСА
PIKK	569.10	ПИК
AFLT	544.00	Аэрофлот
GMKN	977.40	Норильский никель
SVCB	1227.00	Совкомбанк
NLMK	1184.40	НЛМК
MTSS	1861.00	МТС
SMLT	1438.00	ГК Самолет
MOEX	1917.30	Московская Биржа
NVTK	860.20	НОВАТЭК


In [25]:
import os

import pandas as pd

from datetime import timedelta, datetime
import time

from math import floor, log10

from tinkoff.invest import AsyncClient
from tinkoff.invest.constants import INVEST_GRPC_API_SANDBOX
from tinkoff.invest.schemas import (
    Deviation,
    GetTechAnalysisRequest,
    IndicatorInterval,
    CandleInterval,
    CandleSource,
    IndicatorType,
    Smoothing,
    TypeOfPrice,
)
from tinkoff.invest.utils import decimal_to_quotation, now

from dotenv import load_dotenv

from bokeh.models import HoverTool
from bokeh.plotting import figure, show
# Активация Bokeh
from bokeh.io import output_notebook, curdoc

# Токен санбокс т-инвестиции
load_dotenv()
TOKEN_TIN_SanBox = os.getenv("TOKEN_TIN_SanBox")

# Пеобразование тин-чисел в флоат
def format_T_num_in_float(units, nano):
    if nano < 0:
        nano = abs(nano)
    if nano == 0:
        return float(units)
    return units + nano * 10**-(floor(log10(nano))+1)

# Введите данные по условию
# Тикер БА
ticker_BA = 'ROSN'
# Дата исполнения
date_execution = pd.Timestamp(2024, 12, 25)
# Дата покупки(начала пролонгирования)
date_purchase = pd.Timestamp(2024, 12, 11)
# Страйк
option_strike = 500
# Цена премии опциона
premium_price = 22.6
# Лотность опциона
lots_opt = 1
# Количество покупаемых опционов на один лот БА
number_opt = 1
# Стоимость БА на момент покупки(пролонгирования)
price_BA_date_purchase = 482.8
# Цена безубыточности опциона
break_even_price_opt = 476.72

# Запрос дневных свечей за год
async def get_history_candles(
        period, interval, figi='figi', uid='uid'
):
    try:
        async with AsyncClient(TOKEN_TIN_SanBox,
                               target=INVEST_GRPC_API_SANDBOX) as client:
            historical_candles = await client.market_data.get_candles(
                figi=figi,
                from_=now() - timedelta(days=period),
                to=now(),
                interval=interval,
                instrument_id=uid,
                candle_source_type=CandleSource.CANDLE_SOURCE_EXCHANGE
            )
            historical_candles_array = {
                'Время свечи': [x.time for x in historical_candles.candles],
                'Цена открытия': [x.open for x in historical_candles.candles],
                'Макс цена': [x.high for x in historical_candles.candles],
                'Мин цена': [x.low for x in historical_candles.candles],
                'Цена закрытия': [format_T_num_in_float(units=x.close.units, nano=x.close.nano) for x in historical_candles.candles],
                'Сформированость': [x.is_complete for x in historical_candles.candles],
                'Объём торгов': [x.volume for x in historical_candles.candles],
                # 'Тип источника': [x.candle_source.name for x in historical_candles.candles],
            }
            df = pd.DataFrame.from_dict(historical_candles_array)
            return df
    except Exception as e:
        print(
            f"Тип исключения: {type(e).__name__}, сообщение: {str(e)}"
        )
        return None
# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
# Получить опционы из таблицы
df_opt = pd.read_excel(io='all_options.xlsx', engine='openpyxl')
# Получить индекс БА по тикеру инструмента
def index_by_ticker(tiker='tiker'):
    for tik, ind in zip(df_share['Тикер'], df_share['Unnamed: 0']):
        if tik == tiker:
            return [ind, tik]
# Получить массив исторических свеч по тикеру инструмента
ind_tik_BA = (index_by_ticker(tiker=ticker_BA))
df_candles_BA = await get_history_candles(
    365, CandleInterval.CANDLE_INTERVAL_DAY, figi=df_share['FIGI'][ind_tik_BA[0]],
    uid=df_share['UID'][ind_tik_BA[0]]
)

# Цена БА последней сделки
price_BA_last_transaction = df_candles_BA['Цена закрытия'][(len(df_candles_BA['Цена закрытия']) - 1)]

# Цена неубыточности БА
break_even_price_BA = price_BA_date_purchase + (premium_price * number_opt) 

# Тема графика
curdoc().theme = "dark_minimal"
# Панель инструментов
TOOLS = "pan,wheel_zoom,box_zoom,reset, crosshair"
# Данные для изображения в hover
data_in_hover = HoverTool(
    tooltips=[
        ( 'Дата',   '@x{%F}'),
        ( 'Цена закрытия',  '@y{%0.2f} руб' ),
    ],
    formatters={
        '@x': 'datetime',
        '@y': 'printf',
    },
)

# Создание поля графика
p = figure(title=f"{df_share['Название'][ind_tik_BA[0]]} PUT {option_strike} {date_execution:%d.%m.%Y}", x_axis_type="datetime", sizing_mode="stretch_width", tools=[data_in_hover, TOOLS])
# Создание графика исторических свечей
p.line(df_candles_BA['Время свечи'], df_candles_BA['Цена закрытия'], color="blue", line_width=4, legend_label=f"БА: {df_share['Название'][ind_tik_BA[0]]}")

# Создание графика-box убыточности
# Справа
p.segment(x0=date_purchase, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_BA, legend_label="Дата покупки", line_color="gold", line_dash="8 8", line_width=2)
# Сверху
p.segment(x0=date_purchase, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_BA, legend_label="Цена неубыточности БА", line_color="mediumseagreen", line_dash="8 8", line_width=2)
# Слева
p.segment(x0=date_execution, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_opt, legend_label="Дата исполнения", line_color="navajowhite", line_dash="8 8", line_width=2)
# Снизу
p.segment(x0=date_execution, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_opt, legend_label="Цена неубыточности опциона", line_color="tomato", line_dash="8 8", line_width=2)

# Создание легенды
p.legend.title = f"Данные по опциону:\n\
Цена БА на на время покупки: {price_BA_date_purchase}\n\
Цена последней сделки БА: {price_BA_last_transaction}\n\
Динамика: \
{((price_BA_last_transaction - price_BA_date_purchase) / price_BA_date_purchase * 100):.2f}\
%\n\
Цена одного опциона: {premium_price}\n\
Лотность БА: {df_share['Лотность'][ind_tik_BA[0]]}\n\
Лотность опциона: {lots_opt}\n\
Соотношение БА к опциону: 1 к {number_opt}\n\
Цена БА неубыточности: {break_even_price_BA:.2f}"
p.legend.title_text_font_size = "16px"
p.legend.location = "top_left"
p.legend.title_text_color = 'orange'
print(break_even_price_opt)
print(break_even_price_BA)
# Активация Bokeh и вывод графика
output_notebook()
show(p)

476.72
505.40000000000003


Loading BokehJS ...

/home/sergo/TinvestOptions/venv/lib/python3.11/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


# Аэрофлот PUT 49,5 25.12

In [8]:
import pandas as pd

# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
for indx in df_share['Unnamed: 0']:
    if (df_share['Лотность'][indx] * df_share['Цена последней сделки'][indx]) < 1000:
        print(f"{df_share['Тикер'][indx]}\t{(df_share['Цена последней сделки'][indx] * df_share['Лотность'][indx]):.2f}\t{df_share['Название'][indx]}")

IRAO	375.70	Интер РАО ЕЭС
SNGSP	547.70	Сургутнефтегаз - привилегированные акции
ABIO	826.40	Артген
ASTR	504.55	Группа Астра
ROSN	437.85	Роснефть
MTLR	96.63	Мечел
TATN	548.00	Татнефть
TATNP	546.10	Татнефть - привилегированные акции
VKCO	297.40	ВК
RTKM	611.50	Ростелеком
MAGN	365.45	Магнитогорский металлургический комбинат
VTBR	76.21	Банк ВТБ
RUAL	337.45	РУСАЛ
ALRS	475.20	АЛРОСА
PIKK	569.10	ПИК
AFLT	544.00	Аэрофлот
GMKN	977.40	Норильский никель
NVTK	860.20	НОВАТЭК


In [26]:
import os

import pandas as pd

from datetime import timedelta, datetime
import time

from math import floor, log10

from tinkoff.invest import AsyncClient
from tinkoff.invest.constants import INVEST_GRPC_API_SANDBOX
from tinkoff.invest.schemas import (
    Deviation,
    GetTechAnalysisRequest,
    IndicatorInterval,
    CandleInterval,
    CandleSource,
    IndicatorType,
    Smoothing,
    TypeOfPrice,
)
from tinkoff.invest.utils import decimal_to_quotation, now

from dotenv import load_dotenv

from bokeh.models import HoverTool
from bokeh.plotting import figure, show
# Активация Bokeh
from bokeh.io import output_notebook, curdoc

# Токен санбокс т-инвестиции
load_dotenv()
TOKEN_TIN_SanBox = os.getenv("TOKEN_TIN_SanBox")

# Пеобразование тин-чисел в флоат
def format_T_num_in_float(units, nano):
    if nano < 0:
        nano = abs(nano)
    if nano == 0:
        return float(units)
    return units + nano * 10**-(floor(log10(nano))+1)

# Введите данные по условию
# Тикер БА
ticker_BA = 'AFLT'
# Дата исполнения
date_execution = pd.Timestamp(2024, 12, 25)
# Дата покупки(начала пролонгирования)
date_purchase = pd.Timestamp(2024, 12, 13)
# Страйк
option_strike = 49.5
# Цена премии опциона
premium_price = 2.39
# Лотность опциона
lots_opt = 1
# Количество покупаемых опционов на один лот БА
number_opt = 1
# Стоимость БА на момент покупки(пролонгирования)
price_BA_date_purchase = 47.97
# Цена безубыточности опциона
break_even_price_opt = 47.04

# Запрос дневных свечей за год
async def get_history_candles(
        period, interval, figi='figi', uid='uid'
):
    try:
        async with AsyncClient(TOKEN_TIN_SanBox,
                               target=INVEST_GRPC_API_SANDBOX) as client:
            historical_candles = await client.market_data.get_candles(
                figi=figi,
                from_=now() - timedelta(days=period),
                to=now(),
                interval=interval,
                instrument_id=uid,
                candle_source_type=CandleSource.CANDLE_SOURCE_EXCHANGE
            )
            historical_candles_array = {
                'Время свечи': [x.time for x in historical_candles.candles],
                'Цена открытия': [x.open for x in historical_candles.candles],
                'Макс цена': [x.high for x in historical_candles.candles],
                'Мин цена': [x.low for x in historical_candles.candles],
                'Цена закрытия': [format_T_num_in_float(units=x.close.units, nano=x.close.nano) for x in historical_candles.candles],
                'Сформированость': [x.is_complete for x in historical_candles.candles],
                'Объём торгов': [x.volume for x in historical_candles.candles],
                # 'Тип источника': [x.candle_source.name for x in historical_candles.candles],
            }
            df = pd.DataFrame.from_dict(historical_candles_array)
            return df
    except Exception as e:
        print(
            f"Тип исключения: {type(e).__name__}, сообщение: {str(e)}"
        )
        return None
# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
# Получить опционы из таблицы
df_opt = pd.read_excel(io='all_options.xlsx', engine='openpyxl')
# Получить индекс БА по тикеру инструмента
def index_by_ticker(tiker='tiker'):
    for tik, ind in zip(df_share['Тикер'], df_share['Unnamed: 0']):
        if tik == tiker:
            return [ind, tik]
# Получить массив исторических свеч по тикеру инструмента
ind_tik_BA = (index_by_ticker(tiker=ticker_BA))
df_candles_BA = await get_history_candles(
    365, CandleInterval.CANDLE_INTERVAL_DAY, figi=df_share['FIGI'][ind_tik_BA[0]],
    uid=df_share['UID'][ind_tik_BA[0]]
)

# Цена БА последней сделки
price_BA_last_transaction = df_candles_BA['Цена закрытия'][(len(df_candles_BA['Цена закрытия']) - 1)]

# Цена неубыточности БА
break_even_price_BA = price_BA_date_purchase + (premium_price * number_opt) 

# Тема графика
curdoc().theme = "dark_minimal"
# Панель инструментов
TOOLS = "pan,wheel_zoom,box_zoom,reset, crosshair"
# Данные для изображения в hover
data_in_hover = HoverTool(
    tooltips=[
        ( 'Дата',   '@x{%F}'),
        ( 'Цена закрытия',  '@y{%0.2f} руб' ),
    ],
    formatters={
        '@x': 'datetime',
        '@y': 'printf',
    },
)

# Создание поля графика
p = figure(title=f"{df_share['Название'][ind_tik_BA[0]]} PUT {option_strike} {date_execution:%d.%m.%Y}", x_axis_type="datetime", sizing_mode="stretch_width", tools=[data_in_hover, TOOLS])
# Создание графика исторических свечей
p.line(df_candles_BA['Время свечи'], df_candles_BA['Цена закрытия'], color="blue", line_width=4, legend_label=f"БА: {df_share['Название'][ind_tik_BA[0]]}")

# Создание графика-box убыточности
# Справа
p.segment(x0=date_purchase, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_BA, legend_label="Дата покупки", line_color="gold", line_dash="8 8", line_width=2)
# Сверху
p.segment(x0=date_purchase, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_BA, legend_label="Цена неубыточности БА", line_color="mediumseagreen", line_dash="8 8", line_width=2)
# Слева
p.segment(x0=date_execution, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_opt, legend_label="Дата исполнения", line_color="navajowhite", line_dash="8 8", line_width=2)
# Снизу
p.segment(x0=date_execution, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_opt, legend_label="Цена неубыточности опциона", line_color="tomato", line_dash="8 8", line_width=2)

# Создание легенды
p.legend.title = f"Данные по опциону:\n\
Цена БА на на время покупки: {price_BA_date_purchase}\n\
Цена последней сделки БА: {price_BA_last_transaction}\n\
Динамика: \
{((price_BA_last_transaction - price_BA_date_purchase) / price_BA_date_purchase * 100):.2f}\
%\n\
Цена одного опциона: {premium_price}\n\
Лотность БА: {df_share['Лотность'][ind_tik_BA[0]]}\n\
Лотность опциона: {lots_opt}\n\
Соотношение БА к опциону: 1 к {number_opt}\n\
Цена БА неубыточности: {break_even_price_BA:.2f}"
p.legend.title_text_font_size = "16px"
p.legend.location = "top_left"
p.legend.title_text_color = 'orange'
print(break_even_price_opt)
print(break_even_price_BA)
# Активация Bokeh и вывод графика
output_notebook()
show(p)

47.04
50.36


Loading BokehJS ...

/home/sergo/TinvestOptions/venv/lib/python3.11/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))


# Татнефть PUT 550 25.12

In [27]:
import os

import pandas as pd

from datetime import timedelta, datetime
import time

from math import floor, log10

from tinkoff.invest import AsyncClient
from tinkoff.invest.constants import INVEST_GRPC_API_SANDBOX
from tinkoff.invest.schemas import (
    Deviation,
    GetTechAnalysisRequest,
    IndicatorInterval,
    CandleInterval,
    CandleSource,
    IndicatorType,
    Smoothing,
    TypeOfPrice,
)
from tinkoff.invest.utils import decimal_to_quotation, now

from dotenv import load_dotenv

from bokeh.models import HoverTool
from bokeh.plotting import figure, show
# Активация Bokeh
from bokeh.io import output_notebook, curdoc

# Токен санбокс т-инвестиции
load_dotenv()
TOKEN_TIN_SanBox = os.getenv("TOKEN_TIN_SanBox")

# Пеобразование тин-чисел в флоат
def format_T_num_in_float(units, nano):
    if nano < 0:
        nano = abs(nano)
    if nano == 0:
        return float(units)
    return units + nano * 10**-(floor(log10(nano))+1)

# Введите данные по условию
# Тикер БА
ticker_BA = 'TATN'
# Дата исполнения
date_execution = pd.Timestamp(2024, 12, 25)
# Дата покупки(начала пролонгирования)
date_purchase = pd.Timestamp(2024, 12, 13)
# Страйк
option_strike = 550
# Цена премии опциона
premium_price = 9.09
# Лотность опциона
lots_opt = 1
# Количество покупаемых опционов на один лот БА
number_opt = 1
# Стоимость БА на момент покупки(пролонгирования)
price_BA_date_purchase = 567.2
# Цена безубыточности опциона
break_even_price_opt = 540.41

# Запрос дневных свечей за год
async def get_history_candles(
        period, interval, figi='figi', uid='uid'
):
    try:
        async with AsyncClient(TOKEN_TIN_SanBox,
                               target=INVEST_GRPC_API_SANDBOX) as client:
            historical_candles = await client.market_data.get_candles(
                figi=figi,
                from_=now() - timedelta(days=period),
                to=now(),
                interval=interval,
                instrument_id=uid,
                candle_source_type=CandleSource.CANDLE_SOURCE_EXCHANGE
            )
            historical_candles_array = {
                'Время свечи': [x.time for x in historical_candles.candles],
                'Цена открытия': [x.open for x in historical_candles.candles],
                'Макс цена': [x.high for x in historical_candles.candles],
                'Мин цена': [x.low for x in historical_candles.candles],
                'Цена закрытия': [format_T_num_in_float(units=x.close.units, nano=x.close.nano) for x in historical_candles.candles],
                'Сформированость': [x.is_complete for x in historical_candles.candles],
                'Объём торгов': [x.volume for x in historical_candles.candles],
                # 'Тип источника': [x.candle_source.name for x in historical_candles.candles],
            }
            df = pd.DataFrame.from_dict(historical_candles_array)
            return df
    except Exception as e:
        print(
            f"Тип исключения: {type(e).__name__}, сообщение: {str(e)}"
        )
        return None
# Получить БА из таблицы
df_share = pd.read_excel(io='all_shares.xlsx', engine='openpyxl')
# Получить опционы из таблицы
df_opt = pd.read_excel(io='all_options.xlsx', engine='openpyxl')
# Получить индекс БА по тикеру инструмента
def index_by_ticker(tiker='tiker'):
    for tik, ind in zip(df_share['Тикер'], df_share['Unnamed: 0']):
        if tik == tiker:
            return [ind, tik]
# Получить массив исторических свеч по тикеру инструмента
ind_tik_BA = (index_by_ticker(tiker=ticker_BA))
df_candles_BA = await get_history_candles(
    365, CandleInterval.CANDLE_INTERVAL_DAY, figi=df_share['FIGI'][ind_tik_BA[0]],
    uid=df_share['UID'][ind_tik_BA[0]]
)

# Цена БА последней сделки
price_BA_last_transaction = df_candles_BA['Цена закрытия'][(len(df_candles_BA['Цена закрытия']) - 1)]

# Цена неубыточности БА
break_even_price_BA = price_BA_date_purchase + (premium_price * number_opt) 

# Тема графика
curdoc().theme = "dark_minimal"
# Панель инструментов
TOOLS = "pan,wheel_zoom,box_zoom,reset, crosshair"
# Данные для изображения в hover
data_in_hover = HoverTool(
    tooltips=[
        ( 'Дата',   '@x{%F}'),
        ( 'Цена закрытия',  '@y{%0.2f} руб' ),
    ],
    formatters={
        '@x': 'datetime',
        '@y': 'printf',
    },
)

# Создание поля графика
p = figure(title=f"{df_share['Название'][ind_tik_BA[0]]} PUT {option_strike} {date_execution:%d.%m.%Y}", x_axis_type="datetime", sizing_mode="stretch_width", tools=[data_in_hover, TOOLS])
# Создание графика исторических свечей
p.line(df_candles_BA['Время свечи'], df_candles_BA['Цена закрытия'], color="blue", line_width=4, legend_label=f"БА: {df_share['Название'][ind_tik_BA[0]]}")

# Создание графика-box убыточности
# Справа
p.segment(x0=date_purchase, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_BA, legend_label="Дата покупки", line_color="gold", line_dash="8 8", line_width=2)
# Сверху
p.segment(x0=date_purchase, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_BA, legend_label="Цена неубыточности БА", line_color="mediumseagreen", line_dash="8 8", line_width=2)
# Слева
p.segment(x0=date_execution, y0=break_even_price_BA, x1=date_execution, y1=break_even_price_opt, legend_label="Дата исполнения", line_color="navajowhite", line_dash="8 8", line_width=2)
# Снизу
p.segment(x0=date_execution, y0=break_even_price_opt, x1=date_purchase, y1=break_even_price_opt, legend_label="Цена неубыточности опциона", line_color="tomato", line_dash="8 8", line_width=2)

# Создание легенды
p.legend.title = f"Данные по опциону:\n\
Цена БА на на время покупки: {price_BA_date_purchase}\n\
Цена последней сделки БА: {price_BA_last_transaction}\n\
Динамика: \
{((price_BA_last_transaction - price_BA_date_purchase) / price_BA_date_purchase * 100):.2f}\
%\n\
Цена одного опциона: {premium_price}\n\
Лотность БА: {df_share['Лотность'][ind_tik_BA[0]]}\n\
Лотность опциона: {lots_opt}\n\
Соотношение БА к опциону: 1 к {number_opt}\n\
Цена БА неубыточности: {break_even_price_BA:.2f}"
p.legend.title_text_font_size = "16px"
p.legend.location = "top_left"
p.legend.title_text_color = 'orange'
print(break_even_price_opt)
print(break_even_price_BA)
# Активация Bokeh и вывод графика
output_notebook()
show(p)

540.41
576.2900000000001


Loading BokehJS ...

/home/sergo/TinvestOptions/venv/lib/python3.11/site-packages/bokeh/util/serialization.py:242: UserWarning: no explicit representation of timezones available for np.datetime64
  return convert(array.astype("datetime64[us]"))
